## Web Search and Scrape
The **Web Search and Scrape** ant demonstrates the use of a Web API secured using an API key with an auth header. 
This sample requires you to create a Web Search (bing) resource in Azure and enter its corresponding API key in [0-AI-settings](0-AI-settings.ipynb)

This demo uses a lot of tokens. If your quota is low, you will get throttled.

## First steps
- First, download AntRunLib from Nuget
- At least once, set up the environment using **[0-AI-settings](0-AI-settings.ipynb)**

In [1]:
#r "nuget: AntRunnerLib, 0.8.8"

using AntRunnerLib;
using AntRunnerLib.Identity;
using static AntRunnerLib.ClientUtility;
using System.IO;

#!import config/Settings.cs 

var envVariables = Settings.GetEnvironmentVariables();
foreach (var kvp in envVariables)
{
    Environment.SetEnvironmentVariable(kvp.Key, kvp.Value);
}

var config = AzureOpenAiConfigFactory.Get();
var client = GetOpenAiClient(config);


Installed Packages AntRunnerLib, 0.8.8

## Ensure the Web Search and Scrape assistant exists
The definition of this assistant is located in the ".\AssistantDefinitions\Web Search and Scrape" folder

".\AssistantDefinitions\" is a default path. You can override it by setting the **ASSISTANTS_BASE_FOLDER_PATH** environment variable.

"Web Search and Scrape" contains the following files:
```
│   HtmlAgilityPack.dll - Loaded by the notebook, a great open library for c# web scraping
│   HtmlAgilityPackExtensions.dll - Loaded by the notebook, part of this project to do a simple conversion to markdown for less tokens and noise
│   HtmlAgilityPackExtensions.pdb - If you want to debug
│   instructions.md - The prompt
│   manifest.json - The assistant definition
│   
└───OpenAPI
        api.bing.microsoft.com.json - Azure Web Search (crawl operation)
        auth.json - Auth setup for web search
        tool.localhost.json - Local HtmlAgility.HtmlAgilityPackExtensions.ConvertUrlToMarkdown c# function 
```

## Crawl versus ConvertUrlToMarkdown tools

Crawl returns search results with snippets and only has a small summary or excerpt of the page

ConvertUrlToMarkdown fetches the whole page and turns it all into markdown for consumption

## auth.json
AntRunnerLib uses the instructions in this file to create the correct auth header(s) for the API. In this case, a header is added to the web request, **Ocp-Apim-Subscription-Key**, the header's value is supplied by an environment variable named, SEARCH_API_KEY. The environment variable name is user defined and ensuring it has a value is the responsibility of the application. 

```json
{
  "hosts": {
    "api.bing.microsoft.com": {
      "auth_type": "service_http",
      "header_name": "Ocp-Apim-Subscription-Key",
      "header_value_env_var": "SEARCH_API_KEY"
    }
  }
}
```

### Explanation
`AssistantUtility.Create` acts based on the contents of a folder structure following a convention laid out in these notebooks. In this case, it copied the contents of instructions.md into the assistant manifest.json. It also parsed the api.bing.microsoft.com swagger which follows an Open API schema into tool defintions for the assitant.

In [2]:
var assistantId = await AssistantUtility.Create("Web Search and Scrape", config);
Console.WriteLine(assistantId)

asst_pf8svGQ6wOmKpEOigTKwyk6i


In [3]:
AntLoader.LoadAssembly(@".\AssistantDefinitions\Web Search and Scrape\HtmlAgilityPack.dll");
AntLoader.LoadAssembly(@".\AssistantDefinitions\Web Search and Scrape\HtmlAgilityPackExtensions.dll");

## Run the Assistant

`output.Dialog` shows the conversation.

In [4]:
var assistantRunOptions = new AssistantRunOptions() {
    AssistantName = "Web Search and Scrape",
    Instructions = "Use python to get todays date and then use it to find things to do in Atlanta today. Be careful to not include events in the past in your final answer.",
};
var output = await AntRunnerLib.AssistantRunner.RunThread(assistantRunOptions, config);
output.Dialog.DisplayAs("text/markdown")


User: Use python to get todays date and then use it to find things to do in Atlanta today. Be careful to not include events in the past in your final answer.

Assistant: I ran this code:
```
from datetime import datetime

# Get today's date
today_date = datetime.today().strftime('%Y-%m-%d')
today_date
```

I called the tool named crawl with these arguments:
```
{
  "q": "things to do in Atlanta on September 25, 2024",
  "textFormat": "HTML"
}
```
and got this result:
```
{"queryContext":{"originalQuery":"things to do in Atlanta on September 25, 2024"},"webPages":{"value":[{"name":"Cool Things To Do in September in Atlanta","url":"https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-september-in-atlanta/","displayUrl":"https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-september-in-atlanta","snippet":"The Sweet Auburn Music Fest, which takes place Sept. 28-29. The festival, located on Auburn Avenue, one of Atlanta\u2019s most iconic streets, offers fun-filled entertainment, live music \u2014 everything from R\u0026amp;B to gospel to alternative favorites \u2014 international food court, national artists, celebrities and vendors of all types.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002Bon\u002BSeptember\u002B25%2c\u002B2024\u0026d=4692574352310340\u0026mkt=en-US\u0026setlang=en-US\u0026w=c-5Oucqqe-FV74GkIkdoRE4k4vk9MNbJ","thumbnailUrl":"https://www.bing.com/th?id=OIP.m0ENCio7ytS2o9PQRWDCRgHaD4\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Atlanta Events September 2024: Concerts, Shows, Sports...","url":"https://atlanta-ga.events/september/","displayUrl":"https://atlanta-ga.events/september","snippet":"Check Out The Atlanta Events Calendar for September 2024. See the List Of All Current \u0026amp; Upcoming Events at the Lowest Possible Price. ... Sep 25, 2024 6:30 PM . Lorna Shore . Upcoming Events: 2. The Tabernacle - GA . Sep 25, 2024 6:30 PM ...","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002Bon\u002BSeptember\u002B25%2c\u002B2024\u0026d=4854189676169673\u0026mkt=en-US\u0026setlang=en-US\u0026w=73iRaVHIvJefbkmdL1lCvH4mS1bjztOY"},{"name":"Atlanta Events - September 2024 - Go South Atlanta","url":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-september.html","displayUrl":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-september.html","snippet":"More details. \u2013 Dragon Con, August 29 - September 2 2024. One of the biggest annual pop culture conventions in the country. More details. \u2013 Atlanta Black Theatre Festival, August 30 - September 1 2024. Annual festival celebrating the Black experience in the performing arts, with plays, panels, masterclasses and other events.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002Bon\u002BSeptember\u002B25%2c\u002B2024\u0026d=4712253893574850\u0026mkt=en-US\u0026setlang=en-US\u0026w=aWkJWqw243Je4oOvzq1dGpeg_GE4sHjt","thumbnailUrl":"https://www.bing.com/th?id=OIP.xm-3dn3hYJZHsVVU9WO5qwHaD4\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"20 Spectacular Things To Do This September In Atlanta - Secret Atlanta","url":"https://secretatlanta.co/things-to-do-september/","displayUrl":"https://secretatlanta.co/things-to-do-september","snippet":"There\u0026#39;s so much going on in Atlanta this September, so check out this list of things to do to celebrate the end of summer \u0026amp; start of fall! ... When: September 5th \u2013 October 27th, 2024. Where: Helen Festhalle | 1074 Edelweiss Strasse, Helen, GA 30545. ... See also: This Atlanta Restaurant Is One Of The Top 25 \u2018Best New Restaurants In The ...","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002Bon\u002BSeptember\u002B25%2c\u002B2024\u0026d=4724958405090999\u0026mkt=en-US\u0026setlang=en-US\u0026w=HkmWmkX0emVkqeBkOazQ7cCeg5xgwMQ3","thumbnailUrl":"https://www.bing.com/th?id=OIP.mwxBnPRtbggmsDGaNhGACQHaE8\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"80\u002B fall festivals \u0026amp; fairs around metro Atlanta: Fun things to do in ...","url":"https://www.atlantaonthecheap.com/fall-festivals-atlanta/","displayUrl":"https://www.atlantaonthecheap.com/fall-festivals-atlanta","snippet":"\u2666 Atlanta Streets Alive. September 15, 2024 West End to Grant Park FREE. This is a new route for Fall of 2024! On these Sundays, streets are are temporarily closed to traffic for four hours \u2014 so you can come out to walk, run, stroll, bike, scoot, roll, and just have fun.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002Bon\u002BSeptember\u002B25%2c\u002B2024\u0026d=5012794228946274\u0026mkt=en-US\u0026setlang=en-US\u0026w=AqqJM900-WHnjjGt9vluoyq1DPm6ofe-"},{"name":"The Best Things to Do in Atlanta in September","url":"https://unexpectedatlanta.com/the-best-things-to-do-in-atlanta-in-september/","displayUrl":"https://unexpectedatlanta.com/the-best-things-to-do-in-atlanta-in-september","snippet":"Atlanta Events: September 2024 Experience the Best of Atlanta\u2019s Food \u0026amp; History. Try some of the best eats in Atlanta! The #1-rated Grant Park Past \u0026amp; Future Food Walk is a fun walking tour showcasing the best of Atlanta\u2019s food and history. On this tour, you\u2019ll taste eight modern and Southern foods from three outstanding locally owned food stops.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002Bon\u002BSeptember\u002B25%2c\u002B2024\u0026d=4781604727099753\u0026mkt=en-US\u0026setlang=en-US\u0026w=X_Y32Shq0Tl-bHwGvtgeUoQiW0MCD0-O","thumbnailUrl":"https://www.bing.com/th?id=OIP.BV6NvB-FhOL3LSCX1dcUJwHaHa\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"September in Atlanta: Top things to do this month","url":"https://www.atlantaonthecheap.com/september-things-to-do-atlanta/","displayUrl":"https://www.atlantaonthecheap.com/september-things-to-do-atlanta","snippet":"Details on Museums On Us. \u2022 The Marietta History Center hosts its FREE monthly Family Fun Day on Saturday, September 21, 2024. \u2022 The Carter Presidential Library \u0026amp; Museum offers FREE admission on Saturday, September 28, 2024, in honor of the former president\u2019s 100th birthday!","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002Bon\u002BSeptember\u002B25%2c\u002B2024\u0026d=4531508777067252\u0026mkt=en-US\u0026setlang=en-US\u0026w=dBvGHC3HqGEMsXsP8SfKEFE-tgkhrh_1","thumbnailUrl":"https://www.bing.com/th?id=OIP.87SMQKoDZPtIKmUHIBiCMQHaFj\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Explore Atlanta Events for Today, This Week or Weekend - Discover Atlanta","url":"https://discoveratlanta.com/events/main/","displayUrl":"https://discoveratlanta.com/events/m","snippet":"Explore the Atlanta events calendar for things to do today, this week or this weekend. Find upcoming festivals, seasonal events and more! ... Every Wednesday until 09/25/2024 ... September 25. Wednesday. BEAT: Steve Vai, Adrian Belew, Tony Levin, Danny Carey 9/25/24. September 25.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002Bon\u002BSeptember\u002B25%2c\u002B2024\u0026d=4819881473496114\u0026mkt=en-US\u0026setlang=en-US\u0026w=zvJBiG0Dg4eP1GMx_bJv9FTM8ziERJGY","thumbnailUrl":"https://www.bing.com/th?id=OIP.FkxSV6JbrWatsFMM3_8dIAHaEh\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"10 things to do in Atlanta this weekend - accessatlanta.com","url":"https://www.accessatlanta.com/atlanta-things-to-do/10-things-to-do-in-atlanta-this-weekend/RXJV557GDFFJVMY3LMAOYQTNYI/","displayUrl":"https://www.accessatlanta.com/atlanta-things-to-do/10-things-to-do-in-atlanta-this...","snippet":"Location: Kirkwood - Hosea L Williams Dr NE; Date: September 20; Time: 7-10 p.m.; Admission: $70; Website: kirkwoodwinestroll.com Revolt World. This 3-day immersive experience features appearances ...","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002Bon\u002BSeptember\u002B25%2c\u002B2024\u0026d=2036019664925\u0026mkt=en-US\u0026setlang=en-US\u0026w=ecYZf3pOPl4MgsqlUNgwD_67J-TONvpM"},{"name":"35\u002B Best Things to Do in Atlanta in 2024 - Unexpected Atlanta","url":"https://unexpectedatlanta.com/best-things-to-do-in-atlanta/","displayUrl":"https://unexpectedatlanta.com/best-things-to-do-in-atlanta","snippet":"1. Visit the National Parks on a Fee-Free Day. First on our list of the best things to do in Atlanta in 2024, check out America\u2019s National Park System\u2019s fee-free days this year. On these special days, you can visit any of the United States\u2019 400\u002B National Parks for free, including places in and around Georgia.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002Bon\u002BSeptember\u002B25%2c\u002B2024\u0026d=4885792044096348\u0026mkt=en-US\u0026setlang=en-US\u0026w=SRiegEmhRGKtzRRUG028yHS95G2gOIKx","thumbnailUrl":"https://www.bing.com/th?id=OIP.HwzfdFE6-tSHfhoFlTxXKQHaHa\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"}]}}
```

I called the tool named GetContentFromUrl with these arguments:
```
{"url": "https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-september-in-atlanta/"}
```
and got this result:
```
"Cool Things To Do in September in Atlanta - Discover Atlanta[DiscoverAtlanta.com homepage](https://discoveratlanta.com)* Things to Do\r\n* Events\r\n* Plan Your Visit\r\n* Meetings\r\n\r\nMenu* Discover Atlanta Take a Tour Neighborhoods Atlanta\u2019s Food Story Michelin Restaurants Black Travel Guide Local Guides LGBTQ Atlanta\u2019s Stories\r\n* Things to Do 50 Fun Things To Do Attractions Eat Arts \u0026#038; Culture Outdoors History \u0026#038; Heritage Shopping Nightlife Sports This Weekend\r\n* Events Festivals Virtual Events Seasonal \u0026#038; Holidays Submit Your Event\r\n* Plan Your Visit Stay Deals Itineraries Visitor Guide Getting Around View All Hotels\r\n\r\n* Meeting Planners Why Atlanta New Developments Convenient Travel Getting Around Meeting Districts Unique Facilities \u0026#038; Venues Key Industries Plan a Meeting Facilities \u0026#038; Venues Hotels Suppliers Submit Request for Proposal (RFP) Destination Services Community Engagement Promote A Meeting Planner\u2019s Timeline City Guides \u0026amp; Brochures Atlanta\u2019s Meetings Stories Contact Us \u2013 Sales\r\n* Travel Pros Tour Services Contact Us\r\n* Newsroom\r\n* Partners Employment Contact Us Become a Member Publications Directions \u0026#038; Parking\r\n\r\nCheck out these storiesLoading# Cool Things To Do in September in Atlanta\r\n\r\nBy Mary Welch\r\n\r\nPublished August 23, 2024Last Updated September 11, 2024\r\n\r\nIt\u2019s September. In Atlanta that means it\u2019s time for festivals, costumed aliens walking down Peachtree Street and food \u2014 glorious food.\r\n\r\n## Food Extravaganza\r\n\r\n![](https://res.cloudinary.com/atlanta/images/w_1024,h_683,c_scale/v1723648163/newAtlanta.com/ATL-food-Credit-raftermen-photography-copy/ATL-food-Credit-raftermen-photography-copy.jpg?_i=AA)\r\nTake a big bite. There is plenty to sample at the Atlanta Food \u0026#038; Wine Festival with more than 200 chefs, sommeliers and mixologists. (Photo by Raftermen Photography)The Atlanta Food \u0026amp; Wine Festival returns to Historic Old Fourth Ward Park for a four-day celebration of incomparable food, wine, beer and craft spirits. More than 200 of the most celebrated chefs, sommeliers, mixologists and industry insiders will offer bites and drinks that showcase the exceptional flavors of the region. The festival runs Sept. 12-15.\r\n\r\n## Burger Bash\r\n\r\n![](https://res.cloudinary.com/atlanta/images/w_1024,h_683,c_scale/v1723647613/newAtlanta.com/Burger-benefit-resized/Burger-benefit-resized.jpg?_i=AA)\r\nThe team from Fox Bros. BBQ takes to the grill to showcase their burger skills at the Burger Benefit Sept. 22. (Photo courtesy of NFA Burger)Some of the city\u2019s and the nation\u2019s leading chefs including personality Andrew Zimmern; Shai Livi of The Third Space; Billy Kramer of NFA Burger; and Nick Leahy of Vice Kitchen will duke it out for an all-day burger showdown at NFA Burger in Chamblee. The Burger Benefit takes place Sept. 22 and features four burger masters commanding the grill in shifts of 90 minutes. Last year the burger benefit raised more than $53,000 in support of The Giving Kitchen, an Atlanta-based nonprofit for hospitality workers. Tickets start at $30, which includes one burger and a side.\r\n\r\n## Spooky Time\r\n\r\n![](https://res.cloudinary.com/atlanta/images/w_684,h_1024,c_scale/v1723647650/newAtlanta.com/NeteherworldMortis-copy/NeteherworldMortis-copy.jpg?_i=AA)\r\nNecrotis is just one of the friendly characters waiting to greet \u2014 and scare \u2014 you at NETHERWORLD. (Photo courtesy of NETHERWORLD)As the haunt season approaches, NETHERWORLD is opening on weekends starting Sept. 20 and 21 and will continue for 34 nights of terror through September, October and November. For its 28th Season of Screams, the attraction is offering two terrifying new haunts, Wake the Dead and Mr. Grendel\u2019s Birthday Party of Horrors. Also returning is Dr. Octane\u2019s Bizarre Beverages featuring colorful, bubbling nonalcoholic drinks served in an optional collectible light-up glass. There\u2019s no shortage of Halloween things to do in Atlanta.\r\n\r\n## Harvest on the Hooch\r\n\r\n![](https://res.cloudinary.com/atlanta/images/w_790,h_1024,c_scale/v1723648296/newAtlanta.com/Harvest-on-the-hooch/Harvest-on-the-hooch.jpg?_i=AA)\r\nEnjoy getting together with friends at the ninth annual Harvest on the Hooch at Chattahoochee Nature Center. (Photo courtesy of Chattahoochee Nature Center)The Chattahoochee Nature Center is hosting the ninth annual Harvest on the Hooch Sept. 22 to raise awareness for the Nature Center\u2019s Unity Garden, which has supplied more than 50 tons of fresh produce to North Fulton Community Charities food pantry. The event will feature food tastings from local chefs, live bluegrass music, garden games, samples from local craft breweries, face painting, and meet and greets with animals.\r\n\r\n## Hispanic Heritage Month\r\n\r\n![](https://res.cloudinary.com/atlanta/images/w_1024,h_683,c_scale/v1723647690/newAtlanta.com/hispanic-festival/hispanic-festival.jpg?_i=AA)\r\nEnjoy a cultural experience Sept. 17 for the Fiestas Patrias, celebrating Mexican Independence.Plaza Fiesta on Buford Highway will host a Fiestas Patrias celebration Sept. 17. The Mexican Consulate\u2019s \u201CEl Grito \u2014 Mexican Independence Celebration\u201D has been a local favorite for more than 19 years. Last year more than 60,000 people celebrated. Plan to go and enjoy a cultural experience that embraces heritage, entertainment and community. Celebrate Hispanic Heritage Month around Atlanta.\r\n\r\n## Baldie Con\r\n\r\n![](https://res.cloudinary.com/atlanta/images/w_668,h_1024,c_scale/v1724428012/newAtlanta.com/Baldie-Con-flyer/Baldie-Con-flyer.jpeg?_i=AA)\r\nBaldie Con is an empowerment conference for the bald community.Happening at the JW Marriott Atlanta Buckhead, Baldie Con\u0026nbsp;celebrates, empowers and unites women and children who are bald (Baldies) from alopecia, lupus, cancer or by choice. The three-day conference starts with a meet and greet on Sept. 27. There will be a Baldie Ball with an award ceremony and gala dinner on Saturday night. The weekend wraps up on Sept. 29 with a Sunday Jazz Brunch.\r\n\r\n## It\u2019s so Sweet\r\n\r\n![](https://res.cloudinary.com/atlanta/images/w_1024,h_442,c_scale/v1602355972/newAtlanta.com/Zz05MmIyZjQyZWNhOWYxMWVhYjNjMTBhODU2ZjA4Y2UwNw-scaled/Zz05MmIyZjQyZWNhOWYxMWVhYjNjMTBhODU2ZjA4Y2UwNw-scaled.jpg?_i=AA)\r\nAuburn Avenue is where you\u2019ll find the Sweet Auburn Music Fest.The Sweet Auburn Music Fest, which takes place Sept. 28-29. The festival, located on Auburn Avenue, one of Atlanta\u2019s most iconic streets, offers fun-filled entertainment, live music \u2014 everything from R\u0026amp;B to gospel to alternative favorites \u2014 international food court, national artists, celebrities and vendors of all types.\r\n\r\n## Forward Warrior\r\n\r\nThe annual Forward Warrior live art event is happening all day, 9 a.m. until 9 p.m. Saturday, on Wylie Street in Cabbagetown.\r\n\r\n## Celebrate a President\r\n\r\n![](https://res.cloudinary.com/atlanta/images/w_1024,h_683,c_scale/v1723649158/newAtlanta.com/FoxATL2017_Concert_ALIVECOVERAGE_27323552fb/FoxATL2017_Concert_ALIVECOVERAGE_27323552fb.jpg?_i=AA)\r\nThe Fox Theatre plans a musical extravaganza to celebrate President Carter\u2019s 100th birthday. (Photo courtesy of Fox Theatre)Jimmy Carter turns 100 on Oct. 1, and the Fox Theatre is celebrating in a big way. Presented by Delta Air Lines, a variety of artists including Chuck Leavell, a Georgia resident and keyboardist for the Rolling Stones, D-Nice, Eric Church, Aren Morris and the Atlanta Symphony Orchestra Chamber Chorus will take the stage Sept. 17 to present a musical extravaganza. The event is a fund-raiser for the Carter Center.\r\n\r\n## Sandy Springs Salutes Creativity\r\n\r\n![](https://res.cloudinary.com/atlanta/images/w_1024,h_658,c_scale/v1723648070/newAtlanta.com/sandy-springs-arts-festival/sandy-springs-arts-festival.jpg?_i=AA)\r\nA couple takes a break from the Sandy Springs Arts Festival to enjoy people watching. (Photo courtesy of Caren West PR)The Sandy Springs Fall Arts Festival returns on Sept. 14-15, featuring more than 150 painters, photographers, sculptors, leather and metal craft persons, glass blowers, jewelers and more. Here are more September (and fall) festivals to visit.\r\n\r\nRead More* Cool Things to Do in November in Atlanta\r\n* Cool Things To Do in October in Atlanta\r\n* Cool Things To Do in August in Atlanta\r\n\r\n## Dragon Con Takes Over\r\n\r\n![](https://res.cloudinary.com/atlanta/images/v1723647734/newAtlanta.com/Media_Relations23-1024x682-1/Media_Relations23-1024x682-1.jpg?_i=AA)\r\nYou never know who you\u2019ll meet on Peachtree Street during Dragon Con. (Photo courtesy of Dragon Con Photography \u00C2\u00A9 Dragon Con, Inc.)Dragon Con, the worlds largest multimedia, popular culture convention focusing on science fiction and fantasy, gaming, comics, literature, art, music and film in the universe comes to Atlanta and, frankly, takes over the city. Walk the streets and meet thousands of costumed folks participating in the workshops, meet and greets with celebrities, the art show, and comic and pop artist alley. But, the biggest fun comes with the parade down Peachtree Street on Aug. 31. Among the celebrities appearing at Dragon Con, which takes place Aug. 29-Sept. 2, are: Gil Gerard, Erin Grey, Seth Green, Greg Grunberg, Cobie Smulders and Ming-Na Wen.\r\n\r\n## Piedmont Park Celebrates 120 Years\r\n\r\n![](https://res.cloudinary.com/atlanta/images/w_1024,h_1024,c_scale/v1725563202/newAtlanta.com/P4PP-IG-Post/P4PP-IG-Post.png?_i=AA)\r\nParty like it\u2019s 1989 as Piedmont Parks celebrates its 120th birthday and the 35th anniversary of the Piedmont Park Conservatory! The nostalgia-themed party will have food from local restaurants, cocktails, a silent auction and other throwback vibes. All proceeds will support the Piedmont Park Conservatory and its ongoing work to maintain and enhance Atlanta\u2019s most visited public park. Find more information about the event here.\r\n\r\n## College Football is Back\r\n\r\nSeptember, of course, heralds the return of college football season. No matter your alma mater, there are bars all around town, which may show your team\u2019s games, which means you can cheer along with like-minded fans. Find out Where to Watch College Football in Atlanta.\r\n\r\n## Gone Fishin\u2019\r\n\r\nSummer is winding down so it\u2019s the perfect time to drop a line and enjoy a day of fishing. There are several places to fish including Jones Creek Park, Lullwater Park, the Chattahoochee River National Recreation Area, Morgan Falls Overlook Park, Sweetwater Creek State Park, Piedmont Park, Murphey Candler Park and Lake Lanier\r\n\r\n## Try an Omakase Restaurant\r\n\r\n![](https://res.cloudinary.com/atlanta/images/w_1024,h_683,c_scale/v1723647780/newAtlanta.com/Mujo-Andrew-Thomas-Lee-resized/Mujo-Andrew-Thomas-Lee-resized.jpg?_i=AA)\r\nMuj\u00C5\u008D is one of two Atlanta omakase restaurants to receive a MICHELIN star, but there are several other restaurants that offer this unique culinary experience. (Photo of Muj\u00C5\u008D by Andrew Thomas Lee)Omakase is a Japanese dining experience where guests leave the menu in the hands of the chef and receive a seasonal, creative and unforgettable dinner with a progression of small courses. Atlanta has several omakase restaurants, and two \u2014 Muj\u014D and Hayakawa \u2014 were awarded MICHELIN STARS. But Atlanta has several others that offer amazing experiences as well. Try Omakase Table (West Marietta Street); Nagomiya (West Peachtree Street); Saito (Andrew Young International Boulevard); Taka Sushi (Roswell Road); MF Sushi (North Highland Avenue); Chirori (14th Street); Umi (Peachtree Road); and Brush (Peachtree Road). Brush, in fact, just started a \u201Crestaurant-within-a restaurant\u201D concept by offering a chef-driven sushi omakase-only experience in addition to a regular dining experience.\r\n\r\n## News for Kids\r\n\r\nAtlanta\u2019s must-see attractions are stretching your summer fun \u2013 and dollars \u2013 with a September-only promotion for families. Kids free admission this September is available at most attractions and all offer exciting experiences.\u0026nbsp;Plan your Kids Free September Atlanta discovery outing today. All offers start after Labor Day (Sept. 3-30).\r\n\r\n## Yellow Daisy Festival\r\n\r\nVoted one of the top arts and crafts shows in the nation by Sunshine Artist magazine, the Yellow Daisy Festival at Stone Mountain Park draws more than 400 artists and crafters as well as food vendors, crafter demonstrations, live music and a beer garden. The festival, which takes place Sept. 5-8, is great family fun \u2014 and you can get a head start on holiday shopping.\r\n\r\n## Buckhead Arts Festival\r\n\r\n![](https://res.cloudinary.com/atlanta/images/w_1024,h_819,c_scale/v1723647966/newAtlanta.com/Buckhead-arts-festival/Buckhead-arts-festival.jpg?_i=AA)\r\nThe Buckhead Fine Arts Festival is a great way to sip, snack and shop in the heart of Buckhead. (Photo courtesy of Caren West PR.)Sip, shop and stroll in the heart of Buckhead at the Buckhead Fine Arts Festival Sept. 21-22. Presented by the Atlanta Foundation for Public Spaces, the outdoor event will feature more than 100 painters, photographers, sculptors, metalworkers, glass artists, jewelers and more. Enjoy a variety of food and beverage options from vendors and merchants in Buckhead Village.\r\n\r\n## Inaugural Street Festival\r\n\r\nHosted by Blacks in Technology Foundation, the Atlanta Street Food Festival \u0026amp; Bazaar takes place Sept. 2 at Westside Park, Atlanta\u2019s largest park. The first-time festival is a family-friendly event with music, interactive demos and other entertainment. There\u2019s even a dedicated children\u2019s zone with games, activities and entertainment. Come hungry because it is a gastronomic adventure featuring an array of foods from the city\u2019s finest chefs, food trucks and local eateries. There will be everything from barbecue to gourmet burgers to vegan delicacies.\r\n\r\n## Opera in the Yards\r\n\r\n![](https://res.cloudinary.com/atlanta/images/w_1024,h_682,c_scale/v1723647893/newAtlanta.com/pullman-credit-Smily-Eyes-Media/pullman-credit-Smily-Eyes-Media.jpg?_i=AA)\r\nYes, the Atlanta Opera performs at Pullman Yards. (Photo by Smile Eyes Media)The Atlanta Opera Discoveries series continues the \u201CBoh\u00E8me Project\u201D with two immersive productions at Pullman Yards. Produced by\u0026nbsp;Tomer Zvulun, the project comprises repertory performances of a modern-day\u0026nbsp;\u201CLa boh\u00E8me\u201D \u2014 with the COVID-19 pandemic in place of tuberculosis \u2014 with the Broadway show it inspired,\u0026nbsp;\u201CRent,\u201D which updated Puccini\u2019s story of friendship, passion and art by setting it in the midst of the 1990\u2019s HIV/AIDS crisis. Both performed on the same set, with the action taking place in and around the seated audience, the productions will feature different casts, and musical forces and will alternate nights for most of the run.\r\n\r\n## JapanFest\r\n\r\nCelebrate 50 years of culture, friendship and cultural exchange between Japan and Georgia with JapanFest Sept. 21-22 at Gas South Convention Center in Duluth. Enjoy the mesmerizing beauty of traditional Japanese Rakugo, Kabuki, taiko drumming, martial arts demonstration and workshops in bonsai, calligraphy, ikebana flower design and Kimono fitting. Don\u2019t miss the Japanese Food Court where local restaurants will offer bento boxes, rice bowls, ramen noodles, sushi, yahisoba and shaved ice. In addition there will be three traditional Japanese craftsman giving demonstrations in stone carving, Kumihimo braided cords and tatami weaving.\r\n\r\n## PGA\u2019s TOUR Championship Returns\r\n\r\n![](https://res.cloudinary.com/atlanta/images/w_1024,h_682,c_scale/v1685605842/newAtlanta.com/18_TTC_0921_26792_CCp_cc_20604e9ab6/18_TTC_0921_26792_CCp_cc_20604e9ab6.jpg?_i=AA)\r\nFans of professional golf should head to East Lake Golf Club late August through Sept. 1 for the PGA TOUR Championship.East Lake Golf Club hosts the PGA\u2019s TOUR Championship Aug. 29 through Sept. 1.\r\n\r\nAlways keep an eye on our calendar of events for even more things to do in Atlanta.\r\n\r\n![](https://discoveratlanta.com/wp-content/themes/atlanta/assets/images/pixel.png)\r\nJournalist Mary Welch writes business and lifestyle stories for local and national publications.\r\n\r\n## Discover More\r\n\r\nGet to know Atlanta, and discover the best things to do around the city.\r\n\r\n[Arts \u0026amp; CultureOctober brings lots of cool things to do in Atlanta, and Halloween is just one. Cool Things To Do in October in Atlanta](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-october-in-atlanta/)[Arts \u0026amp; CultureIn May, Atlantans head outdoors to enjoy the sun, flowers and the diversity of things to do. Cool Things To Do in May in Atlanta](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-may-in-atlanta/)[Things to doFrom end of summer festivals to new exhibits at the Atlanta History Center, discover cool things to do in August in Atlanta. Cool Things To Do in August in Atlanta](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-august-in-atlanta/)[Arts \u0026amp; CultureWe\u2019ll admit it can get hot in Atlanta during July, so here are some cool things to do. Cool Things To Do in July in Atlanta](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-july-in-atlanta/)[Arts \u0026amp; CultureEnjoy all that Atlanta has to offer during the awesome month of April. Cool Things To Do in April in Atlanta](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-april-in-atlanta/)[AttractionsNovember in Atlanta brings popular festivals, a new Cirque du Soleil show and the beginning of holiday things to do. Cool Things to Do in November in Atlanta](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-november-in-atlanta/)## Sign up for email updates\r\n\r\nEmail AddressZip Code* Discover Atlanta\r\n* Things to Do\r\n* Events\r\n* Plan Your Visit\r\n* Meeting Planners\r\n* Travel Pros\r\n* Newsroom\r\n* Partners\r\n* About Us\r\n* Become a Member\r\n* Employment\r\n* Contact Us\r\n\r\n[DiscoverAtlanta.com homepage](https://discoveratlanta.com)Established in 1913, Atlanta Convention \u0026 Visitors Bureau is the official destination marketing organization for the city and serves to favorably impact Atlanta\u0027s economy through conventions and tourism.\r\n\r\n## Atlanta Convention \u0026 Visitors Bureau\r\n\r\n233 Peachtree Street Suite #1400 Atlanta, GA 30303\r\n\r\n(800)ATLANTA (404)521-6600\r\n\r\nSelect site languageSelect languageEnglishEspa\u00F1olFran\u00E7aisDeutschPortugu\u00EAs\u4E2D\u6587Submit[](https://www.facebook.com/discoveratlanta)[](https://twitter.com/#!/discoveratlanta)[](https://www.instagram.com/discoveratlanta)[](https://www.youtube.com/visitatlanta)[](https://www.pinterest.com/discoveratlanta/)* Site Map\r\n* Privacy Policy\r\n* Terms of Use\r\n* UGC Agreement\r\n\r\nAtlanta Convention \u0026 Visitors Bureau.\r\n\r\nAll Rights Reserved. Copyright \u00A92024\r\n\r\n![](/wp-content/themes/atlanta/assets/images/wtp-icons/PlanYourTrip.png)\r\n![](/wp-content/themes/atlanta/assets/images/wtp-icons/close-btn.png)\r\n"
```

I called the tool named GetContentFromUrl with these arguments:
```
{"url": "https://atlanta-ga.events/september/"}
```
and got this result:
```
"Atlanta Events September 2024: Concerts, Shows, Sports...JavaScript support required!\r\n\r\n[](https://atlanta-ga.events)\u00D7Categories[All Events](https://atlanta-ga.events/)[Concerts](https://atlanta-ga.events/concerts/)[Shows](https://atlanta-ga.events/shows/)[Comedy](https://atlanta-ga.events/comedy-shows/)[Sports](https://atlanta-ga.events/sports/)[Family Shows](https://atlanta-ga.events/family-shows/)[Festivals](https://atlanta-ga.events/festivals/)[Opera](https://atlanta-ga.events/opera-tickets/)[Venues](https://atlanta-ga.events/entertainment-venues/)* All Events\r\n* Concerts\r\n* Shows\r\n* Comedy\r\n* Sports\r\n* More Family Shows Festivals Opera Venues\r\n\r\n* Home\r\n* September\r\n\r\n# Atlanta Events September 2024\r\n\r\nExplore our comprehensive calendar for all the current and upcoming events in Atlanta, GA, for September 2024. This guide is your gateway to discovering the best things to do in Atlanta, from Concerts and Theater Shows to Sports events and music festivals. You\u2019ll find a detailed list of all the most engaging and exciting activities happening throughout the month. Easily search and select your preferred events in Atlanta with our user-friendly calendar. Secure your tickets with 100% assurance at the lowest prices.\r\n\r\nStart DateSep 25, 2024ResetCancelApplyEnd DateSep 27, 2024CategoryAll EventsConcertsTheaterSportsSearchContentsUpcoming EventsList Of All EventsAtlanta, GA## Upcoming Events \u0026 Tickets\r\n\r\nSelect Your DatesResetCancelApplyUpcoming Events: 108* Sep 25 2024 6:30 PM Wed Lorna Shore The Tabernacle - GA | Capacity: 2600 30303, 152 Luckie St, Atlanta, GA, US Prices from $20 Avg. price ~ $89 Available Tickets: 81 Tickets\r\n* Sep 25 2024 6:30 PM Wed Sevendust The Eastern - GA 30312, 800 Old Flat Shoals Road SE, Atlanta, GA, US Prices from $39 Avg. price ~ $61 21 tickets remaining! Tickets\r\n* Sep 25 2024 7:00 PM Wed Creed, 3 Doors Down \u0026 Finger Eleven Ameris Bank Amphitheatre | Capacity: 12000 30004, 2200 Encore Parkway, Alpharetta, GA, US Prices from $39 Avg. price ~ $223 Available Tickets: 223 Tickets\r\n* Sep 25 2024 7:20 PM Wed Atlanta Braves vs. New York Mets Truist Park | Capacity: 41149 30339, 1100 Circle 75 Pkwy, Atlanta, GA, US Prices from $7 Avg. price ~ $75 Available Tickets: 1977 Tickets\r\n* Sep 25 2024 7:30 PM Wed Kings of Leon \u0026 Phantogram State Farm Arena - GA | Capacity: 21000 30303, One Philips Drive, Atlanta, GA, US Prices from $14 Avg. price ~ $123 Available Tickets: 804 Tickets\r\n* Sep 25 2024 8:00 PM Wed Shaboozey Variety Playhouse | Capacity: 1000 30307, 1099 Euclid Ave, Atlanta, GA, US Prices from $51 Avg. price ~ $88 28 tickets remaining! Tickets\r\n* Sep 25 2024 8:00 PM Wed Lasso - Artist Buckhead Theatre | Capacity: 2500 30305, 3110 Roswell Road, Atlanta, GA, US Prices from $58 Avg. price ~ $90 Available Tickets: 54 Tickets\r\n* Sep 25 2024 8:00 PM Wed Gritz and Jelly Butter City Winery - Atlanta | Capacity: 300 30308, 675 Ponce De Leon Ave, Atlanta, GA, US Prices from $74 Avg. price ~ $89 8 tickets remaining! Tickets\r\n* Sep 26 2024 6:30 PM Thu Rain City Drive Hell at The Masquerade - GA | Capacity: 300 30303, 75 MLK Jr Dr SW, Atlanta, GA, US Prices from $34 Avg. price ~ $40 22 tickets remaining! Tickets\r\n* Sep 26 2024 7:00 PM Thu Mico Purgatory Stage at Masquerade - GA | Capacity: 300 30303, 75 MLK Jr Dr SW, Atlanta, GA, US Prices from $37 Avg. price ~ $96 31 tickets remaining! Tickets\r\n* Sep 26 2024 7:00 PM Thu Lyfe Jennings City Winery - Atlanta | Capacity: 300 30308, 675 Ponce De Leon Ave, Atlanta, GA, US Prices from $98 Avg. price ~ $113 4 tickets remaining! Tickets\r\n* Sep 26 2024 7:20 PM Thu Atlanta Braves vs. New York Mets Truist Park | Capacity: 41149 30339, 1100 Circle 75 Pkwy, Atlanta, GA, US Prices from $16 Avg. price ~ $136 Available Tickets: 277 Tickets\r\n* Sep 26 2024 7:30 PM Thu Khruangbin Fabulous Fox Theatre - Atlanta | Capacity: 4678 30305, 660 Peachtree St NE, Atlanta, GA, US Prices from $41 Avg. price ~ $83 Available Tickets: 149 Tickets\r\n* Sep 26 2024 7:30 PM Thu Disney On Ice Gas South Arena | Capacity: 13100 30097, 6400 Sugarloaf Parkway, Duluth, GA, US Prices from $36 Avg. price ~ $91 Available Tickets: 894 Tickets\r\n* Sep 26 2024 8:00 PM Thu Atlanta Symphony Orchestra: Nathalie Stutzmann - Mahler \u0026 Shostakovich Atlanta Symphony Hall | Capacity: 1762 30309, 1280 Peachtree St, Atlanta, GA, US Prices from $55 Avg. price ~ $116 Available Tickets: 333 Tickets\r\n* Sep 26 2024 8:00 PM Thu Lawrence The Tabernacle - GA | Capacity: 2600 30303, 152 Luckie St, Atlanta, GA, US Prices from $25 Avg. price ~ $80 Available Tickets: 106 Tickets\r\n* Sep 26 2024 8:00 PM Thu The Buena Vista Social Orchestra Center Stage Theatre | Capacity: 1050 30309, 1374 West Peachtree Street Northwest, Atlanta, GA, US Prices from $64 Avg. price ~ $101 Available Tickets: 255 Tickets\r\n* Sep 26 2024 8:00 PM Thu The California Honeydrops Variety Playhouse | Capacity: 1000 30307, 1099 Euclid Ave, Atlanta, GA, US Prices from $33 Avg. price ~ $54 25 tickets remaining! Tickets\r\n* Sep 26 2024 8:00 PM Thu Jesus Adrian Romero Cobb Energy Performing Arts Centre | Capacity: 2750 30339, 2800 Cobb Galleria Pkwy, Atlanta, GA, US Prices from $58 Avg. price ~ $214 Available Tickets: 125 Tickets\r\n* Sep 26 2024 8:00 PM Thu Wasia Project Terminal West At King Plow Arts Center | Capacity: 625 30318, 887 West Marietta Studio C, Atlanta, GA, US Prices from $30 Avg. price ~ $43 19 tickets remaining! Tickets\r\n* Sep 26 2024 10:00 PM Thu Lyfe Jennings City Winery - Atlanta | Capacity: 300 30308, 675 Ponce De Leon Ave, Atlanta, GA, US Prices from $98 Avg. price ~ $124 16 tickets remaining! Tickets\r\n* Sep 27 2024 11:00 AM Fri Disney On Ice Gas South Arena | Capacity: 13100 30097, 6400 Sugarloaf Parkway, Duluth, GA, US Prices from $39 Avg. price ~ $85 Available Tickets: 639 Tickets\r\n* Sep 27 2024 6:00 PM Fri Red Lace Burlesque Show and Variety Show Sanctuary Nightclub - Atlanta 30305, 3209 Paces Ferry Pl NW, Atlanta, GA, US Prices from $51 Avg. price ~ $405 Available Tickets: 60 Tickets\r\n* Sep 27 2024 7:00 PM Fri Georgia Tech Yellow Jackets Women\u0027s Volleyball vs. Southern Methodist (SMU) Mustangs O\u0027Keefe Gymnasium 30332, 151 Sixth Street NW, Atlanta, GA, US Prices from $48 Avg. price ~ $55 2 tickets remaining! Tickets\r\n* Sep 27 2024 7:00 PM Fri Coral Moons Altar at Masquerade - GA 30303, 75 MLK Jr Dr SW, Atlanta, GA, US Prices from $28 Avg. price ~ $33 16 tickets remaining! Tickets\r\n* Sep 27 2024 7:00 PM Fri The Red Pears Purgatory Stage at Masquerade - GA | Capacity: 300 30303, 75 MLK Jr Dr SW, Atlanta, GA, US Prices from $35 Avg. price ~ $45 36 tickets remaining! Tickets\r\n* Sep 27 2024 7:00 PM Fri The Warning Hell at The Masquerade - GA | Capacity: 300 30303, 75 MLK Jr Dr SW, Atlanta, GA, US Prices from $38 Avg. price ~ $86 Available Tickets: 79 Tickets\r\n* Sep 27 2024 7:00 PM Fri Lyfe Jennings City Winery - Atlanta | Capacity: 300 30308, 675 Ponce De Leon Ave, Atlanta, GA, US Prices from $148 Avg. price ~ $171 2 tickets remaining! Tickets\r\n* Sep 27 2024 7:20 PM Fri Atlanta Braves vs. Kansas City Royals VIP Packages Truist Park | Capacity: 41149 30339, 1100 Circle 75 Pkwy, Atlanta, GA, US Prices from $10 Avg. price ~ $69 Available Tickets: 2510 Tickets\r\n* Sep 27 2024 7:30 PM Fri T.K. Kirkland Uptown Comedy Corner 30318, 800 Marietta St, Atlanta, GA, US Prices from $78 Avg. price ~ $114 10 tickets remaining! Tickets\r\n\r\nShow More### All Events This Month\r\n\r\n[Sevendust Upcoming Events: 2 The Eastern - GA Sep 25, 2024 6:30 PM](https://atlanta-ga.events/tickets/sevendust/)[Lorna Shore Upcoming Events: 2 The Tabernacle - GA Sep 25, 2024 6:30 PM](https://atlanta-ga.events/tickets/lorna-shore/)[Finger Eleven Upcoming Events: 2 Ameris Bank Amphitheatre Sep 25, 2024 7:00 PM](https://atlanta-ga.events/tickets/finger-eleven/)[Creed Upcoming Events: 2 Ameris Bank Amphitheatre Sep 25, 2024 7:00 PM](https://atlanta-ga.events/tickets/creed/)[3 Doors Down Upcoming Events: 2 Ameris Bank Amphitheatre Sep 25, 2024 7:00 PM](https://atlanta-ga.events/tickets/3-doors-down/)[New York Mets Upcoming Events: 6 Truist Park Sep 25, 2024 7:20 PM](https://atlanta-ga.events/tickets/new-york-mets/)[Atlanta Braves Upcoming Events: 15 Truist Park Sep 25, 2024 7:20 PM](https://atlanta-ga.events/tickets/atlanta-braves/)[Phantogram Upcoming Events: 2 State Farm Arena - GA Sep 25, 2024 7:30 PM](https://atlanta-ga.events/tickets/phantogram/)[Kings Of Leon Upcoming Events: 2 State Farm Arena - GA Sep 25, 2024 7:30 PM](https://atlanta-ga.events/tickets/kings-of-leon/)[Lasso - Artist Upcoming Events: 2 Buckhead Theatre Sep 25, 2024 8:00 PM](https://atlanta-ga.events/tickets/lasso-artist/)[Shaboozey Upcoming Events: 2 Variety Playhouse Sep 25, 2024 8:00 PM](https://atlanta-ga.events/tickets/shaboozey/)[Gritz and Jelly Butter Upcoming Events: 2 City Winery - Atlanta Sep 25, 2024 8:00 PM](https://atlanta-ga.events/tickets/gritz-and-jelly-butter/)[Rain City Drive Upcoming Events: 2 Hell at The Masquerade - GA Sep 26, 2024 6:30 PM](https://atlanta-ga.events/tickets/rain-city-drive/)[Lyfe Jennings Upcoming Events: 6 City Winery - Atlanta Sep 26, 2024 7:00 PM](https://atlanta-ga.events/tickets/lyfe-jennings/)[Mico Upcoming Events: 2 Purgatory Stage at Masquerade - GA Sep 26, 2024 7:00 PM](https://atlanta-ga.events/tickets/mico/)[Khruangbin Upcoming Events: 6 Fabulous Fox Theatre - Atlanta Sep 26, 2024 7:30 PM](https://atlanta-ga.events/tickets/khruangbin/)[Disney On Ice Upcoming Events: 24 Tulsa Expo Square - Pavilion Sep 26, 2024 7:30 PM](https://atlanta-ga.events/tickets/disney-on-ice/)[Jesus Adrian Romero Upcoming Events: 2 Cobb Energy Performing Arts Centre Sep 26, 2024 8:00 PM](https://atlanta-ga.events/tickets/jesus-adrian-romero/)[Atlanta Symphony Orchestra Upcoming Events: 6 Atlanta Symphony Hall Sep 26, 2024 8:00 PM](https://atlanta-ga.events/tickets/atlanta-symphony-orchestra/)[Lawrence - Band Upcoming Events: 2 The Tabernacle - GA Sep 26, 2024 8:00 PM](https://atlanta-ga.events/tickets/lawrence-band/)[Wasia Project Upcoming Events: 2 Terminal West At King Plow Arts Center Sep 26, 2024 8:00 PM](https://atlanta-ga.events/tickets/wasia-project/)[The California Honeydrops Upcoming Events: 2 Variety Playhouse Sep 26, 2024 8:00 PM](https://atlanta-ga.events/tickets/the-california-honeydrops/)[The Buena Vista Social Orchestra Upcoming Events: 2 Center Stage Theatre Sep 26, 2024 8:00 PM](https://atlanta-ga.events/tickets/the-buena-vista-social-orchestra/)[Nathalie Stutzmann Upcoming Events: 4 Atlanta Symphony Hall Sep 26, 2024 8:00 PM](https://atlanta-ga.events/tickets/nathalie-stutzmann/)[Red Lace Burlesque Show and Variety Show Upcoming Events: 4 Red Lace Burlesque - Myrtle Beach Sep 27, 2024 6:00 PM](https://atlanta-ga.events/tickets/red-lace-burlesque-show-and-variety-show/)[Georgia Tech Yellow Jackets Women\u0027s Volleyball Upcoming Events: 6 O\u0027Keefe Gymnasium Sep 27, 2024 7:00 PM](https://atlanta-ga.events/tickets/georgia-tech-yellow-jackets-womens-volleyball/)[Georgia Tech Yellow Jackets Upcoming Events: 6 O\u0027Keefe Gymnasium Sep 27, 2024 7:00 PM](https://atlanta-ga.events/tickets/georgia-tech-yellow-jackets/)[The Warning Upcoming Events: 2 Hell at The Masquerade - GA Sep 27, 2024 7:00 PM](https://atlanta-ga.events/tickets/the-warning/)[Southern Methodist (SMU) Mustangs Upcoming Events: 3 O\u0027Keefe Gymnasium Sep 27, 2024 7:00 PM](https://atlanta-ga.events/tickets/southern-methodist-smu-mustangs/)[Coral Moons Upcoming Events: 2 Altar at Masquerade - GA Sep 27, 2024 7:00 PM](https://atlanta-ga.events/tickets/coral-moons/)[The Red Pears Upcoming Events: 2 Purgatory Stage at Masquerade - GA Sep 27, 2024 7:00 PM](https://atlanta-ga.events/tickets/the-red-pears/)[Southern Methodist (SMU) Mustangs Women\u0027s Volleyball Upcoming Events: 3 O\u0027Keefe Gymnasium Sep 27, 2024 7:00 PM](https://atlanta-ga.events/tickets/southern-methodist-smu-mustangs-womens-volleyball/)[Kansas City Royals Upcoming Events: 9 Truist Park Sep 27, 2024 7:20 PM](https://atlanta-ga.events/tickets/kansas-city-royals/)[Maxwell Upcoming Events: 2 State Farm Arena - GA Sep 27, 2024 7:30 PM](https://atlanta-ga.events/tickets/maxwell/)[La Santa Grifa Upcoming Events: 2 Buckhead Theatre Sep 27, 2024 7:30 PM](https://atlanta-ga.events/tickets/la-santa-grifa/)[T.K. Kirkland Upcoming Events: 8 Uptown Comedy Corner Sep 27, 2024 7:30 PM](https://atlanta-ga.events/tickets/t-k-kirkland/)[Dharius Upcoming Events: 2 Buckhead Theatre Sep 27, 2024 7:30 PM](https://atlanta-ga.events/tickets/dharius/)[Tropical Fuck Storm Upcoming Events: 2 Terminal West At King Plow Arts Center Sep 27, 2024 8:00 PM](https://atlanta-ga.events/tickets/tropical-fuck-storm/)[Sisters Of Mercy Upcoming Events: 2 Coca-Cola Roxy Sep 27, 2024 8:00 PM](https://atlanta-ga.events/tickets/sisters-of-mercy/)[Lalah Hathaway Upcoming Events: 4 Center Stage Theatre Sep 27, 2024 8:00 PM](https://atlanta-ga.events/tickets/lalah-hathaway/)Show MoreAtlanta Events Calendar[](https://atlanta-ga.events)[Contact Us](#)Stay up to date! Join our newsletter now to stay up to date on all events in Atlanta plus the occasional special offer and discount code! Epic music events and incredible deals straight to your inbox. Either way, you\u2019ll be the first to know about the best shows in your area.\r\n\r\n[](https://www.facebook.com/)[](https://www.youtube.com/)[](https://www.instagram.com/)[](https://twitter.com/)[](https://www.pinterest.com/)Subscription DealsNameEmail7 \u002B 15 =SubscribeAtlanta-GA.Events|[Sitemap](https://atlanta-ga.events/sitemaps/)|[Privacy Policy](#)|All Rights Reserved 2024"
```

I called the tool named GetContentFromUrl with these arguments:
```
{"url": "https://gosouthatlanta.com/things-to-do/events/atlanta-in-september.html"}
```
and got this result:
```
"Atlanta Events - September 2024This site uses cookies. Opt out at any time. More details\r\n\r\nHome \u0026rsaquo; Things To Do \u0026rsaquo; September Events\r\n\r\n# Atlanta Events - September 2024\r\n\r\nInterested in a specific type of event? See Atlanta\u0026rsquo;s calendars of upcoming: \u0026ndash; Comedy shows \u0026ndash; Beer and wine festivals \u0026ndash; Music festivals \u0026ndash; Musicals \u0026ndash; Food festivals\r\n\r\nSee also: \u0026ndash; Atlanta\u0026rsquo;s festival calendar \u0026ndash; Atlanta weather in September \u0026ndash; Things to do in Atlanta \u0026ndash; Best time of year to visit Atlanta\r\n\r\n!Event dates, venue and other details can change at short notice. Please check all information at the official website before making plans to attend an event.\r\n\r\n## Events From Labor Day Weekend To The Weekend Of September 7-8 2024\r\n\r\n\u0026rarr; See August events\r\n\r\n\u0026rarr; For concerts this week, see Atlanta concerts in September 2024\r\n\r\n\u0026ndash; Wild With Happy, August 16 - September 15 2024Play at Horizon Theatre. More details\r\n\r\n\u0026ndash; Atlanta Black Pride Weekend, August 28 - September 2 2024Annual LGBTQ\u002B festival with male, female and youth events across Atlanta, including parties, block parties, food events, performances, film festival, and more. More details\r\n\r\n\u0026ndash; Dragon Con, August 29 - September 2 2024One of the biggest annual pop culture conventions in the country. More details\r\n\r\n\u0026ndash; Atlanta Black Theatre Festival, August 30 - September 1 2024Annual festival celebrating the Black experience in the performing arts, with plays, panels, masterclasses and other events. More details\r\n\r\n\u0026ndash; The Mountaintop, August 30 - September 22 2024Play at Alliance Theatre. More details\r\n\r\n\u0026ndash; Atlanta Hip Hop Day Festival, August 31 - September 1 2024Annual festival of hip hop music at Historic 4th Ward Park. More details\r\n\r\n\u0026ndash; Paw Patrol Live!, August 31 - September 1 2024Children\u0026rsquo;s performance at the Fox Theatre. More details\r\n\r\n\u0026ndash; Monster Jam, August 31 - September 1 2024Monster truck show at the Gas South Arena. More details\r\n\r\n\u0026ndash; Art in the Park, August 31 - September 2 2024Annual Labor Day weekend arts festival in Marietta Square, with fine art market featuring artists from across the country, chalk art, children\u0026rsquo;s craft activities and more. More details\r\n\r\n\u0026ndash; Pure Heat Community Festival, September 1 2024Very popular annual LGBTQ\u002B festival in Piedmont Park as part of Black Pride Weekend, with youth festival, live music and other performances, vendors, food and more. More details\r\n\r\n\u0026ndash; Big Peach Sizzler 10K \u0026amp; 5K, September 2 202410K and 5K run/walk along Peachtree road between Brookhaven/Chamblee and Buckhead, to benefit Miles for Cystic Fibrosis. More details\r\n\r\n\u0026ndash; Prog Power USA, September 4-7 2024Annual progressive and power metal festival. More details\r\n\r\n\u0026ndash; Supreme Court Justice Ketanji Brown Jackson: Lovely One Book Tour, September 5 2024Author talk at Atlanta Symphony Hall. More details\r\n\r\n\u0026ndash; Yellow Daisy Festival, September 5-8 2024Big annual arts and crafts festival at Stone Mount Park, with 400 artists and artisans, live music, festival food, craft demonstrations, beer garden, vintage market and more. More details\r\n\r\n\u0026ndash; Shane Gillis Live, September 6 2024Comedy show at State Farm Arena. More details\r\n\r\n\u0026ndash; Atlanta Home Show, September 6-8 2024Annual fall home improvements trade show at the Cobb Galleria Centre. More details\r\n\r\n\u0026ndash; Lakewood Antiques Market, September 6-8 2024Big monthly antiques and collectibles market in Cumming. More details\r\n\r\n\u0026ndash; Meredith Willson\u0026rsquo;s The Music Man, September 6-22 2024Musical at the Sandy Springs Performing Arts Center. More details\r\n\r\n\u0026ndash; Beloved Community International Expo, September 7 2024Annual international festival at The King Center, with global foods, performances, art, music, dance and more. More details\r\n\r\n\u0026ndash; Bill Maher: The WTF? Tour, September 7 2024Comedy show at the Cobb Energy Performing Arts Centre. More details\r\n\r\n\u0026ndash; Stuff You Should Know Live, September 7 2024Podcast live show at Atlanta Symphony Hall. More details\r\n\r\n\u0026ndash; Dale Elliott Jr: No Hard Feelings, September 7 2024Comedy show at the Buckhead Theatre. More details\r\n\r\n\u0026ndash; Tacos \u0026amp; Tequilas Marietta Square, September 7 2024Taco and margarita themed bar crawl in downtown Marietta. More details\r\n\r\n\u0026ndash; Coulda Fest, September 7 2024Comedy and hip-hop music festival at State Farm Arena. More details\r\n\r\n\u0026ndash; Chaucer\u0026rsquo;s The Canterbury Tales, September 7-29 2024Play at the Shakespeare Tavern Playhouse. More details\r\n\r\n\u0026ndash; Giggly Squad Live: Club Giggly, September 8 2024Comedy show at The Tabernacle. More details\r\n\r\n\u0026ndash; The Wait Wait Stand-Up Tour, September 8 2024Comedy show at Atlanta Symphony Hall. More details\r\n\r\n## Events From September 9 To The Weekend Of September 14-15 2024\r\n\r\n\u0026rarr; For concerts this week, see Atlanta concerts in September 2024\r\n\r\n\u0026ndash; Chaucer\u0026rsquo;s The Canterbury Tales, September 7-29 2024Play at the Shakespeare Tavern Playhouse. More details\r\n\r\n\u0026ndash; Figgmunity, September 9 2024Podcast live show at Terminal West. More details\r\n\r\n\u0026ndash; Wild With Happy, August 16 - September 15 2024Play at Horizon Theatre. More details\r\n\r\n\u0026ndash; The Mountaintop, August 30 - September 22 2024Play at Alliance Theatre. More details\r\n\r\n\u0026ndash; Meredith Willson\u0026rsquo;s The Music Man, September 6-22 2024Musical at the Sandy Springs Performing Arts Center. More details\r\n\r\n\u0026ndash; Spider-Man: Across the Spider-Verse Live in Concert, September 11 2024Film screening with live musical accompaniment at Atlanta Symphony Hall. More details\r\n\r\n\u0026ndash; An Evening with Dr Jane Goodall, September 11 2024Talk event at the Fox Theatre. More details\r\n\r\n\u0026ndash; Adam Ray, September 12 2024Comedy show at the Buckhead Theatre. More details\r\n\r\n\u0026ndash; Andres Lopez: La Pelota de Letras 20 A\u0026ntilde;os, September 12 2024Spanish-language comedy show at the Gas South Theater. More details\r\n\r\n\u0026ndash; Scott Antique Market, September 12-15 2024Monthly antiques market at the Atlanta Expo Center. More details\r\n\r\n\u0026ndash; The Princess Bride in Concert, September 13-14 2024Film screening with live orchestral accompaniment at Atlanta Symphony Hall. More details\r\n\r\n\u0026ndash; Atlanta Food \u0026amp; Wine Festival, September 13-15 2024Big annual food festival with a variety of events including tastings, chef talks, and more. More details\r\n\r\n\u0026ndash; Fall Into Rhythm: Atlanta Ballet, September 13-15 2024Ballet performance at the Cobb Energy Performing Arts Centre. More details\r\n\r\n\u0026ndash; Daddy Long Legs, September 13 - October 6 2024Musical at The Legacy Theatre, Tyrone. More details\r\n\r\n\u0026ndash; West Ponce Music Stroll, September 14 2024Free annual music festival along West Ponce de Leon Avenue, Decatur, with outdoor concerts, chalk art, 5K, fitness classes, makers market and more. More details\r\n\r\n\u0026ndash; Sandy Springs Jam Fest, September 14 2024Inaugural festival of local and regional music at The Springs Cinema \u0026amp; Taphouse. More details\r\n\r\n\u0026ndash; Kurtis Conner: The Goodfellow World Tour, September 14 2024Comedy show at The Tabernacle. More details\r\n\r\n\u0026ndash; Jim Jefferies with Mark Normand and Dan Soder, September 14 2024Comedy show at the Fox Theatre. More details\r\n\r\n\u0026ndash; Major League Wrestling Presents: Fightland, September 14 2024Wrestling event at Center Stage. More details\r\n\r\n\u0026ndash; Men Are From Mars, Women Are From Venus Live!, September 14 2024Comedy show at the Gas South Theater. More details\r\n\r\n\u0026ndash; Sandy Springs Fall Arts and Craft Festival, September 14-15 2024Twice-annual arts festival in downtown Sandy Springs with 100\u002B arts and crafts vendors, local musicians, art activities and more. More details\r\n\r\n\u0026ndash; Out of the Box: Series II, September 14-15, 21-22 2024Ballet performance by Terminus Modern Ballet Theatre at the White Box Theater. More details\r\n\r\n\u0026ndash; Georgia Bridal Show, September 15 2024Wedding trade show at the Gas South District. More details\r\n\r\n\u0026ndash; An Evening with Jamie Lissow, September 15 2024Comedy show at the Gas South Theater. More details\r\n\r\n\u0026ndash; The Sneaker Exit, September 15 2024Footwear trade show at the Gas South Convention Center. More details\r\n\r\n## Events From September 16 To The Weekend Of September 21-22 2024\r\n\r\n\u0026rarr; For concerts this week, see Atlanta concerts in September 2024\r\n\r\n\u0026ndash; The Mountaintop, August 30 - September 22 2024Play at Alliance Theatre. More details\r\n\r\n\u0026ndash; Meredith Willson\u0026rsquo;s The Music Man, September 6-22 2024Musical at the Sandy Springs Performing Arts Center. More details\r\n\r\n\u0026ndash; Chaucer\u0026rsquo;s The Canterbury Tales, September 7-29 2024Play at the Shakespeare Tavern Playhouse. More details\r\n\r\n\u0026ndash; Daddy Long Legs, September 13 - October 6 2024Musical at The Legacy Theatre, Tyrone. More details\r\n\r\n\u0026ndash; Boh\u0026egrave;me Project: La boh\u0026egrave;me, September 18 - October 5 2024Opera at Pullman Yards. More details\r\n\r\n\u0026ndash; The Chinese Lady, September 18 - October 13 2024Play at Alliance Theatre. More details\r\n\r\n\u0026ndash; Wild \u0026rsquo;N Out Live: The Final Lap, September 19 2024Comedy show at State Farm Arena. More details\r\n\r\n\u0026ndash; Geeked Week 24, September 19 2024Netflix preview event at The Eastern. More details\r\n\r\n\u0026ndash; Georgia Latino International Film Festival, September 19-22 2024Annual festival of Latino/Latina-produced films. More details\r\n\r\n\u0026ndash; Hauntlanta, September 19-22 2024Annual convention of paranormal culture at Lanier Islands, Buford. More details\r\n\r\n\u0026ndash; North Georgia State Fair, September 19-29 2024Annual fair at Jim R Miller Park, Marietta, with carnival rides, animal shows, performances, rodeo, car events and more. More details\r\n\r\n\u0026ndash; Boh\u0026egrave;me Project: Rent, September 19 - October 6 2024Opera at Pullman Yards. More details\r\n\r\n\u0026ndash; Kirkwood Wine Stroll, September 20 2024Annual fundraiser of the Kirkwood Business Owners\u0026rsquo; Association, with wine-tastings at 50\u002B neighborhood establishments, demonstrations, live music, and more. More details\r\n\r\n\u0026ndash; Iliza: The Get Ready Tour, September 20 2024Comedy show at the Fox Theatre. More details\r\n\r\n\u0026ndash; Eddie Pokropski \u0026ndash; Case 84: Adopted in the USA, September 21 2024Comedy show at Red Clay Music Foundry, Duluth. More details\r\n\r\n\u0026ndash; Atlanta Greek Festival, September 20-22 2024Annual festival of Greek food and heritage at Annunciation Greek Orthodox Cathedral, with traditional Greek foods and food demonstrations, cathedral tours, live music and dancing, performances and more. More details\r\n\r\n\u0026ndash; Little Shop of Horrors: An Art Farm at Serenbe and Matt Logan Production, September 20-29 2024Musical under the tent in the Wildflower Meadow at Serenbe, Chattahoochee Hills. More details\r\n\r\n\u0026ndash; Porches and Pies, September 21 2024Annual pie-themed food festival in Adair Park, with pie tastings and food trucks, vendor market, live entertainment, beer and cocktail garden, children\u0026rsquo;s activities and more. More details\r\n\r\n\u0026ndash; Pop-In For Family Fun: Books, Books, Books, September 21 2024Monthly free family day at the Marietta History Center. More details\r\n\r\n\u0026ndash; HBCU AccessFest, September 21 2024Free college and career fair at Woodruff Arts Center, with representatives from Historically Black Colleges and Universities from across the country. More details\r\n\r\n\u0026ndash; Million Meal Pack, September 21 2024Charity event at State Farm Arena at which volunteers pack meals for local people experiencing food insecurity. More details\r\n\r\n\u0026ndash; Brett Goldstein: The Second Best Night Of Your Life, September 21 2024Comedy show at the Fox Theatre. More details\r\n\r\n\u0026ndash; Sippin\u0026rsquo; Safari, September 21 2024After-hours wine festival at Zoo Atlanta, with wine tasting and live music amidst the animal displays. More details\r\n\r\n\u0026ndash; Out of the Box: Series II, September 14-15, 21-22 2024Ballet performance by Terminus Modern Ballet Theatre at the White Box Theater. More details\r\n\r\n\u0026ndash; Buckhead Arts Festival, September 21-22 2024Annual outdoor fine arts festival in Buckhead, with 100\u002B artists working in a variety of media, art demonstrations and live music. More details\r\n\r\n\u0026ndash; JapanFest, September 21-22 2024Annual festival of Japanese culture at the Gas South Convention Center, with Japanese food from local restaurants and food trucks, live performances, vendors and more. More details\r\n\r\n\u0026ndash; Peachtree Corners Festival, September 21-22 2024Annual fall community festival at Peachtree Corners Town Green, with arts and crafts, classic car show, live music, food and children\u0026rsquo;s activities. More details\r\n\r\n\u0026ndash; The Curious Cardinal, September 21 - October 27 2024Children\u0026rsquo;s performance at Alliance Theatre. More details\r\n\r\n\u0026ndash; Matteo Lane: Can\u0026rsquo;t Stop Talking, September 22 2024Comedy show at the Fox Theatre. More details\r\n\r\n\u0026ndash; My Brother, My Brother \u0026amp; Me, September 22 2024Podcast live show at Atlanta Symphony Hall. More details\r\n\r\n## Events From September 23 To The Weekend Of September 28-29 2024\r\n\r\n\u0026rarr; For concerts this week, see Atlanta concerts in September 2024\r\n\r\n\u0026ndash; Chaucer\u0026rsquo;s The Canterbury Tales, September 7-29 2024Play at the Shakespeare Tavern Playhouse. More details\r\n\r\n\u0026ndash; Daddy Long Legs, September 13 - October 6 2024Musical at The Legacy Theatre, Tyrone. More details\r\n\r\n\u0026ndash; Boh\u0026egrave;me Project: La boh\u0026egrave;me, September 18 - October 5 2024Opera at Pullman Yards. More details\r\n\r\n\u0026ndash; The Chinese Lady, September 18 - October 13 2024Play at Alliance Theatre. More details\r\n\r\n\u0026ndash; North Georgia State Fair, September 19-29 2024Annual fair at Jim R Miller Park, Marietta, with carnival rides, animal shows, performances, rodeo, car events and more. More details\r\n\r\n\u0026ndash; Boh\u0026egrave;me Project: Rent, September 19 - October 6 2024Opera at Pullman Yards. More details\r\n\r\n\u0026ndash; Little Shop of Horrors: An Art Farm at Serenbe and Matt Logan Production, September 20-29 2024Musical under the tent in the Wildflower Meadow at Serenbe, Chattahoochee Hills. More details\r\n\r\n\u0026ndash; The Curious Cardinal, September 21 - October 27 2024Children\u0026rsquo;s performance at Alliance Theatre. More details\r\n\r\n\u0026ndash; Disney On Ice, September 26-29 2024Family ice-skating performance at the Gas South Arena. More details\r\n\r\n\u0026ndash; Out on Film, September 26 - October 6 2024Annual festival of LGBTQ\u002B filmmaking. More details\r\n\r\n\u0026ndash; Aqua Vino, September 27 2024Annual wine tasting event at the Georgia Aquarium to benefit its research and conservation initiatives, with unlimited wine samples, small bites and live entertainment amidst the sea life exhibits. More details\r\n\r\n\u0026ndash; Sweet Auburn Music Fest, September 27-29 2024Popular annual R\u0026amp;B and gospel music street festival in the Sweet Auburn district, with international food, art and other vendors, local businesses and more. More details\r\n\r\n\u0026ndash; Joelanta, September 27-29 2024Action figure and collectibles convention at the Gas South Convention Center. More details\r\n\r\n\u0026ndash; East Atlanta Strut, September 28 2024Annual neighborhood festival in East Atlanta, with parade, contests, marketplaces, music and more. More details\r\n\r\n\u0026ndash; Yappy Hour, September 28 2024Pet-friendly event series at The Hamilton Hotel Alpharetta, with pet adoptions and specialty cocktails. More details\r\n\r\n\u0026ndash; AIDS Walk Atlanta Music Festival \u0026amp; 5K Run, September 28 20245K run/walk and music festival in Piedmont Park to benefit AIDS services in Atlanta. More details\r\n\r\n\u0026ndash; French Market Atlanta, September 28 2024French-style outdoor market in Buckhead Village District, with French food, crafts and live music. More details\r\n\r\n\u0026ndash; Fine Wine Festival, September 28 2024Wine festival at the Home Depot Backyard, with wine tastings, R\u0026amp;B music and more. More details\r\n\r\n\u0026ndash; Trae Crowder, September 28 2024Comedy show at the Buckhead Theatre. More details\r\n\r\n\u0026ndash; Ghost Files Live!, September 28 2024Paranormal investigation live show at the Cobb Energy Performing Arts Centre. More details\r\n\r\n\u0026ndash; Riverfest, September 28-29 2024Annual juried arts and crafts festival in Etowah River Park, Canton, with 150\u002B artists and artisans from across the southeast, local food trucks and musicians, children\u0026rsquo;s activities, football screenings and more. More details\r\n\r\n\u0026ndash; Roswell Wine Festival, September 28-29 2024Annual wine festival in downtown Roswell, with wine tastings, food from local restaurants and live music. More details\r\n\r\n\u0026ndash; Duluth Fall Festival, September 28-29 2024Popular fall neighborhood festival in downtown Duluth, with parade, 5K race, concert and other entertainment, arts and crafts, food and more. More details\r\n\r\n\u0026ndash; Everyday Heroes, September 28-29 2024Dance performance at the Gas South Theater. More details\r\n\r\n\u0026ndash; Atlanta Comic Convention, September 29 2024Comic book and collectibles convention at the Marriott Hotel - Century Center. More details\r\n\r\n## Events From September 30 To The Weekend Of October 5-6 2024\r\n\r\n\u0026rarr; For concerts this week, see Atlanta concerts in September 2024\r\n\r\n\u0026ndash; Daddy Long Legs, September 13 - October 6 2024Musical at The Legacy Theatre, Tyrone. More details\r\n\r\n\u0026ndash; Boh\u0026egrave;me Project: La boh\u0026egrave;me, September 18 - October 5 2024Opera at Pullman Yards. More details\r\n\r\n\u0026ndash; The Chinese Lady, September 18 - October 13 2024Play at Alliance Theatre. More details\r\n\r\n\u0026ndash; Boh\u0026egrave;me Project: Rent, September 19 - October 6 2024Opera at Pullman Yards. More details\r\n\r\n\u0026ndash; The Curious Cardinal, September 21 - October 27 2024Children\u0026rsquo;s performance at Alliance Theatre. More details\r\n\r\n\u0026ndash; Out on Film, September 26 - October 6 2024Annual festival of LGBTQ\u002B filmmaking. More details\r\n\r\n\u0026ndash; The Moth Atlanta GrandSLAM, October 1 2024Storytelling competition at Center Stage. More details\r\n\r\n\u0026ndash; Disney\u0026rsquo;s The Lion King, October 2-20 2024Musical at the Fox Theatre. More details\r\n\r\n\u0026ndash; Kumail Nanjiani: Doing This Again, October 4 2024Comedy show at The Tabernacle. More details\r\n\r\n\u0026ndash; Stand Up Comedy at the Clay! Hosted by Yoshee, October 4 2024Comedy show at Red Clay Music Foundry, Duluth. More details\r\n\r\n\u0026ndash; ExpoContratista, October 4-5 2024Hispanic construction trade show at the Cobb Galleria Centre. More details\r\n\r\n\u0026ndash; How to Train Your Dragon in Concert with the ASO, October 4-5 2024Film screening with live musical accompaniment at Atlanta Symphony Hall. More details\r\n\r\n\u0026ndash; Atlanta Horror Film Festival, October 4-6 2024Annual festival of horror film at Limeliight Theater. More details\r\n\r\n\u0026ndash; DreamHack Atlanta, October 4-6 2024Annual esports convention at the Georgia World Congress Center. More details\r\n\r\n\u0026ndash; Atlanta Sci-Fi Film Festival, October 4-6 2024Annual science fiction film festival in Peachtree Corners. More details\r\n\r\n\u0026ndash; Lakewood Antiques Market, October 4-6 2024Big monthly antiques and collectibles market in Cumming. More details\r\n\r\n\u0026ndash; Am\u0026eacute;lie: The Musical, October 4 - November 3 2024Musical at Horizon Theatre. More details\r\n\r\n\u0026ndash; Wings \u0026amp; Whitewalls, October 5 2024Fall car show at the Aviation History \u0026amp; Technology Center, Marietta, with vintage aircraft, cars and more. More details\r\n\r\n\u0026ndash; Taste of Tucker, October 5 2024Annual food festival along Main Street, Tucker, featuring food and drink from local restaurants and breweries, arts and crafts, music and more. More details\r\n\r\n\u0026ndash; EGGtoberfest, October 5 2024Barbecue festival at Coolray Field, Lawrenceville, with cook-off and cooking demonstrations, children\u0026rsquo;s activities and more. More details\r\n\r\n\u0026ndash; WWE Bad Blood, October 5 2024Wrestling event at State Farm Arena. More details\r\n\r\n\u0026ndash; Festival on Ponce, October 5-6 2024Twice-annual arts festival in Olmsted Linear Park, with 125\u002B vendors of fine art, folk art and outsider art, plus food and drink, and children\u0026rsquo;s activities. More details\r\n\r\n\u0026ndash; Candler Park Fall Fest, October 5-6 2024Popular annual festival in Candler Park and neighborhood, with live music, art, food, rides, games and more. More details\r\n\r\n\u0026ndash; Atlanta Arab Festival, October 5-6 2024Annual festival of Arabic food and culture at the Alif Institute, with traditional dishes from Arabic countries, live performances, vendors, exhibits, workshops, games and more. More details\r\n\r\n\u0026ndash; Aladdin, October 5-6 2024Ballet performance at the Gas South Theater. More details\r\n\r\n\u0026ndash; ASC Presents Havoc Movement\u0026rsquo;s Dracula: The Failings of Men, October 5 - November 3 2024Performance at the Shakespeare Tavern Playhouse. More details\r\n\r\n\u0026rarr; See October events\r\n\r\n\u0026copy; Melanie K Jones 2024\r\n\r\n* Privacy\r\n* Terms\r\n* Contact\r\n\r\n"
```

I called the tool named GetContentFromUrl with these arguments:
```
{"url": "https://secretatlanta.co/things-to-do-september/"}
```
and got this result:
```
"Response status code does not indicate success: 403 (Forbidden)."
```

I called the tool named GetContentFromUrl with these arguments:
```
{"url": "https://unexpectedatlanta.com/the-best-things-to-do-in-atlanta-in-september/"}
```
and got this result:
```
"The Best Things to Do in Atlanta in September - Unexpected Atlanta![fbpx](https://www.facebook.com/tr?id=2689885501249973\u0026ev=PageView\u0026noscript=1)\r\n[Skip to main content](#ajax-content-wrap)* Blog\r\n* About Our Team Hiring Press Contact\r\n* Tour Calendar\r\n* Gift Certificates\r\n* Call Us\r\n* Buy Tickets\r\n\r\n[](https://unexpectedatlanta.com)[Menu](#sidewidgetarea)* Public Tours Atlanta Market Food Tour \u002B Biscuit Cooking Class Prohibition Progressive Dinner \u2013 Atlanta Cocktails and Food Experience Grant Park Atlanta Food Tour Martin Luther King Jr Atlanta Tour Cabbagetown \u0026#038; BeltLine Street Art Tour\r\n* Team Building Private Prohibition Progressive Dinner Private Grant Park Food \u0026#038; History Tour Private Market Food Tour and Biscuit Class Private Martin Luther King Jr Atlanta Tour Private Cabbagetown \u0026#038; BeltLine Street Art Tour Private Dinner and Street Art Tour 1920s Big Bash Party Conference Networking Games\r\n* Family and Student Groups Family Reunions Party Packages MLK Tour for Kids Student Martin Luther King Jr Atlanta Tour\r\n* Book Now!\r\n\r\n* Menu\r\n\r\n[Close Search](#)[Things to Do in Atlanta](https://unexpectedatlanta.com/category/things-to-do-atlanta/)# The Best Things to Do in Atlanta in September\r\n\r\nBy[Akila McConnell](https://unexpectedatlanta.com/author/akila/)June 19, 2024August 21st, 2024[No Comments](https://unexpectedatlanta.com/the-best-things-to-do-in-atlanta-in-september/#respond)![The Best Things to Do in Atlanta in September](https://unexpectedatlanta.com/wp-content/uploads/2023/08/The-Best-Things-to-Do-in-Atlanta-in-September.png)\r\nSoak up the last of summer and celebrate the start of fall with the best things to do in Atlanta in September 2024!\r\n\r\nThis month brings comfortable temperatures as summer cools down \u0026#8211; with long, golden days that spill into evening. Make the most of it with the best things to do in Atlanta!\r\n\r\nFrom exciting sports matchups to concerts, food festivals, apple picking, IG and TikTok-worthy sunflower fields, and more \u0026#8211; we\u0026#8217;re sharing our favorite activities around the city. Your Atlanta September memories start here!\r\n\r\nThe Best Things to Do in Atlanta in SeptemberRead on for additional details on these Atlanta September events and more ATL fun!\r\n\r\n* Cheer on the Atlanta Braves at a home game\r\n* Taste the best of Atlanta\u0026#8217;s Soul Food and Southern favorites\r\n* See musical acts like SOFI TUKKER, Childish Gambino, The Avett Brothers, and more\r\n* Experience culture and cuisine at A Taste of Nigeria\r\n* Take a guided walking food tour to taste 8 delicious modern Southern dishes\r\n* Explore Fernbank\u0026#8230;but Later\r\n\r\nIn this post, you\u0026#8217;ll find:* Fall festivals: Atlanta and beyond\r\n* Ideas to enjoy Atlanta in September\r\n* Concerts, food events, and sports games in Atlanta September 2024\r\n* September family-friendly events in Atlanta\r\n* Atlanta fall date ideas\r\n* Fun things to do during a business trip to Atlanta in September\r\n* FAQs about visiting Atlanta in September, including Atlanta September weather\r\n\r\n## Atlanta Events: September 2024\r\n\r\n### Experience the Best of Atlanta\u0026#8217;s Food \u0026amp; History\r\n\r\n\r\n\r\nTry some of the best eats in Atlanta! The #1-rated Grant Park Past \u0026amp; Future Food Walk is a fun walking tour showcasing the best of Atlanta\u0026#8217;s food and history. On this tour, you\u0026#8217;ll taste eight modern and Southern foods from three outstanding locally owned food stops.\r\n\r\nAlong the way, delve into Atlanta\u0026#8217;s history with an expert guide who will share fascinating stories of our city and answer questions. You\u0026#8217;ll also explore the unique architecture of Atlanta\u0026#8217;s historic Grant Park neighborhood. The 2.5 hour guided experience includes a jaunt through Historic Oakland Cemetery, where many noteworthy Atlantans have been laid to rest.\r\n\r\nWith more than 1,500 five-star TripAdvisor reviews, Unexpected Atlanta offers outstanding and unique walking tours for Atlanta locals and visitors!\r\n\r\n### Attend the Taste of Soul Festival: August 30 and September 1, 2024\r\n\r\nCraving even more Soul food in the heart of America\u0026#8217;s Soul Food capital? Head to the Taste of Soul Festival! Saturday, August 30 and Sunday, September 1 bring two days packed full of delicious Soul food and culture!\r\n\r\nTaste of Soul will take place across from Mercedes-Benz Stadium near Downtown Atlanta. This free, family-friendly festival is open to all. However, those who wish to attend the \u0026#8220;Soul in the City\u0026#8221; concert as part of this event must purchase tickets.\r\n\r\nThe festival includes Soul and Southern food for purchase, live music, arts and crafts vendors, kids\u0026#8217; activities, and a special cocktail and wine festival.\r\n\r\n### Cheer On the Home Teams at Falcons \u0026amp; Braves Games\r\n\r\n\r\n\r\nSeptember is a great month to be a sports fan! During September 2024, the Atlanta Braves baseball team will take on the LA Dodgers, Colorado Rockies, Kansas City Royals, New York Mets, and Toronto Blue Jays at Truist Park.\r\n\r\nFor college football games, check out Georgia Tech, Georgia State University, and \u0026#8211; if you\u0026#8217;re up for a day trip \u0026#8211; UGA in Athens, GA!\r\n\r\n### Experience Fernbank\u0026#8230;but Later: Sept. 27, 2024\r\n\r\nExplore Fernbank\u0026#8230;but Later with Stellar Science on Friday, September 27, 2024. This opportunity to see exhibits with later access to Fernbank is a fun way to socialize. You\u0026#8217;ll experience all the great indoor exhibit discoveries and outdoor nature explorations.\r\n\r\nAll ages are welcome, but if you\u0026#8217;re 21\u002B, there will be alcoholic beverages for purchase. Non-alcoholic drinks and snacks also available for purchase.\r\n\r\n### Experience Culture \u0026amp; Cuisine at A Taste of Nigeria\r\n\r\nOn Saturday, September 28, bring the family to A Taste of Nigeria at legendary Ford Hall in Hapeville, GA!\r\n\r\nAt this immersive Atlanta summer festival, you\u0026#8217;ll see a vibrant celebration filled Nigerian art, music, dance, comedy, drama, a fashion show, market, and more! Kids get free festival entry, with plenty of family-friendly activities to enjoy.\r\n\r\n### Attend Concerts in Atlanta: September 2024\r\n\r\nAs the capital of the South, Atlanta is a hotspot for concert-goers of all genres. In September 2023, Atlanta will attract chart-topping talent, with concerts for all tastes. From country to pop, hip-hop to classic rock, alternative, and more \u0026#8211; it\u0026#8217;s all here in Atlanta!\r\n\r\nAtlanta concerts in September 2024 \u0026#8211; a few highlights:* Childish Gambino: September 2 at State Farm Arena\r\n* Lynyrd Synyrd and The Outlaws: September 5 at Ameris Bank Amphitheatre\r\n* Mitski: September 6, 7, and 8 at the Fox Theater\r\n* Young Miko: September 6 at Coca-Cola Roxy\r\n* John Legend and Atlanta Symphony Orchestra: September 9 at Ameris Bank Amphitheatre\r\n* The Avett Brothers: September 14 at Gas South Arena\r\n* Zen Selekta: September 14 at Terminal West\r\n* Darius Rucker: September 20 at Ameris Bank Amphitheatre\r\n* Hail the sun and A Lot Like Birds: September 20 at The Masquerade\r\n* SOFI TUKKER: September 24 at Tabernacle\r\n* Severdust: September 25 at the The Eastern\r\n* Gracie Adams: September 30 at Coca-Cola Roxy\r\n\r\n### Take a Day Trip to North Georgia: Apple-Picking \u0026amp; Sunflowers\r\n\r\n### \r\n\r\nLet\u0026#8217;s face it \u0026#8211; once back-to-school season rolls around, we\u0026#8217;re ready for fall! While it\u0026#8217;s not quite time for pumpkins, haunted houses, and endless screenings of Hocus Pocus just yet \u0026#8211; there\u0026#8217;s plenty of early fall fun to experience in North Georgia!\r\n\r\nFor apple-picking, we love Mercier Orchards in Blue Ridge, GA. For over 80 years, they\u0026#8217;ve delighted guests with \u0026#8220;U-Pick\u0026#8221; experiences and tractor tours. Plus, visit their market to purchase delicious pies, ice cream, baked goods, fresh produce, and more! Mercier\u0026#8217;s apple season is August-October.\r\n\r\nIn mid-late September, head to family-owned Faucett Farms to see endless rows of 900,000\u002B beautiful sunflowers. Be sure to check their website before visiting, as the exact season opening date is TBD based on weather! Located just south of Amicalola Falls and near Burt\u0026#8217;s Pumpkin Farm, this day trip could also be a fun weekend trip! Head to Burt\u0026#8217;s Pumpkin Farm starting September 1, 2024 for pumpkin picking, hayrides, a country store, and more.\r\n\r\n### Additional Atlanta Ideas for September\r\n\r\nLooking for more Atlanta things to do during September? Other popular Atlanta activities in September include:\r\n\r\n* Learn about Dr. King\u0026#8217;s life and legacy on Unexpected Atlanta\u0026#8217;s top-rated Martin Luther King, Jr. Atlanta History Tour\r\n* Explore the walking and cycling paths of the Atlanta Beltline or the Silver Comet Trail\r\n* Visit one of Atlanta\u0026#8217;s many local farmers markets\r\n* See late-summer flowers and the beginnings of fall at the Atlanta Botanical Garden\r\n* Experience on-trend shopping, delicious cuisine, and a rooftop amusement park at Ponce City Market\r\n* Get wild at Zoo Atlanta or take a journey under the sea at Georgia Aquarium\r\n* Take a prohibition themed progressive dinner with a character guide from the 1920s!\r\n* Take a picnic break or toss a football at Centennial Olympic Park in Downtown Atlanta\r\n* Learn about the world\u0026#8217;s most iconic soft drink at the World of Coca-Cola\r\n* Check out bustling Atlanta suburbs like Dunwoody, Roswell, and Chamblee\r\n\r\n## FAQs: Visiting Atlanta in September\r\n\r\n### \r\n\r\n### Is September a good time of year to visit Atlanta?\r\n\r\nYes. In fact, September is one of the best times to visit Atlanta! In September, summer temperatures cool down, there\u0026#8217;s still plenty of sunlight after the 9-5 workday, and back-to-school season means that the city\u0026#8217;s popular destinations are likely to be less crowded.\r\n\r\nPlus, September in Atlanta brings a wide variety of fun \u0026#8211; from sporting events to concerts, festivals, foodie events, and more.\r\n\r\n### What is Atlanta September weather like?\r\n\r\nSeptember is a beautiful and generally comfortable time of year to visit Atlanta. September weather in Atlanta brings sunny days with mild temperatures. In fact, many people are comfortable visiting this time of year with a light sweater or jacket.\r\n\r\nAccording to the National Weather Service, September Atlanta temperatures range from lows around 66 F (19 C) to highs of 84 F (29 C), with occasional rain. During September, Atlanta typically sees 3.8 inches of rain, making this one of the least rainy months to visit. Depending when in September you\u0026#8217;re visiting, average sunrise times in Atlanta in September range from 7:11-7:30 AM. Atlanta September sunsets range from 8:03-7:23 PM as the days get shorter.\r\n\r\nTo make the most of your time in Atlanta, experience the best of the city with expert guides! Unexpected Atlanta\u0026#8217;s unique walking tours are a can\u0026#8217;t-miss activity for anyone who loves good food and history. Join us and see why we have more than 1,500 five-star reviews on TripAdvisor!\r\n\r\nBack to all posts\r\n\r\nRecent Posts* 30\u002B Best Cocktail Bars: Atlanta \u0026#038; Beyond September 1, 2024\r\n* 25\u002B Unique \u0026#038; Fun Date Ideas: Atlanta August 29, 2024\r\n* Atlanta Airport: Ultimate Guide for Travelers August 26, 2024\r\n\r\n## \r\n\r\n### See Tours Available On Your Preferred Date\r\n\r\n[](/tours/grant-park-atlanta-food-tour/)**Book Atlanta\u0027s #1 rated food tour now.**8 tastings from 3 restaurants in a 2.5 hour guided tour through historic Grant Park.### Get Atlanta Tips and Fun In Your Inbox!\r\n\r\n[](#)[Share](#)[Tweet](#)[Share](#)[Pin](#)\u0026copy; 2024 Unexpected Atlanta. All Rights Reserved.\r\n\r\n[Waiver \u0026amp; Release](/waiver-release/)|[Terms \u0026amp; Conditions](/terms-conditions/)|[Privacy Policy](/privacy-policy/)* twitter\r\n* facebook\r\n* pinterest\r\n* instagram\r\n* yelp\r\n* tripadvisor\r\n\r\n[Close Menu](#)* Home\r\n* Public Tours Atlanta Market Food Tour \u002B Biscuit Cooking Class Martin Luther King Jr Tour Cabbagetown \u0026#038; BeltLine Street Art Tour Grant Park Food Tour Prohibition Progressive Dinner\r\n* Team Building\r\n* Family and Student Groups\r\n* Things To Do: Atlanta Blog\r\n* Buy Tickets\r\n* Gift Certificates\r\n\r\n[](#)[Call Us](tel:\u002B14702232203)[Buy Tickets](https://fareharbor.com/embeds/book/atlantafoodwalks/?full-items=yes)[View Tours](/tours/)"
```

Here are some exciting events and activities happening in Atlanta on September 25, 2024:

1. **Concerts and Shows**:
    - **Lorna Shore** at The Tabernacle - GA at 6:30 PM [Tickets](https://atlanta-ga.events/tickets/lorna-shore/)
    - **Sevendust** at The Eastern - GA at 6:30 PM [Tickets](https://atlanta-ga.events/tickets/sevendust/)
    - **Creed, 3 Doors Down & Finger Eleven** at Ameris Bank Amphitheatre at 7:00 PM [Tickets](https://atlanta-ga.events/tickets/creed/)
    - **Kings of Leon & Phantogram** at State Farm Arena - GA at 7:30 PM [Tickets](https://atlanta-ga.events/tickets/kings-of-leon/)
    - **Shaboozey** at Variety Playhouse at 8:00 PM [Tickets](https://atlanta-ga.events/tickets/shaboozey/)
    - **Gritz and Jelly Butter** at City Winery - Atlanta at 8:00 PM [Tickets](https://atlanta-ga.events/tickets/gritz-and-jelly-butter/)

2. **Sports**:
    - **Atlanta Braves vs. New York Mets** at Truist Park at 7:20 PM [Tickets](https://atlanta-ga.events/tickets/atlanta-braves/)

3. **Cultural and Festival Events**:
    - **Sweet Auburn Music Fest**: Taking place from September 28-29, enjoy live music, international food court, national artists, celebrities, and various vendors on Auburn Avenue. [More Details](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-september-in-atlanta/)
    - **Harvest on the Hooch**: Happening at the Chattahoochee Nature Center on September 22, featuring food tastings, live music, garden games, and more. [More Details](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-september-in-atlanta/)

4. **Unique Experiences**:
    - **NETHERWORLD Haunted House**: Open on weekends starting September 20-21, experience two new terrifying haunts, Wake the Dead and Mr. Grendel’s Birthday Party of Horrors. [More Details](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-september-in-atlanta/)

For a full list of events and more details, you can visit these sources:
- [Cool Things To Do in September in Atlanta - Discover Atlanta](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-september-in-atlanta/)
- [Atlanta Events September 2024 - Atlanta-GA.Events](https://atlanta-ga.events/september/)
- [The Best Things to Do in Atlanta in September - Unexpected Atlanta](https://unexpectedatlanta.com/the-best-things-to-do-in-atlanta-in-september/)

Enjoy your time in Atlanta!


In [5]:
output.LastMessage.DisplayAs("text/markdown");

Here are some exciting events and activities happening in Atlanta on September 25, 2024:

1. **Concerts and Shows**:
    - **Lorna Shore** at The Tabernacle - GA at 6:30 PM [Tickets](https://atlanta-ga.events/tickets/lorna-shore/)
    - **Sevendust** at The Eastern - GA at 6:30 PM [Tickets](https://atlanta-ga.events/tickets/sevendust/)
    - **Creed, 3 Doors Down & Finger Eleven** at Ameris Bank Amphitheatre at 7:00 PM [Tickets](https://atlanta-ga.events/tickets/creed/)
    - **Kings of Leon & Phantogram** at State Farm Arena - GA at 7:30 PM [Tickets](https://atlanta-ga.events/tickets/kings-of-leon/)
    - **Shaboozey** at Variety Playhouse at 8:00 PM [Tickets](https://atlanta-ga.events/tickets/shaboozey/)
    - **Gritz and Jelly Butter** at City Winery - Atlanta at 8:00 PM [Tickets](https://atlanta-ga.events/tickets/gritz-and-jelly-butter/)

2. **Sports**:
    - **Atlanta Braves vs. New York Mets** at Truist Park at 7:20 PM [Tickets](https://atlanta-ga.events/tickets/atlanta-braves/)

3. **Cultural and Festival Events**:
    - **Sweet Auburn Music Fest**: Taking place from September 28-29, enjoy live music, international food court, national artists, celebrities, and various vendors on Auburn Avenue. [More Details](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-september-in-atlanta/)
    - **Harvest on the Hooch**: Happening at the Chattahoochee Nature Center on September 22, featuring food tastings, live music, garden games, and more. [More Details](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-september-in-atlanta/)

4. **Unique Experiences**:
    - **NETHERWORLD Haunted House**: Open on weekends starting September 20-21, experience two new terrifying haunts, Wake the Dead and Mr. Grendel’s Birthday Party of Horrors. [More Details](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-september-in-atlanta/)

For a full list of events and more details, you can visit these sources:
- [Cool Things To Do in September in Atlanta - Discover Atlanta](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-september-in-atlanta/)
- [Atlanta Events September 2024 - Atlanta-GA.Events](https://atlanta-ga.events/september/)
- [The Best Things to Do in Atlanta in September - Unexpected Atlanta](https://unexpectedatlanta.com/the-best-things-to-do-in-atlanta-in-september/)

Enjoy your time in Atlanta!

In [6]:
var assistantRunOptions = new AssistantRunOptions() {
    AssistantName = "Web Search and Scrape",
    Instructions = "Use python to get todays date and then use it to find things to do in Atlanta for each of the next four weekends. Be careful to not include events in the past in your final answer. You should perform four searches at once.",
};
var output2 = await AntRunnerLib.AssistantRunner.RunThread(assistantRunOptions, config);
output2.Dialog.DisplayAs("text/markdown");


User: Use python to get todays date and then use it to find things to do in Atlanta for each of the next four weekends. Be careful to not include events in the past in your final answer. You should perform four searches at once.

Assistant: I ran this code:
```
from datetime import datetime, timedelta

# Get today's date
today = datetime.now().date()

# Calculate the next four weekends
weekends = []
for i in range(4):
    # Calculate the upcoming Saturday
    saturday = today + timedelta((5-today.weekday()) % 7 + 7*i)
    # Add the weekend (Saturday and Sunday) to the list
    weekends.append((saturday, saturday + timedelta(1)))

weekends
```

I called the tool named crawl with these arguments:
```
{"q": "events in Atlanta September 28-29, 2024", "textFormat": "HTML"}
```
and got this result:
```
{"queryContext":{"originalQuery":"events in Atlanta September 28-29, 2024"},"webPages":{"value":[{"name":"Atlanta Events September 2024: Concerts, Shows, Sports...","url":"https://atlanta-ga.events/september/","displayUrl":"https://atlanta-ga.events/september","snippet":"Explore our comprehensive calendar for all the current and upcoming events in Atlanta, GA, for September 2024. ... Prices from $28 Avg. price ~ $33. 20 tickets remaining! Tickets; Sep. 24. 2024. 7:00 PM Tue. ... Prices from $29 Avg. price ~ $93. Available Tickets: 145. Tickets; Show More.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=events\u002Bin\u002BAtlanta\u002BSeptember\u002B28-29%2c\u002B2024\u0026d=4854189676169673\u0026mkt=en-US\u0026setlang=en-US\u0026w=73iRaVHIvJefbkmdL1lCvH4mS1bjztOY"},{"name":"Discover Events Happening this from 2024-09-28 to 2024-09-29 in Atlanta ...","url":"https://eventcartel.com/events/all-categories/atlanta/from_2024-09-28_to_2024-09-29","displayUrl":"https://eventcartel.com/events/all-categories/atlanta/from_2024-09-28_to_2024-09-29","snippet":"Discover what\u2019s happening near Atlanta this from 2024-09-28 to 2024-09-29 with EventCartel\u2019s diverse range of event offerings. From electrifying nightlife to family fun from 2024-09-28 to 2024-09-29, our platform connects you to a plethora of activities including Atlanta concerts, theaters, family events, parties, festivals, and more.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=events\u002Bin\u002BAtlanta\u002BSeptember\u002B28-29%2c\u002B2024\u0026d=593829212581\u0026mkt=en-US\u0026setlang=en-US\u0026w=ArLA-810ZZkaWFgm_VL8k_3YAMhRPChX"},{"name":"Cool Things To Do in September in Atlanta","url":"https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-september-in-atlanta/","displayUrl":"https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-september-in-atlanta","snippet":"Published August 23, 2024 Last Updated September 11, 2024. It\u2019s September. In Atlanta that means it\u2019s time for festivals, costumed aliens walking down Peachtree Street and food \u2014 glorious food. Food Extravaganza Take a big bite. There is plenty to sample at the Atlanta Food \u0026amp; Wine Festival with more than 200 chefs, sommeliers and mixologists.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=events\u002Bin\u002BAtlanta\u002BSeptember\u002B28-29%2c\u002B2024\u0026d=4692574352310340\u0026mkt=en-US\u0026setlang=en-US\u0026w=c-5Oucqqe-FV74GkIkdoRE4k4vk9MNbJ","thumbnailUrl":"https://www.bing.com/th?id=OIP.m0ENCio7ytS2o9PQRWDCRgHaD4\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Atlanta Events - September 2024 - Go South Atlanta","url":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-september.html","displayUrl":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-september.html","snippet":"More details. \u2013 Dragon Con, August 29 - September 2 2024. One of the biggest annual pop culture conventions in the country. More details. \u2013 Atlanta Black Theatre Festival, August 30 - September 1 2024. Annual festival celebrating the Black experience in the performing arts, with plays, panels, masterclasses and other events.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=events\u002Bin\u002BAtlanta\u002BSeptember\u002B28-29%2c\u002B2024\u0026d=4712253893574850\u0026mkt=en-US\u0026setlang=en-US\u0026w=aWkJWqw243Je4oOvzq1dGpeg_GE4sHjt","thumbnailUrl":"https://www.bing.com/th?id=OIP.xm-3dn3hYJZHsVVU9WO5qwHaD4\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"80\u002B fall festivals \u0026amp; fairs around metro Atlanta: Fun things to do in ...","url":"https://www.atlantaonthecheap.com/fall-festivals-atlanta/","displayUrl":"https://www.atlantaonthecheap.com/fall-festivals-atlanta","snippet":"September 29, 2024 Oakland Cemetery Atlanta. Oakland Cemetery\u2019s beloved fall festival is based on the Victorian tradition of spending a day of leisure in the cemetery. It has always been a free event \u2014 however, in 2023, it\u2019s being combined with Tunes from the Tombs and is a ticketed event. Details for this event","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=events\u002Bin\u002BAtlanta\u002BSeptember\u002B28-29%2c\u002B2024\u0026d=5012794228946274\u0026mkt=en-US\u0026setlang=en-US\u0026w=AqqJM900-WHnjjGt9vluoyq1DPm6ofe-","thumbnailUrl":"https://www.bing.com/th?id=OIP.w0-v0ll6eXl6tJfBd5-ovwHaEV\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Atlanta September 2024: Events, Concerts, Clubs \u0026amp; Things to Do - Experism","url":"https://www.experism.com/atlanta/september","displayUrl":"https://www.experism.com/atlanta/september","snippet":"Atlanta September 2024 Events Atlanta September 2024 Events Destinations Atlanta National events. Search events nationwide. Select Cities. Cancel Apply. Dates ... Sep 28-29, 2024 312 Irwin St NE View on Map. Play Video. The Sweet Auburn Festival will rock your world and your taste buds. This free two-day music festival is a time to celebrate ...","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=events\u002Bin\u002BAtlanta\u002BSeptember\u002B28-29%2c\u002B2024\u0026d=4642636760352967\u0026mkt=en-US\u0026setlang=en-US\u0026w=CrMoojBamU4_Ok9mubm7deAiSBD33s7x","thumbnailUrl":"https://www.bing.com/th?id=OIP.opEpXe5xeSIGAQVvWLFBbAHaEK\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Explore Atlanta Events for Today, This Week or Weekend","url":"https://discoveratlanta.com/events/main/","displayUrl":"https://discoveratlanta.com/events/m","snippet":"Explore the Atlanta events calendar for things to do today, this week or this weekend. Find upcoming festivals, seasonal events and more! ... Every Saturday until 09/28/2024. Protect Yo Energy Con 9/24/24 \u2013 11/9/24. Weekend Creekwater Alpaca Farm Barn Tour ... September 29. Sunday. Ellen Ector\u2019s \u2013 Aging Blackwards 5K Walk/Run 9/29/24 ...","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=events\u002Bin\u002BAtlanta\u002BSeptember\u002B28-29%2c\u002B2024\u0026d=4819881473496114\u0026mkt=en-US\u0026setlang=en-US\u0026w=zvJBiG0Dg4eP1GMx_bJv9FTM8ziERJGY","thumbnailUrl":"https://www.bing.com/th?id=OIP.FkxSV6JbrWatsFMM3_8dIAHaEh\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Upcoming Events - Discover Atlanta","url":"https://discoveratlanta.com/events/all/","displayUrl":"https://discoveratlanta.com/events/all","snippet":"Explore a list of Atlanta events and upcoming events in the Atlanta area, including events happening this weekend! ... 9/28/24 \u2013 9/29/24. Sweet Auburn Music Fest. ... Multiple dates until 09/29/2024. Roswell Wine Festival. Come out to Historic Roswell for the 11th anniversary of the Roswell Wine Festival. Take\u2026","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=events\u002Bin\u002BAtlanta\u002BSeptember\u002B28-29%2c\u002B2024\u0026d=4829094184573839\u0026mkt=en-US\u0026setlang=en-US\u0026w=J3F_lNIMaK2UBKZlUeD3eG4MmVcBwK29"},{"name":"Things to Do in Atlanta in 2024: Events, festivals, and exhibits ...","url":"https://www.thrillist.com/events/atlanta/events-calendar-in-atlanta-this-year","displayUrl":"https://www.thrillist.com/events/atlanta/events-calendar-in-atlanta-this-year","snippet":"Make plans to attend these annual events in Atlanta that keep the party going all year. By Okla Jones. Updated on July 9, 2024 at 12:40 PM. ... September 28\u201329, 2024 Old Fourth Ward","thumbnailUrl":"https://www.bing.com/th?id=OIP.WUHCPl7_RpepE3pSEdAEMgHaE8\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Concerts In Atlanta, September 2024 - Go South Atlanta","url":"https://gosouthatlanta.com/things-to-do/events/atlanta-concerts-september.html","displayUrl":"https://gosouthatlanta.com/things-to-do/events/atlanta-concerts-september.html","snippet":"Calendar of upcoming concerts in Atlanta in September 2024: rock, hip hop, pop, country, metal, classical, free concerts, tribute shows, and more. ... Concerts On Saturday September 28 2024 \u2013 Staind \u0026amp; Breaking Benjamin with Daughtry at the Ameris Bank ... Concerts On Sunday September 29 2024 \u2013 Fletcher \u2013 In Search Of The Antidote Tour at ...","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=events\u002Bin\u002BAtlanta\u002BSeptember\u002B28-29%2c\u002B2024\u0026d=4997727481785510\u0026mkt=en-US\u0026setlang=en-US\u0026w=7wIlOPp8iibgtBrujIlQ6WBS_NbfP6S4"}]}}
```

I called the tool named crawl with these arguments:
```
{"q": "events in Atlanta October 5-6, 2024", "textFormat": "HTML"}
```
and got this result:
```
{"queryContext":{"originalQuery":"events in Atlanta October 5-6, 2024"},"webPages":{"value":[{"name":"Atlanta Events - October 2024 - Go South Atlanta","url":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-october.html","displayUrl":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-october.html","snippet":"Wrestling event at State Farm Arena. More details. \u2013 Festival on Ponce, October 5-6 2024. Twice-annual arts festival in Olmsted Linear Park, with 125\u002B vendors of fine art, folk art and outsider art, plus food and drink, and children\u2019s activities. More details. \u2013 Candler Park Fall Fest, October 5-6 2024.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=events\u002Bin\u002BAtlanta\u002BOctober\u002B5-6%2c\u002B2024\u0026d=4907344187949765\u0026mkt=en-US\u0026setlang=en-US\u0026w=jtKzhu3KOsa3mkDLUvcm_m1afK8iBTpt"},{"name":"Atlanta Events October 2024: Concerts, Shows, Sports...","url":"https://atlanta-ga.events/october/","displayUrl":"https://atlanta-ga.events/october","snippet":"Explore our comprehensive calendar for all the current and upcoming events in Atlanta, GA, for October 2024. This guide is your gateway to discovering the best things to do in Atlanta, from Concerts and Theater Shows to Sports events and music festivals. You\u2019ll find a detailed list of all the most engaging and exciting activities happening throughout the month.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=events\u002Bin\u002BAtlanta\u002BOctober\u002B5-6%2c\u002B2024\u0026d=4898908879865838\u0026mkt=en-US\u0026setlang=en-US\u0026w=vC_zzFLZ91uzxHogtJy0oBuKycEPAm_-"},{"name":"80\u002B fall festivals \u0026amp; fairs around metro Atlanta: Fun things to do in ...","url":"https://www.atlantaonthecheap.com/fall-festivals-atlanta/","displayUrl":"https://www.atlantaonthecheap.com/fall-festivals-atlanta","snippet":"October 5 and 6, 2024 Candler Park Atlanta FREE. This popular street festival includes an artist market, live acoustic music on two stages, rides and special attractions, a tour of homes, food, kids activities, and more. ... October 12 and 13, 2024 Piedmont Park. An annual October event! Atlanta Pride features a festival and parade in Midtown ...","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=events\u002Bin\u002BAtlanta\u002BOctober\u002B5-6%2c\u002B2024\u0026d=5012794228946274\u0026mkt=en-US\u0026setlang=en-US\u0026w=AqqJM900-WHnjjGt9vluoyq1DPm6ofe-","thumbnailUrl":"https://www.bing.com/th?id=OIP.w0-v0ll6eXl6tJfBd5-ovwHaEV\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Cool Things To Do in October in Atlanta","url":"https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-october-in-atlanta/","displayUrl":"https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-october-in-atlanta","snippet":"Corn Dogs, Rides and Whack a Mole. The Atlanta Fair brings the best of a traditional fair to Atlanta with everything from cotton candy, a Ferris wheel, fishing for goldfish, a tilt-a-whirl and a carousel. The fair, located across from the Georgia State University Stadium, will be in town Oct. 4-Nov. 3. Read More.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=events\u002Bin\u002BAtlanta\u002BOctober\u002B5-6%2c\u002B2024\u0026d=4714289705734711\u0026mkt=en-US\u0026setlang=en-US\u0026w=6a-yO_DfCUxfz-dx9C91WDDECk49pD7V","thumbnailUrl":"https://www.bing.com/th?id=OIP.4jf9xqKu3QRk_XhZ3J7DYQHaD4\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Atlanta in October | Concerts, Festivals, Tickets \u0026amp; Tour Dates - Songkick","url":"https://www.songkick.com/metro-areas/4120-us-atlanta/october-2024","displayUrl":"https://www.songkick.com/metro-areas/4120-us-atlanta/october-2024","snippet":"Concerts in Atlanta in October. Find tickets to all live music, concerts, tour dates and festivals in and around Atlanta. Currently there are 564 upcoming events. Filter by artist. All Your favorite artists. Filter by date. All Tonight ... Tuesday 01 October 2024; The Used Plain White T\u0026#39;s and Raue. The Eastern, Atlanta, GA, US .","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=events\u002Bin\u002BAtlanta\u002BOctober\u002B5-6%2c\u002B2024\u0026d=4726551829817777\u0026mkt=en-US\u0026setlang=en-US\u0026w=GyNsJVWjOONlYyuTqLMEVsVddRXByGfo"},{"name":"Atlanta Festival Calendar: 2024","url":"https://unexpectedatlanta.com/atlanta-festival-calendar-2024/","displayUrl":"https://unexpectedatlanta.com/atlanta-festival-calendar-2024","snippet":"Atlanta Pride Festival and Parade \u2013 October 12-13, 2024. The Atlanta Pride Festival and Parade is a vibrant event that celebrates and supports the LGBTQ\u002B community. By attending, be a part of a joyful and inclusive atmosphere where diversity, equality, and love are celebrated. Little 5 Points Halloween Festival \u0026amp; Parade \u2013 October 19-20, 2024","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=events\u002Bin\u002BAtlanta\u002BOctober\u002B5-6%2c\u002B2024\u0026d=4520195830737555\u0026mkt=en-US\u0026setlang=en-US\u0026w=Y_UYHSuQi3QHjG8exIRi2zfYmaaF9rTA","thumbnailUrl":"https://www.bing.com/th?id=OIP.8LvXI6tOuWvo9KZdL-C3MwHaHa\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Concerts \u0026amp; Events in Atlanta, GA - Bandsintown","url":"https://www.bandsintown.com/c/atlanta-ga","displayUrl":"https://www.bandsintown.com/c/atlanta-ga","snippet":"October 5\u20136, 2024. Candler Park Fall Fest 2024. Candler Park - Atlanta, GA. October 18\u201320, 2024. Dharma Glow 2024. Curate - Atlanta, GA. October 5, 2024. Jazz at the Mount 2024. Mt Welcome Baptist Church - Decatur, GA. ... What are the most popular concert and event venues in Atlanta, GA? Atlanta, GA is home to many different event venues ...","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=events\u002Bin\u002BAtlanta\u002BOctober\u002B5-6%2c\u002B2024\u0026d=5019021928971711\u0026mkt=en-US\u0026setlang=en-US\u0026w=9PwJrbYlo2nqY3VQqr9oYHDepyH33NQH"},{"name":"Here are the Top Events to See this October in Atlanta","url":"https://atlantanmagazine.com/agenda-oct24","displayUrl":"https://atlantanmagazine.com/agenda-oct24","snippet":"Here are our top events to see this October in Atlanta. Brendan John Carroll, \u201CModern Marsh Land,\u201D (2023, resin, oil and wax on canvas), 16 inches by 20 inches. PHOTO COURTESY OF POEM 88. 10/3-6 Atlanta Art Fair. Art Market Productions will host the inaugural Atlanta Art Fair at Pullman Yards this October.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=events\u002Bin\u002BAtlanta\u002BOctober\u002B5-6%2c\u002B2024\u0026d=4795550485187307\u0026mkt=en-US\u0026setlang=en-US\u0026w=992-IBGAdByEw6Q9NjSl8y6nw9Sjstg2"},{"name":"Explore Atlanta Events for Today, This Week or Weekend","url":"https://discoveratlanta.com/events/main/","displayUrl":"https://discoveratlanta.com/events/m","snippet":"Explore the Atlanta events calendar for things to do today, this week or this weekend. Find upcoming festivals, seasonal events and more! ... October 10/5/24. Rise and Dine with the Animals: Gorillas Multiple dates until 10/12/2024. Roberta Lea Live ... Venture Atlanta 2024 10/8/24 \u2013 10/9/24. Atlanta miniFEST 10/9/24 \u2013 10/10/24. Homeshake","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=events\u002Bin\u002BAtlanta\u002BOctober\u002B5-6%2c\u002B2024\u0026d=4819881473496114\u0026mkt=en-US\u0026setlang=en-US\u0026w=zvJBiG0Dg4eP1GMx_bJv9FTM8ziERJGY","thumbnailUrl":"https://www.bing.com/th?id=OIP.FkxSV6JbrWatsFMM3_8dIAHaEh\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"XION Friday w/ Karlitos at Future - Events - Universe","url":"https://www.universe.com/events/xion-friday-w-karlitos-at-future-tickets-Y0DST6","displayUrl":"https://www.universe.com/events/xion-friday-w-karlitos-at-future-tickets-Y0DST6","snippet":"XION Friday w/ Karlitos at Future by Future Atlanta - Saturday, October 5, 2024 03:00 AM at 50 Lower Alabama Street, Atlanta, GA 30303, USA. Buy tickets and find information on Universe. Universe. Find Events; Host Events. ... This event will be held at Future. Times. Start: 2024-10-05T03:00:00-04:00. End: 2024-10-05T06:00:00-04:00.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=events\u002Bin\u002BAtlanta\u002BOctober\u002B5-6%2c\u002B2024\u0026d=1212323762189\u0026mkt=en-US\u0026setlang=en-US\u0026w=yhilqm2PNBioPIxy0GCCkK5WlSIJu_tY"}]},"images":{"value":[{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.CBOPpCO8Y3waxP9CuDEALwHaE8\u0026pid=Api","contentUrl":"http://nique.net/wp-content/uploads/2015/07/ShakyKnees1-982x656.jpg","hostPageUrl":"https://karylbzahara.pages.dev/cuemjln-jazz-fest-2024-atlanta-tickets-photos-hjkczwj/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.HcjQ6z-h4igpUXCCdeGRhQHaJx\u0026pid=Api","contentUrl":"https://cdn-az.allevents.in/events5/banners/97c75a3533b4caa59a6527d23af88f874706d146af619b5761d0c0a16d4aa854-rimg-w728-h960-gmir.jpg?v=1689179059","hostPageUrl":"https://editheqcharmain.pages.dev/rpwhnd-concerts-in-atlanta-2024-ivpqpqsmr-photos/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.9Xtz2sTUsM_3iBDrjvrBwwAAAA\u0026pid=Api","contentUrl":"https://images.axios.com/jQXdEyUufKrFUqNqdRB1qgkHLGY=/2022/04/06/1649258524237.jpg","hostPageUrl":"https://cassaundrawmaria.pages.dev/hkwzkr-events-in-atlanta-october-2024-riwspw/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.Bl2DjwplU79zODVAWYeZWQAAAA\u0026pid=Api","contentUrl":"https://www.lilwaynehq.com/images/blog/lil-wayne-headline-one-musicfest-atlanta-october2.jpg","hostPageUrl":"https://dorrieqhildegarde.pages.dev/tzchdz-music-festival-in-atlanta-october-2024-dcbgdp/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.e_L3Pn468GRPqw2ze_jK-AHaHa\u0026pid=Api","contentUrl":"https://static.wixstatic.com/media/f9ea57_407089b938c34658bb5208af0b7bee04~mv2.jpeg/v1/fill/w_1280,h_1280,al_c,q_85,enc_auto/WhatsApp%20Image%202023-06-08%20at%2012_24_31.jpeg","hostPageUrl":"https://www.atlantacarnival.org/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.WoE56TfoQ63T7_SpwvmGKgHaE8\u0026pid=Api","contentUrl":"https://www.experism.com/explore/wp-content/uploads/2023/02/Atlanta-1550x1034.jpg","hostPageUrl":"https://maggeewkimmy.pages.dev/tbgwnw-eventos-atlanta-2024-tcnwiz/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.mSINMavL_OAioQHRCxqbAQAAAA\u0026pid=Api","contentUrl":"https://cdn2.allevents.in/thumbs/thumb648a895bc9367.jpg","hostPageUrl":"https://livaqgeorgena.pages.dev/kiynhg-atlanta-new-years-2024-ctdrpkhrf-photos/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.kBui6aaDMHm147Hqnp3iMwHaEK\u0026pid=Api","contentUrl":"https://img.youtube.com/vi/LFHz7N6gzcE/maxresdefault.jpg","hostPageUrl":"https://elsieemmeline.pages.dev/ypimhvd-atlanta-concerts-2024-calendar-of-events-photos-smzdulp/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.ZhPF1J7Tuazkx5WbE5qaUQAAAA\u0026pid=Api","contentUrl":"https://www.foxtheatre.org/assets/img/new-atl-comedy-jam-2220x1140-2-e6477d1197.jpg","hostPageUrl":"https://maggeewkimmy.pages.dev/tbgwnw-eventos-atlanta-2024-tcnwiz/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIF.L5Ftbh%2ffXMV6BbX6EWERgQ\u0026pid=Api","contentUrl":"https://365atlantatraveler.com/wp-content/uploads/2021/09/HALLOWEEN-EVENTS-2.jpg","hostPageUrl":"https://ricibannmarie.pages.dev/mkdjdha-events-in-october-2024-near-me-for-adults-images-azvsqfe/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.Ctt7cutpo0kduVop1QcE-AAAAA\u0026pid=Api","contentUrl":"https://res.cloudinary.com/atlanta/images/f_auto,q_auto/w_2560,h_1565/v1601396905/newAtlanta.com/Atlanta_OneMusicFest_COP-scaled/Atlanta_OneMusicFest_COP-scaled.jpg","hostPageUrl":"https://discoveratlanta.com/events/main/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.CtAVxgjkXx1Xn73fzgt0zAHaEK\u0026pid=Api","contentUrl":"https://www.advocate.com/media-library/pride-parade.jpg?id=33782068\u0026width=980\u0026quality=85","hostPageUrl":"https://violettawfulvia.pages.dev/icwbmi-pride-week-atlanta-2024-dgkcpb/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.zco_XrsR7HWXyvmCHgOF7wAAAA\u0026pid=Api","contentUrl":"https://res.cloudinary.com/atlanta/images/f_auto,q_auto/v1602606068/newAtlanta.com/Atlanta_OneMusicFest_Crowd/Atlanta_OneMusicFest_Crowd.jpg","hostPageUrl":"https://discoveratlanta.com/things-to-do/nightlife/best-festivals-in-the-atl/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.xm-3dn3hYJZHsVVU9WO5qwHaD4\u0026pid=Api","contentUrl":"https://gosouthatlanta.com/images/atlanta-tree-canopy-social-media.jpg","hostPageUrl":"https://gosouthatlanta.com/things-to-do/events/musicals-in-atlanta.html"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIF.%2f7xM2vDII3%2f%2bAAqD9hkrVA\u0026pid=Api","contentUrl":"https://menusall.com/wp-content/uploads/2024/01/Upcoming-Festivals-in-Florida-October-2024.webp","hostPageUrl":"https://dinaywilmette.pages.dev/rztondh-october-2024-festivals-in-ga-images-wradzck/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIF.XT1OwgQKRVd2itXsxBJl%2bA\u0026pid=Api","contentUrl":"https://images.foxtv.com/static.fox5atlanta.com/www.fox5atlanta.com/content/uploads/2024/09/764/432/Orange-And-Black-Festive-Fall-Instagram-Post.jpg?ve=1\u0026tl=1","hostPageUrl":"https://www.fox5atlanta.com/news/fall-festivals-metro-atlanta-north-georgia-2024"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIF.CbvWp6sM1134JkgLuHHWgA\u0026pid=Api","contentUrl":"https://365atlantatraveler.com/wp-content/uploads/2018/02/Street-Dance-featuring-the-Swingin-Medallions.jpg","hostPageUrl":"https://marcibchristin.pages.dev/sprlpwz-art-festivals-atlanta-2024-dates-images-zmapyul/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.74DCwqZ91dlxldGmbHyuegHaEK\u0026pid=Api","contentUrl":"https://i.ytimg.com/vi/iWhs-q4ulrc/maxresdefault.jpg","hostPageUrl":"https://www.youtube.com/watch?v=iWhs-q4ulrc"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIF.sT%2bGTlb4oUt9XGn2cQmCFw\u0026pid=Api","contentUrl":"https://jobfairpro.com/wp-content/uploads/2020/07/Atlanta-Job-Fairs-Virtual-Events-Job-Fair-Pro.jpg","hostPageUrl":"https://eddyymarley.pages.dev/knzlour-job-fair-atlanta-2024-tickets-fwozjln/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIF.jjuH%2bZbZemR3fp%2bs%2b2bgeQ\u0026pid=Api","contentUrl":"https://www.atlantamotorspeedway.com/images/2024_eventswebheaderupdate_v1-lg.jpg","hostPageUrl":"https://dionisytammara.pages.dev/hprpekn-atlanta-event-calendar-october-2025-atlanta-qhjdwma/"}]}}
```

I called the tool named crawl with these arguments:
```
{"q": "events in Atlanta October 12-13, 2024", "textFormat": "HTML"}
```
and got this result:
```
{"queryContext":{"originalQuery":"events in Atlanta October 12-13, 2024"},"webPages":{"value":[{"name":"Atlanta Events - October 2024 - Go South Atlanta","url":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-october.html","displayUrl":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-october.html","snippet":"Events From October 1 To The Weekend Of October 5-6 2024. \u2192 See September events. \u2192 For concerts this week, see Atlanta concerts in October 2024. \u2013 Daddy Long Legs, September 13 - October 6 2024. Musical at The Legacy Theatre, Tyrone. More details.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=events\u002Bin\u002BAtlanta\u002BOctober\u002B12-13%2c\u002B2024\u0026d=4907344187949765\u0026mkt=en-US\u0026setlang=en-US\u0026w=jtKzhu3KOsa3mkDLUvcm_m1afK8iBTpt"},{"name":"Atlanta Events October 2024: Concerts, Shows, Sports...","url":"https://atlanta-ga.events/october/","displayUrl":"https://atlanta-ga.events/october","snippet":"Check Out The Atlanta Events Calendar for October 2024: Current \u0026amp; Upcoming Events \u002B Best \u0026amp; Fun Thing To Do in the City.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=events\u002Bin\u002BAtlanta\u002BOctober\u002B12-13%2c\u002B2024\u0026d=4898908879865838\u0026mkt=en-US\u0026setlang=en-US\u0026w=vC_zzFLZ91uzxHogtJy0oBuKycEPAm_-"},{"name":"80\u002B fall festivals \u0026amp; fairs around metro Atlanta: Fun things to do in ...","url":"https://www.atlantaonthecheap.com/fall-festivals-atlanta/","displayUrl":"https://www.atlantaonthecheap.com/fall-festivals-atlanta","snippet":"The best fall festivals in Atlanta and suburbs -- including festivals, fairs, and carnivals in September, October, and November.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=events\u002Bin\u002BAtlanta\u002BOctober\u002B12-13%2c\u002B2024\u0026d=5012794228946274\u0026mkt=en-US\u0026setlang=en-US\u0026w=AqqJM900-WHnjjGt9vluoyq1DPm6ofe-","thumbnailUrl":"https://www.bing.com/th?id=OIP.w0-v0ll6eXl6tJfBd5-ovwHaEV\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Atlanta Festival Calendar: 2024","url":"https://unexpectedatlanta.com/atlanta-festival-calendar-2024/","displayUrl":"https://unexpectedatlanta.com/atlanta-festival-calendar-2024","snippet":"Atlanta Pride Festival and Parade \u2013 October 12-13, 2024 The Atlanta Pride Festival and Parade is a vibrant event that celebrates and supports the LGBTQ\u002B community.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=events\u002Bin\u002BAtlanta\u002BOctober\u002B12-13%2c\u002B2024\u0026d=4520195830737555\u0026mkt=en-US\u0026setlang=en-US\u0026w=Y_UYHSuQi3QHjG8exIRi2zfYmaaF9rTA","thumbnailUrl":"https://www.bing.com/th?id=OIP.8LvXI6tOuWvo9KZdL-C3MwHaHa\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Atlanta in October | Concerts, Festivals, Tickets \u0026amp; Tour Dates - Songkick","url":"https://www.songkick.com/metro-areas/4120-us-atlanta/october-2024","displayUrl":"https://www.songkick.com/metro-areas/4120-us-atlanta/october-2024","snippet":"Find live music near you. Buy tickets for every upcoming concert, festival, gig and tour date taking place in Atlanta in October.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=events\u002Bin\u002BAtlanta\u002BOctober\u002B12-13%2c\u002B2024\u0026d=4726551829817777\u0026mkt=en-US\u0026setlang=en-US\u0026w=GyNsJVWjOONlYyuTqLMEVsVddRXByGfo"},{"name":"Atlanta Concert Tickets - Upcoming event tickets in ATL | Ticketmaster","url":"https://www.ticketmaster.com/discover/concerts/atlanta","displayUrl":"https://www.ticketmaster.com/discover/concerts/atlanta","snippet":"Buy Atlanta concert tickets on Ticketmaster. Find your favorite Music event tickets, schedules and seating charts in the Atlanta area."},{"name":"Atlanta Events \u2013 Upcoming Festivals in Atlanta 2024 - Go South Atlanta","url":"https://gosouthatlanta.com/things-to-do/events/atlanta-festivals.html","displayUrl":"https://gosouthatlanta.com/things-to-do/events/atlanta-festivals.html","snippet":"\u2013 Atlanta Pride Festival, October 12-13 2024 One of the biggest Pride festivals in the country, held in Piedmont Park, with live performances, parade, marches, marketplace and more.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=events\u002Bin\u002BAtlanta\u002BOctober\u002B12-13%2c\u002B2024\u0026d=4630752588355502\u0026mkt=en-US\u0026setlang=en-US\u0026w=pglyf_s8wUM50qWfsSic_uix0GcCbDYr","thumbnailUrl":"https://www.bing.com/th?id=OIP.1kWLxDO_JldjYbZyTlfSrQAAAA\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Concerts In Atlanta, October 2024 - Go South Atlanta","url":"https://gosouthatlanta.com/things-to-do/events/atlanta-concerts-october.html","displayUrl":"https://gosouthatlanta.com/things-to-do/events/atlanta-concerts-october.html","snippet":"Calendar of upcoming concerts in Atlanta in October 2024, including hip hop, rock, pop, country, metal, classical, tribute shows, and more.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=events\u002Bin\u002BAtlanta\u002BOctober\u002B12-13%2c\u002B2024\u0026d=4991061691999650\u0026mkt=en-US\u0026setlang=en-US\u0026w=I17XSEQ6_0vdrEf4HxlkWHNc_Lnct5jK"},{"name":"2024 Atlanta Pride Festival - Midtown Atlanta","url":"https://www.midtownatl.com/do/2024-atlanta-pride-festival","displayUrl":"https://www.midtownatl.com/do/2024-atlanta-pride-festival","snippet":"Join us at this year\u2019s Atlanta Pride Festival in Piedmont Park on October 12-13, 2024. Upgrade your Atlanta Pride Weekend with access to our luxurious Premium Lounge, where you\u0026#39;ll enjoy covered seating, premium bathroom trailers, private bars, delicious snacks, phone charging stations, video monitors, and exclusive access to meet-and-greet ...","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=events\u002Bin\u002BAtlanta\u002BOctober\u002B12-13%2c\u002B2024\u0026d=4983932044520816\u0026mkt=en-US\u0026setlang=en-US\u0026w=nCLkuucjn9Labi-rueMxEJinIupkoAdk"},{"name":"Explore Atlanta Events for Today, This Week or Weekend","url":"https://discoveratlanta.com/events/main/","displayUrl":"https://discoveratlanta.com/events/m","snippet":"Search the Atlanta events calendar below or explore our annual festivals and seasonal events taking place throughout the year. If you\u2019re in ATL now, make sure you see what\u2019s happening today, this week and find things to do this weekend so you don\u2019t miss any one-of-a-kind Atlanta events.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=events\u002Bin\u002BAtlanta\u002BOctober\u002B12-13%2c\u002B2024\u0026d=4819881473496114\u0026mkt=en-US\u0026setlang=en-US\u0026w=zvJBiG0Dg4eP1GMx_bJv9FTM8ziERJGY","thumbnailUrl":"https://www.bing.com/th?id=OIP.FkxSV6JbrWatsFMM3_8dIAHaEh\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"}]}}
```

I called the tool named crawl with these arguments:
```
{"q": "events in Atlanta October 19-20, 2024", "textFormat": "HTML"}
```
and got this result:
```
{"queryContext":{"originalQuery":"events in Atlanta October 19-20, 2024"},"webPages":{"value":[{"name":"Atlanta Events - October 2024 - Go South Atlanta","url":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-october.html","displayUrl":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-october.html","snippet":"\u2013 PNC Atlanta 10 Miler \u0026amp; 5K, October 19-20 2024 Annual 10-mile, 5K and 1-mile youth runs through Buckhead, from Atlantic Station. ... October 19-20, 26-27 2024 Annual Halloween event at Zoo Atlanta, with trick or treating, Halloween-themed activities, and more. Costumes welcome.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=events\u002Bin\u002BAtlanta\u002BOctober\u002B19-20%2c\u002B2024\u0026d=4907344187949765\u0026mkt=en-US\u0026setlang=en-US\u0026w=jtKzhu3KOsa3mkDLUvcm_m1afK8iBTpt"},{"name":"Atlanta Events October 2024: Concerts, Shows, Sports...","url":"https://atlanta-ga.events/october/","displayUrl":"https://atlanta-ga.events/october","snippet":"Explore our comprehensive calendar for all the current and upcoming events in Atlanta, GA, for October 2024. This guide is your gateway to discovering the best things to do in Atlanta, from Concerts and Theater Shows to Sports events and music festivals. You\u2019ll find a detailed list of all the most engaging and exciting activities happening throughout the month.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=events\u002Bin\u002BAtlanta\u002BOctober\u002B19-20%2c\u002B2024\u0026d=4898908879865838\u0026mkt=en-US\u0026setlang=en-US\u0026w=vC_zzFLZ91uzxHogtJy0oBuKycEPAm_-"},{"name":"Cool Things To Do in October in Atlanta","url":"https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-october-in-atlanta/","displayUrl":"https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-october-in-atlanta","snippet":"Cool Things To Do in October in Atlanta. By Mary Welch. Published September 13, 2024 Last Updated September 19, 2024. The weather has cooled off and Atlanta is ready to embrace the fall with new exhibits, fashion shows and festivals \u2014 lots and lots of festivals. October always brings Atlanta Pride. Prepare with our Pride Survival Guide.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=events\u002Bin\u002BAtlanta\u002BOctober\u002B19-20%2c\u002B2024\u0026d=4714289705734711\u0026mkt=en-US\u0026setlang=en-US\u0026w=6a-yO_DfCUxfz-dx9C91WDDECk49pD7V","thumbnailUrl":"https://www.bing.com/th?id=OIP.4jf9xqKu3QRk_XhZ3J7DYQHaD4\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"80\u002B fall festivals \u0026amp; fairs around metro Atlanta: Fun things to do in ...","url":"https://www.atlantaonthecheap.com/fall-festivals-atlanta/","displayUrl":"https://www.atlantaonthecheap.com/fall-festivals-atlanta","snippet":"An annual October event! Atlanta Pride features a festival and parade in Midtown, plus a wide variety of associated events around town. Details for Atlanta Pride. Deposit Photos. \u2666 Georgia Apple Festival. October 12 and 13, 2024 October 19 and 20, 2024 Ellijay, GA . This is one of the few festivals outside metro Atlanta we include. It takes ...","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=events\u002Bin\u002BAtlanta\u002BOctober\u002B19-20%2c\u002B2024\u0026d=5012794228946274\u0026mkt=en-US\u0026setlang=en-US\u0026w=AqqJM900-WHnjjGt9vluoyq1DPm6ofe-","thumbnailUrl":"https://www.bing.com/th?id=OIP.w0-v0ll6eXl6tJfBd5-ovwHaEV\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Atlanta in October | Concerts, Festivals, Tickets \u0026amp; Tour Dates - Songkick","url":"https://www.songkick.com/metro-areas/4120-us-atlanta/october-2024","displayUrl":"https://www.songkick.com/metro-areas/4120-us-atlanta/october-2024","snippet":"Concerts in Atlanta in October. Find tickets to all live music, concerts, tour dates and festivals in and around Atlanta. ... Sat 19 Oct 2024 Boot Barn Hall at Bourbon Brothers Gainesville, GA, US. ... Sun 20 Oct 2024 The Masquerade - Hell Atlanta, GA, US. pigeon pit. Mon 28 Oct 2024 Wallers Decatur, GA, US. Oso Oso. Sat 05 Oct 2024 The ...","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=events\u002Bin\u002BAtlanta\u002BOctober\u002B19-20%2c\u002B2024\u0026d=4726551829817777\u0026mkt=en-US\u0026setlang=en-US\u0026w=GyNsJVWjOONlYyuTqLMEVsVddRXByGfo"},{"name":"Atlanta Festival Calendar: 2024","url":"https://unexpectedatlanta.com/atlanta-festival-calendar-2024/","displayUrl":"https://unexpectedatlanta.com/atlanta-festival-calendar-2024","snippet":"Atlanta Pride Festival and Parade \u2013 October 12-13, 2024. The Atlanta Pride Festival and Parade is a vibrant event that celebrates and supports the LGBTQ\u002B community. By attending, be a part of a joyful and inclusive atmosphere where diversity, equality, and love are celebrated. Little 5 Points Halloween Festival \u0026amp; Parade \u2013 October 19-20, 2024","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=events\u002Bin\u002BAtlanta\u002BOctober\u002B19-20%2c\u002B2024\u0026d=4520195830737555\u0026mkt=en-US\u0026setlang=en-US\u0026w=Y_UYHSuQi3QHjG8exIRi2zfYmaaF9rTA","thumbnailUrl":"https://www.bing.com/th?id=OIP.8LvXI6tOuWvo9KZdL-C3MwHaHa\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"The Upside Down | October 19-20 | Epic Events and Entertainment - Epic Cons","url":"https://www.epiccons.com/events/the-upside-down","displayUrl":"https://www.epiccons.com/events/the-upside-down","snippet":"Experience The Upside Down event on October 19-20, 2024, featuring celebrity panels, exclusive photo ops, and autograph sessions from celebrities from Stranger Things! Dive into the world of your favorite shows and meet fellow fans! ... Events. The Upside Down (October 19-20) I Was Feeling Festive 2 (December 6-8) 2025 Events. I Was Feeling ...","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=events\u002Bin\u002BAtlanta\u002BOctober\u002B19-20%2c\u002B2024\u0026d=4700464208291417\u0026mkt=en-US\u0026setlang=en-US\u0026w=_24g-Ou5FBxZhmTFjyQoH8RhyWcnYm_y","thumbnailUrl":"https://www.bing.com/th?id=OIP.Wm4-6OSBCztRSKoEZnMp7AHaEL\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Explore Atlanta Events for Today, This Week or Weekend","url":"https://discoveratlanta.com/events/main/","displayUrl":"https://discoveratlanta.com/events/m","snippet":"Explore the Atlanta events calendar for things to do today, this week or this weekend. ... Usher Performing in Atlanta Multiple dates until 10/20/2024. Atlanta History Center 9/24/24 \u2013 9/30/24. Georgia Aquarium 9/24/24 \u2013 9/30/24. ... Trivia Night at Wild Leap Atlanta Every Tuesday until 11/19/2024. Elements- Surya Ensemble 9/24/24 \u2013 10/10/24.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=events\u002Bin\u002BAtlanta\u002BOctober\u002B19-20%2c\u002B2024\u0026d=4819881473496114\u0026mkt=en-US\u0026setlang=en-US\u0026w=zvJBiG0Dg4eP1GMx_bJv9FTM8ziERJGY","thumbnailUrl":"https://www.bing.com/th?id=OIP.FkxSV6JbrWatsFMM3_8dIAHaEh\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Atlanta Events \u2013 Upcoming Festivals in Atlanta 2024 - Go South Atlanta","url":"https://gosouthatlanta.com/things-to-do/events/atlanta-festivals.html","displayUrl":"https://gosouthatlanta.com/things-to-do/events/atlanta-festivals.html","snippet":"The biggest and best events in Atlanta in the coming year, with links to monthly calendars of more upcoming events, concerts, shows and other things to do in Atlanta. Go South! Atlanta; ... October 19-20 2024 Annual arts and crafts festival in Johns Creek featuring 140 artisans from across the country. More details \u2013 Landlocked 8, October 20 ...","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=events\u002Bin\u002BAtlanta\u002BOctober\u002B19-20%2c\u002B2024\u0026d=4630752588355502\u0026mkt=en-US\u0026setlang=en-US\u0026w=pglyf_s8wUM50qWfsSic_uix0GcCbDYr","thumbnailUrl":"https://www.bing.com/th?id=OIP.1kWLxDO_JldjYbZyTlfSrQAAAA\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Concerts In Atlanta, October 2024 - Go South Atlanta","url":"https://gosouthatlanta.com/things-to-do/events/atlanta-concerts-october.html","displayUrl":"https://gosouthatlanta.com/things-to-do/events/atlanta-concerts-october.html","snippet":"Concerts On Friday October 11 2024. \u2013 Lake Street Dive at The Eastern. More details. \u2013 Aerosmith: Peace Out The Farewell Tour at State Farm Arena. More details. \u2013 The Black Jacket Symphony: Performing Led Zeppelin\u2019s \u2018IV\u2019. Tribute concert at Atlanta Symphony Hall. More details. \u2013 Loveless at The Masquerade.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=events\u002Bin\u002BAtlanta\u002BOctober\u002B19-20%2c\u002B2024\u0026d=4991061691999650\u0026mkt=en-US\u0026setlang=en-US\u0026w=I17XSEQ6_0vdrEf4HxlkWHNc_Lnct5jK"}]}}
```

I called the tool named GetContentFromUrl with these arguments:
```
{"url": "https://atlanta-ga.events/september/"}
```
and got this result:
```
"Atlanta Events September 2024: Concerts, Shows, Sports...JavaScript support required!\r\n\r\n[](https://atlanta-ga.events)\u00D7Categories[All Events](https://atlanta-ga.events/)[Concerts](https://atlanta-ga.events/concerts/)[Shows](https://atlanta-ga.events/shows/)[Comedy](https://atlanta-ga.events/comedy-shows/)[Sports](https://atlanta-ga.events/sports/)[Family Shows](https://atlanta-ga.events/family-shows/)[Festivals](https://atlanta-ga.events/festivals/)[Opera](https://atlanta-ga.events/opera-tickets/)[Venues](https://atlanta-ga.events/entertainment-venues/)* All Events\r\n* Concerts\r\n* Shows\r\n* Comedy\r\n* Sports\r\n* More Family Shows Festivals Opera Venues\r\n\r\n* Home\r\n* September\r\n\r\n# Atlanta Events September 2024\r\n\r\nExplore our comprehensive calendar for all the current and upcoming events in Atlanta, GA, for September 2024. This guide is your gateway to discovering the best things to do in Atlanta, from Concerts and Theater Shows to Sports events and music festivals. You\u2019ll find a detailed list of all the most engaging and exciting activities happening throughout the month. Easily search and select your preferred events in Atlanta with our user-friendly calendar. Secure your tickets with 100% assurance at the lowest prices.\r\n\r\nStart DateSep 25, 2024ResetCancelApplyEnd DateSep 27, 2024CategoryAll EventsConcertsTheaterSportsSearchContentsUpcoming EventsList Of All EventsAtlanta, GA## Upcoming Events \u0026 Tickets\r\n\r\nSelect Your DatesResetCancelApplyUpcoming Events: 108* Sep 25 2024 6:30 PM Wed Lorna Shore The Tabernacle - GA | Capacity: 2600 30303, 152 Luckie St, Atlanta, GA, US Prices from $20 Avg. price ~ $89 Available Tickets: 81 Tickets\r\n* Sep 25 2024 6:30 PM Wed Sevendust The Eastern - GA 30312, 800 Old Flat Shoals Road SE, Atlanta, GA, US Prices from $39 Avg. price ~ $61 21 tickets remaining! Tickets\r\n* Sep 25 2024 7:00 PM Wed Creed, 3 Doors Down \u0026 Finger Eleven Ameris Bank Amphitheatre | Capacity: 12000 30004, 2200 Encore Parkway, Alpharetta, GA, US Prices from $39 Avg. price ~ $223 Available Tickets: 223 Tickets\r\n* Sep 25 2024 7:20 PM Wed Atlanta Braves vs. New York Mets Truist Park | Capacity: 41149 30339, 1100 Circle 75 Pkwy, Atlanta, GA, US Prices from $7 Avg. price ~ $75 Available Tickets: 1977 Tickets\r\n* Sep 25 2024 7:30 PM Wed Kings of Leon \u0026 Phantogram State Farm Arena - GA | Capacity: 21000 30303, One Philips Drive, Atlanta, GA, US Prices from $14 Avg. price ~ $123 Available Tickets: 804 Tickets\r\n* Sep 25 2024 8:00 PM Wed Shaboozey Variety Playhouse | Capacity: 1000 30307, 1099 Euclid Ave, Atlanta, GA, US Prices from $51 Avg. price ~ $88 28 tickets remaining! Tickets\r\n* Sep 25 2024 8:00 PM Wed Lasso - Artist Buckhead Theatre | Capacity: 2500 30305, 3110 Roswell Road, Atlanta, GA, US Prices from $58 Avg. price ~ $90 Available Tickets: 54 Tickets\r\n* Sep 25 2024 8:00 PM Wed Gritz and Jelly Butter City Winery - Atlanta | Capacity: 300 30308, 675 Ponce De Leon Ave, Atlanta, GA, US Prices from $74 Avg. price ~ $89 8 tickets remaining! Tickets\r\n* Sep 26 2024 6:30 PM Thu Rain City Drive Hell at The Masquerade - GA | Capacity: 300 30303, 75 MLK Jr Dr SW, Atlanta, GA, US Prices from $34 Avg. price ~ $40 22 tickets remaining! Tickets\r\n* Sep 26 2024 7:00 PM Thu Mico Purgatory Stage at Masquerade - GA | Capacity: 300 30303, 75 MLK Jr Dr SW, Atlanta, GA, US Prices from $37 Avg. price ~ $96 31 tickets remaining! Tickets\r\n* Sep 26 2024 7:00 PM Thu Lyfe Jennings City Winery - Atlanta | Capacity: 300 30308, 675 Ponce De Leon Ave, Atlanta, GA, US Prices from $98 Avg. price ~ $113 4 tickets remaining! Tickets\r\n* Sep 26 2024 7:20 PM Thu Atlanta Braves vs. New York Mets Truist Park | Capacity: 41149 30339, 1100 Circle 75 Pkwy, Atlanta, GA, US Prices from $16 Avg. price ~ $136 Available Tickets: 277 Tickets\r\n* Sep 26 2024 7:30 PM Thu Khruangbin Fabulous Fox Theatre - Atlanta | Capacity: 4678 30305, 660 Peachtree St NE, Atlanta, GA, US Prices from $41 Avg. price ~ $83 Available Tickets: 149 Tickets\r\n* Sep 26 2024 7:30 PM Thu Disney On Ice Gas South Arena | Capacity: 13100 30097, 6400 Sugarloaf Parkway, Duluth, GA, US Prices from $36 Avg. price ~ $91 Available Tickets: 894 Tickets\r\n* Sep 26 2024 8:00 PM Thu Atlanta Symphony Orchestra: Nathalie Stutzmann - Mahler \u0026 Shostakovich Atlanta Symphony Hall | Capacity: 1762 30309, 1280 Peachtree St, Atlanta, GA, US Prices from $55 Avg. price ~ $116 Available Tickets: 333 Tickets\r\n* Sep 26 2024 8:00 PM Thu Lawrence The Tabernacle - GA | Capacity: 2600 30303, 152 Luckie St, Atlanta, GA, US Prices from $25 Avg. price ~ $80 Available Tickets: 106 Tickets\r\n* Sep 26 2024 8:00 PM Thu The Buena Vista Social Orchestra Center Stage Theatre | Capacity: 1050 30309, 1374 West Peachtree Street Northwest, Atlanta, GA, US Prices from $64 Avg. price ~ $101 Available Tickets: 255 Tickets\r\n* Sep 26 2024 8:00 PM Thu The California Honeydrops Variety Playhouse | Capacity: 1000 30307, 1099 Euclid Ave, Atlanta, GA, US Prices from $33 Avg. price ~ $54 25 tickets remaining! Tickets\r\n* Sep 26 2024 8:00 PM Thu Jesus Adrian Romero Cobb Energy Performing Arts Centre | Capacity: 2750 30339, 2800 Cobb Galleria Pkwy, Atlanta, GA, US Prices from $58 Avg. price ~ $214 Available Tickets: 125 Tickets\r\n* Sep 26 2024 8:00 PM Thu Wasia Project Terminal West At King Plow Arts Center | Capacity: 625 30318, 887 West Marietta Studio C, Atlanta, GA, US Prices from $30 Avg. price ~ $43 19 tickets remaining! Tickets\r\n* Sep 26 2024 10:00 PM Thu Lyfe Jennings City Winery - Atlanta | Capacity: 300 30308, 675 Ponce De Leon Ave, Atlanta, GA, US Prices from $98 Avg. price ~ $124 16 tickets remaining! Tickets\r\n* Sep 27 2024 11:00 AM Fri Disney On Ice Gas South Arena | Capacity: 13100 30097, 6400 Sugarloaf Parkway, Duluth, GA, US Prices from $39 Avg. price ~ $85 Available Tickets: 639 Tickets\r\n* Sep 27 2024 6:00 PM Fri Red Lace Burlesque Show and Variety Show Sanctuary Nightclub - Atlanta 30305, 3209 Paces Ferry Pl NW, Atlanta, GA, US Prices from $51 Avg. price ~ $405 Available Tickets: 60 Tickets\r\n* Sep 27 2024 7:00 PM Fri Georgia Tech Yellow Jackets Women\u0027s Volleyball vs. Southern Methodist (SMU) Mustangs O\u0027Keefe Gymnasium 30332, 151 Sixth Street NW, Atlanta, GA, US Prices from $48 Avg. price ~ $55 2 tickets remaining! Tickets\r\n* Sep 27 2024 7:00 PM Fri Coral Moons Altar at Masquerade - GA 30303, 75 MLK Jr Dr SW, Atlanta, GA, US Prices from $28 Avg. price ~ $33 16 tickets remaining! Tickets\r\n* Sep 27 2024 7:00 PM Fri The Red Pears Purgatory Stage at Masquerade - GA | Capacity: 300 30303, 75 MLK Jr Dr SW, Atlanta, GA, US Prices from $35 Avg. price ~ $45 36 tickets remaining! Tickets\r\n* Sep 27 2024 7:00 PM Fri The Warning Hell at The Masquerade - GA | Capacity: 300 30303, 75 MLK Jr Dr SW, Atlanta, GA, US Prices from $38 Avg. price ~ $86 Available Tickets: 79 Tickets\r\n* Sep 27 2024 7:00 PM Fri Lyfe Jennings City Winery - Atlanta | Capacity: 300 30308, 675 Ponce De Leon Ave, Atlanta, GA, US Prices from $148 Avg. price ~ $171 2 tickets remaining! Tickets\r\n* Sep 27 2024 7:20 PM Fri Atlanta Braves vs. Kansas City Royals VIP Packages Truist Park | Capacity: 41149 30339, 1100 Circle 75 Pkwy, Atlanta, GA, US Prices from $10 Avg. price ~ $69 Available Tickets: 2510 Tickets\r\n* Sep 27 2024 7:30 PM Fri T.K. Kirkland Uptown Comedy Corner 30318, 800 Marietta St, Atlanta, GA, US Prices from $78 Avg. price ~ $114 10 tickets remaining! Tickets\r\n\r\nShow More### All Events This Month\r\n\r\n[Sevendust Upcoming Events: 2 The Eastern - GA Sep 25, 2024 6:30 PM](https://atlanta-ga.events/tickets/sevendust/)[Lorna Shore Upcoming Events: 2 The Tabernacle - GA Sep 25, 2024 6:30 PM](https://atlanta-ga.events/tickets/lorna-shore/)[Finger Eleven Upcoming Events: 2 Ameris Bank Amphitheatre Sep 25, 2024 7:00 PM](https://atlanta-ga.events/tickets/finger-eleven/)[Creed Upcoming Events: 2 Ameris Bank Amphitheatre Sep 25, 2024 7:00 PM](https://atlanta-ga.events/tickets/creed/)[3 Doors Down Upcoming Events: 2 Ameris Bank Amphitheatre Sep 25, 2024 7:00 PM](https://atlanta-ga.events/tickets/3-doors-down/)[New York Mets Upcoming Events: 6 Truist Park Sep 25, 2024 7:20 PM](https://atlanta-ga.events/tickets/new-york-mets/)[Atlanta Braves Upcoming Events: 15 Truist Park Sep 25, 2024 7:20 PM](https://atlanta-ga.events/tickets/atlanta-braves/)[Phantogram Upcoming Events: 2 State Farm Arena - GA Sep 25, 2024 7:30 PM](https://atlanta-ga.events/tickets/phantogram/)[Kings Of Leon Upcoming Events: 2 State Farm Arena - GA Sep 25, 2024 7:30 PM](https://atlanta-ga.events/tickets/kings-of-leon/)[Lasso - Artist Upcoming Events: 2 Buckhead Theatre Sep 25, 2024 8:00 PM](https://atlanta-ga.events/tickets/lasso-artist/)[Shaboozey Upcoming Events: 2 Variety Playhouse Sep 25, 2024 8:00 PM](https://atlanta-ga.events/tickets/shaboozey/)[Gritz and Jelly Butter Upcoming Events: 2 City Winery - Atlanta Sep 25, 2024 8:00 PM](https://atlanta-ga.events/tickets/gritz-and-jelly-butter/)[Rain City Drive Upcoming Events: 2 Hell at The Masquerade - GA Sep 26, 2024 6:30 PM](https://atlanta-ga.events/tickets/rain-city-drive/)[Lyfe Jennings Upcoming Events: 6 City Winery - Atlanta Sep 26, 2024 7:00 PM](https://atlanta-ga.events/tickets/lyfe-jennings/)[Mico Upcoming Events: 2 Purgatory Stage at Masquerade - GA Sep 26, 2024 7:00 PM](https://atlanta-ga.events/tickets/mico/)[Khruangbin Upcoming Events: 6 Fabulous Fox Theatre - Atlanta Sep 26, 2024 7:30 PM](https://atlanta-ga.events/tickets/khruangbin/)[Disney On Ice Upcoming Events: 24 Tulsa Expo Square - Pavilion Sep 26, 2024 7:30 PM](https://atlanta-ga.events/tickets/disney-on-ice/)[Jesus Adrian Romero Upcoming Events: 2 Cobb Energy Performing Arts Centre Sep 26, 2024 8:00 PM](https://atlanta-ga.events/tickets/jesus-adrian-romero/)[Atlanta Symphony Orchestra Upcoming Events: 6 Atlanta Symphony Hall Sep 26, 2024 8:00 PM](https://atlanta-ga.events/tickets/atlanta-symphony-orchestra/)[Lawrence - Band Upcoming Events: 2 The Tabernacle - GA Sep 26, 2024 8:00 PM](https://atlanta-ga.events/tickets/lawrence-band/)[Wasia Project Upcoming Events: 2 Terminal West At King Plow Arts Center Sep 26, 2024 8:00 PM](https://atlanta-ga.events/tickets/wasia-project/)[The California Honeydrops Upcoming Events: 2 Variety Playhouse Sep 26, 2024 8:00 PM](https://atlanta-ga.events/tickets/the-california-honeydrops/)[The Buena Vista Social Orchestra Upcoming Events: 2 Center Stage Theatre Sep 26, 2024 8:00 PM](https://atlanta-ga.events/tickets/the-buena-vista-social-orchestra/)[Nathalie Stutzmann Upcoming Events: 4 Atlanta Symphony Hall Sep 26, 2024 8:00 PM](https://atlanta-ga.events/tickets/nathalie-stutzmann/)[Red Lace Burlesque Show and Variety Show Upcoming Events: 4 Red Lace Burlesque - Myrtle Beach Sep 27, 2024 6:00 PM](https://atlanta-ga.events/tickets/red-lace-burlesque-show-and-variety-show/)[Georgia Tech Yellow Jackets Women\u0027s Volleyball Upcoming Events: 6 O\u0027Keefe Gymnasium Sep 27, 2024 7:00 PM](https://atlanta-ga.events/tickets/georgia-tech-yellow-jackets-womens-volleyball/)[Georgia Tech Yellow Jackets Upcoming Events: 6 O\u0027Keefe Gymnasium Sep 27, 2024 7:00 PM](https://atlanta-ga.events/tickets/georgia-tech-yellow-jackets/)[The Warning Upcoming Events: 2 Hell at The Masquerade - GA Sep 27, 2024 7:00 PM](https://atlanta-ga.events/tickets/the-warning/)[Southern Methodist (SMU) Mustangs Upcoming Events: 3 O\u0027Keefe Gymnasium Sep 27, 2024 7:00 PM](https://atlanta-ga.events/tickets/southern-methodist-smu-mustangs/)[Coral Moons Upcoming Events: 2 Altar at Masquerade - GA Sep 27, 2024 7:00 PM](https://atlanta-ga.events/tickets/coral-moons/)[The Red Pears Upcoming Events: 2 Purgatory Stage at Masquerade - GA Sep 27, 2024 7:00 PM](https://atlanta-ga.events/tickets/the-red-pears/)[Southern Methodist (SMU) Mustangs Women\u0027s Volleyball Upcoming Events: 3 O\u0027Keefe Gymnasium Sep 27, 2024 7:00 PM](https://atlanta-ga.events/tickets/southern-methodist-smu-mustangs-womens-volleyball/)[Kansas City Royals Upcoming Events: 9 Truist Park Sep 27, 2024 7:20 PM](https://atlanta-ga.events/tickets/kansas-city-royals/)[Maxwell Upcoming Events: 2 State Farm Arena - GA Sep 27, 2024 7:30 PM](https://atlanta-ga.events/tickets/maxwell/)[La Santa Grifa Upcoming Events: 2 Buckhead Theatre Sep 27, 2024 7:30 PM](https://atlanta-ga.events/tickets/la-santa-grifa/)[T.K. Kirkland Upcoming Events: 8 Uptown Comedy Corner Sep 27, 2024 7:30 PM](https://atlanta-ga.events/tickets/t-k-kirkland/)[Dharius Upcoming Events: 2 Buckhead Theatre Sep 27, 2024 7:30 PM](https://atlanta-ga.events/tickets/dharius/)[Tropical Fuck Storm Upcoming Events: 2 Terminal West At King Plow Arts Center Sep 27, 2024 8:00 PM](https://atlanta-ga.events/tickets/tropical-fuck-storm/)[Sisters Of Mercy Upcoming Events: 2 Coca-Cola Roxy Sep 27, 2024 8:00 PM](https://atlanta-ga.events/tickets/sisters-of-mercy/)[Lalah Hathaway Upcoming Events: 4 Center Stage Theatre Sep 27, 2024 8:00 PM](https://atlanta-ga.events/tickets/lalah-hathaway/)Show MoreAtlanta Events Calendar[](https://atlanta-ga.events)[Contact Us](#)Stay up to date! Join our newsletter now to stay up to date on all events in Atlanta plus the occasional special offer and discount code! Epic music events and incredible deals straight to your inbox. Either way, you\u2019ll be the first to know about the best shows in your area.\r\n\r\n[](https://www.facebook.com/)[](https://www.youtube.com/)[](https://www.instagram.com/)[](https://twitter.com/)[](https://www.pinterest.com/)Subscription DealsNameEmail7 \u002B 15 =SubscribeAtlanta-GA.Events|[Sitemap](https://atlanta-ga.events/sitemaps/)|[Privacy Policy](#)|All Rights Reserved 2024"
```

I called the tool named GetContentFromUrl with these arguments:
```
{"url": "https://eventcartel.com/events/all-categories/atlanta/from_2024-09-28_to_2024-09-29"}
```
and got this result:
```
"Discover Events Happening this from 2024-09-28 to 2024-09-29 in Atlanta GA \u2013 Every Category | EventCartel[EventCartel](/)[](/)* Browse Events\r\n* Create Event\r\n* My Tickets\r\n* Help\r\n\r\n# Events and Things to Do in Atlanta GA this from 2024-09-28 to 2024-09-29\r\n\r\n## Browse events and buy tickets to concerts, theaters, family events, and more in Atlanta.\r\n\r\nAtlanta\r\n\r\nTodayTomorrowThis WeekMonthAll datesChoose Dates\r\n\r\nConcertTheaterFamily EventPartyFestivalQuizzes/Bar GamesSportConferenceNetworkingFashionMovies[\u041A\u041E\u041C\u0415\u0414\u0418\u0419\u041D\u041E-\u0418\u041C\u041F\u0420\u041E\u0412\u0418\u0417\u0410\u0426\u0418\u041E\u041D\u041D\u041E\u0415 \u0428\u041E\u0423 \u0026lt; STORIES \u0026gt; in RoswellSunday, Sep 29 at 6:00 PMBrimstone Restaurant \u0026amp; Tavern](/events/concert/komedijno-improvizatsionnoe-shou-stories-roswell-GA-brimstone-restaurant-tavern-tickets-9357?promo=31aebbaab9f172c11dba0e177f2269f5)## Find Events and Buy Tickets to Local Activities Happening this from 2024-09-28 to 2024-09-29 in Atlanta\r\n\r\nDive into the vibrant local scene of Atlanta with EventCartel! Explore a wide range of events and things to do from 2024-09-28 to 2024-09-29 tailored to offer you the best in entertainment and fun things to do in Atlanta. Whether you\u2019re interested in the latest concerts, theater shows, or other local events happening from 2024-09-28 to 2024-09-29, everything you need is just a few clicks away. Discover all the exciting happenings in Atlanta from 2024-09-28 to 2024-09-29. Find your next great experience today and enjoy the convenience of purchasing tickets right from our platform.## Plan with Our Comprehensive Event and Concert Calendar\r\n\r\nStay on top of upcoming concerts, live music, and more happening this from 2024-09-28 to 2024-09-29 with EventCartel\u0026#x27;s detailed Atlanta event calendar. When searching for concerts or events from 2024-09-28 to 2024-09-29 in Atlanta, our platform makes it simple to find exactly what you\u0026#x27;re looking for. Use our time period filter to quickly pinpoint events that match your schedule, ensuring you never miss out on the best shows and music in Atlanta. Our calendar is your gateway to the hottest event tickets in town.## Explore Event Types: Concerts, Theaters, Parties, Family Events, and More this from 2024-09-28 to 2024-09-29\r\n\r\nDiscover what\u2019s happening near Atlanta this from 2024-09-28 to 2024-09-29 with EventCartel\u2019s diverse range of event offerings. From electrifying nightlife to family fun from 2024-09-28 to 2024-09-29, our platform connects you to a plethora of activities including Atlanta concerts, theaters, family events, parties, festivals, and more. Use the event type filter to quickly navigate to the types of events happening from 2024-09-28 to 2024-09-29 that excite you the most. It\u0026#x27;s your one-stop-shop for all Atlanta events from 2024-09-28 to 2024-09-29, ensuring you always have something to look forward to.Discover what\u2019s happening near Atlanta from 2024-09-28 to 2024-09-29 with EventCartel\u2019s diverse range of event offerings. From electrifying nightlife to family fun from 2024-09-28 to 2024-09-29, our platform connects you to a plethora of activities including Atlanta concerts, theaters, family events, parties, festivals, and more. Use the event type filter to quickly navigate to the types of events happening from 2024-09-28 to 2024-09-29 that excite you the most. It\u0026#x27;s your one-stop-shop for all Atlanta events from 2024-09-28 to 2024-09-29, ensuring you always have something to look forward to.## Find Russian and Ukrainian Cultural Events Concerts this from 2024-09-28 to 2024-09-29 in Atlanta\r\n\r\nExplore the rich cultural tapestry that Russian and Ukrainian events bring to Atlanta this from 2024-09-28 to 2024-09-29. From traditional concerts to family theater shows happening from 2024-09-28 to 2024-09-29, these events offer a unique insight into Slavic cultures. Use the cultural event filter on our platform to effortlessly find Russian and Ukrainian concerts, family events, and more in Atlanta. It\u0026#x27;s a wonderful opportunity to engage with and celebrate the diversity that these communities offer.## Ticket Buying Made Easy: Secure, Affordable, and Accessible\r\n\r\nEventCartel ensures a seamless ticket buying experience, allowing you to buy Atlanta event tickets this from 2024-09-28 to 2024-09-29 with confidence from the comfort of your mobile browser, app, or desktop. We are committed to offering the best prices and a 100% buyer satisfaction guarantee. Whether you are planning ahead or looking for last-minute event tickets from 2024-09-28 to 2024-09-29 in Atlanta, our platform provides a reliable, secure, and user-friendly service that lets you focus on enjoying your event, stress-free.[](/)EventCartel is an event guide and innovative ticketing platform\r\n\r\nSupport ChatFor Organizers\r\n\r\n[Event Ticketing Software](/system-features)[Promote Your Event](/promote)[Event Manager App](https://apps.apple.com/ua/app/eventcartel/id6474256116)Work with us\r\n\r\n[Affiliates](/affiliates)[Artists](/artists)[Venues](/venues)[Promoters](/promoters)Support\r\n\r\n[FAQ](/faq)[Purchase Policy](/purchasepolicy)[Privacy Policy](/privacy)Company\r\n\r\n[About Us](/about)[Sitemap](/)For Organizers\r\n\r\n[Event Ticketing Software](/system-features)[Promote Your Event](/promote)[Event Manager App](https://apps.apple.com/ua/app/eventcartel/id6474256116)Company\r\n\r\n[About Us](/about)[Sitemap](/)Work with us\r\n\r\n[Affiliates](/affiliates)[Artists](/artists)[Venues](/venues)[Promoters](/promoters)Support\r\n\r\n[FAQ](/faq)[Purchase Policy](/purchasepolicy)[Privacy Policy](/privacy)englishrussianFollow us:\r\n\r\n[](https://www.instagram.com/eventcartelinc/)[](https://www.facebook.com/EventCartelNY)[](https://www.youtube.com/@eventcartel)[](https://www.tiktok.com/)2017 - 2024 \u00AE All rights reserved\r\n\r\n"
```

I called the tool named GetContentFromUrl with these arguments:
```
{"url": "https://atlanta-ga.events/october/"}
```
and got this result:
```
"Atlanta Events October 2024: Concerts, Shows, Sports...JavaScript support required!\r\n\r\n[](https://atlanta-ga.events)\u00D7Categories[All Events](https://atlanta-ga.events/)[Concerts](https://atlanta-ga.events/concerts/)[Shows](https://atlanta-ga.events/shows/)[Comedy](https://atlanta-ga.events/comedy-shows/)[Sports](https://atlanta-ga.events/sports/)[Family Shows](https://atlanta-ga.events/family-shows/)[Festivals](https://atlanta-ga.events/festivals/)[Opera](https://atlanta-ga.events/opera-tickets/)[Venues](https://atlanta-ga.events/entertainment-venues/)* All Events\r\n* Concerts\r\n* Shows\r\n* Comedy\r\n* Sports\r\n* More Family Shows Festivals Opera Venues\r\n\r\n* Home\r\n* October\r\n\r\n# Atlanta Events October 2024\r\n\r\nExplore our comprehensive calendar for all the current and upcoming events in Atlanta, GA, for October 2024. This guide is your gateway to discovering the best things to do in Atlanta, from Concerts and Theater Shows to Sports events and music festivals. You\u2019ll find a detailed list of all the most engaging and exciting activities happening throughout the month. Easily search and select your preferred events in Atlanta with our user-friendly calendar. Secure your tickets with 100% assurance at the lowest prices. Experience the diverse and vibrant culture of Atlanta this October!\r\n\r\nStart DateOct 1, 2024ResetCancelApplyEnd DateOct 3, 2024CategoryAll EventsConcertsTheaterSportsSearchContentsUpcoming EventsList Of All EventsAtlanta, GA## Upcoming Events \u0026 Tickets\r\n\r\nSelect Your DatesResetCancelApplyUpcoming Events: 542* Oct 01 2024 3:30 AM Tue NL Wild Card: Atlanta Braves vs. TBD - Game 1 (If Necessary) Truist Park | Capacity: 41149 30339, 1100 Circle 75 Pkwy, Atlanta, GA, US Prices from $62 Avg. price ~ $219 Available Tickets: 3160 Tickets\r\n* Oct 01 2024 7:00 PM Tue The Used VIP Packages The Eastern - GA 30312, 800 Old Flat Shoals Road SE, Atlanta, GA, US Prices from $69 Avg. price ~ $155 Available Tickets: 61 Tickets\r\n* Oct 01 2024 7:00 PM Tue The Love Scene - An Evening Of R\u0026B Love Songs With India Shawn Mable House Barnes Amphitheatre 30126, 5239 Floyd Road, Mableton, GA, US Prices from $52 Avg. price ~ $125 Available Tickets: 703 Tickets\r\n* Oct 01 2024 7:00 PM Tue Brooks Nielsen Hell at The Masquerade - GA | Capacity: 300 30303, 75 MLK Jr Dr SW, Atlanta, GA, US Prices from $39 Avg. price ~ $68 29 tickets remaining! Tickets\r\n* Oct 01 2024 8:00 PM Tue M. Ward Terminal West At King Plow Arts Center | Capacity: 625 30318, 887 West Marietta Studio C, Atlanta, GA, US Prices from $43 Avg. price ~ $53 18 tickets remaining! Tickets\r\n* Oct 01 2024 8:00 PM Tue Kandace Springs City Winery - Atlanta | Capacity: 300 30308, 675 Ponce De Leon Ave, Atlanta, GA, US Prices from $64 Avg. price ~ $111 Available Tickets: 52 Tickets\r\n* Oct 01 2024 8:00 PM Tue Todd Rundgren Buckhead Theatre | Capacity: 2500 30305, 3110 Roswell Road, Atlanta, GA, US Prices from $62 Avg. price ~ $106 Available Tickets: 64 Tickets\r\n* Oct 01 2024 8:00 PM Tue Julius Rodriguez Vinyl - GA 30309, 1374 West Peachtree Street, Atlanta, GA, US Prices from $21 Avg. price ~ $25 16 tickets remaining! Tickets\r\n* Oct 02 2024 3:30 AM Wed NL Wild Card: Atlanta Braves vs. TBD - Game 2 (If Necessary) Truist Park | Capacity: 41149 30339, 1100 Circle 75 Pkwy, Atlanta, GA, US Prices from $60 Avg. price ~ $240 Available Tickets: 3214 Tickets\r\n* Oct 02 2024 7:00 PM Wed Blu DeTiger Altar at Masquerade - GA 30303, 75 MLK Jr Dr SW, Atlanta, GA, US Prices from $104 Avg. price ~ $120 2 tickets remaining! Tickets\r\n* Oct 02 2024 7:00 PM Wed Dayseeker The Eastern - GA 30312, 800 Old Flat Shoals Road SE, Atlanta, GA, US Prices from $49 Avg. price ~ $56 24 tickets remaining! Tickets\r\n* Oct 02 2024 7:00 PM Wed The Beaches Heaven Stage at Masquerade - GA | Capacity: 1000 30303, 75 MLK Jr Dr SW, Atlanta, GA, US Prices from $44 Avg. price ~ $62 16 tickets remaining! Tickets\r\n* Oct 02 2024 7:30 PM Wed Atlanta United FC vs. CF Montreal VIP Packages Mercedes-Benz Stadium | Capacity: 83000 30313, Martin Luther King Jr Dr SW \u0026 Northside Dr NW, Atlanta, GA, US Prices from $10 Avg. price ~ $75 Available Tickets: 1442 Tickets\r\n* Oct 02 2024 7:30 PM Wed The Lion King Fabulous Fox Theatre - Atlanta | Capacity: 4678 30305, 660 Peachtree St NE, Atlanta, GA, US Prices from $74 Avg. price ~ $158 Available Tickets: 441 Tickets\r\n* Oct 02 2024 7:30 PM Wed Jinjer VIP Packages Buckhead Theatre | Capacity: 2500 30305, 3110 Roswell Road, Atlanta, GA, US Prices from $37 Avg. price ~ $257 Available Tickets: 83 Tickets\r\n* Oct 02 2024 8:00 PM Wed Cactus Tate Helium Comedy Club - Atlanta 30005, 3070 Windward Plaza, Alpharetta, GA, US Prices from $63 Avg. price ~ $73 22 tickets remaining! Tickets\r\n* Oct 02 2024 8:00 PM Wed Zheani Terminal West At King Plow Arts Center | Capacity: 625 30318, 887 West Marietta Studio C, Atlanta, GA, US Prices from $16 Avg. price ~ $24 18 tickets remaining! Tickets\r\n* Oct 02 2024 8:00 PM Wed The Hives The Tabernacle - GA | Capacity: 2600 30303, 152 Luckie St, Atlanta, GA, US Prices from $14 Avg. price ~ $91 Available Tickets: 560 Tickets\r\n* Oct 02 2024 8:00 PM Wed The Brothers Comatose Aisle 5 | Capacity: 300 30307, 1123 Euclid Ave Ne, Atlanta, GA, US Prices from $41 Avg. price ~ $70 15 tickets remaining! Tickets\r\n* Oct 02 2024 8:00 PM Wed Lola Young Center Stage Theatre | Capacity: 1050 30309, 1374 West Peachtree Street Northwest, Atlanta, GA, US Prices from $33 Avg. price ~ $72 41 tickets remaining! Tickets\r\n* Oct 02 2024 8:00 PM Wed Fred Wesley and The New JBs City Winery - Atlanta | Capacity: 300 30308, 675 Ponce De Leon Ave, Atlanta, GA, US Prices from $53 Avg. price ~ $78 Available Tickets: 294 Tickets\r\n* Oct 02 2024 8:00 PM Wed Micro Wrestling Fowling Warehouse Atlanta 30318, 1356 English St NW, Atlanta, GA, US Prices from $50 Avg. price ~ $59 18 tickets remaining! Tickets\r\n* Oct 03 2024 3:30 AM Thu NL Wild Card: Atlanta Braves vs. TBD - Game 3 (If Necessary) Truist Park | Capacity: 41149 30339, 1100 Circle 75 Pkwy, Atlanta, GA, US Prices from $62 Avg. price ~ $260 Available Tickets: 3248 Tickets\r\n* Oct 03 2024 7:00 PM Thu Sepultura Buckhead Theatre | Capacity: 2500 30305, 3110 Roswell Road, Atlanta, GA, US Prices from $47 Avg. price ~ $83 Available Tickets: 74 Tickets\r\n* Oct 03 2024 7:00 PM Thu La Luz Purgatory Stage at Masquerade - GA | Capacity: 300 30303, 75 MLK Jr Dr SW, Atlanta, GA, US Prices from $32 Avg. price ~ $40 Available Tickets: 57 Tickets\r\n* Oct 03 2024 7:00 PM Thu The Slaps Altar at Masquerade - GA 30303, 75 MLK Jr Dr SW, Atlanta, GA, US Prices from $27 Avg. price ~ $57 22 tickets remaining! Tickets\r\n* Oct 03 2024 7:30 PM Thu The Marley Brothers Lakewood Amphitheatre | Capacity: 19000 30315, 2002 Lakewood Avenue Southeast, Atlanta, GA, US Prices from $21 Avg. price ~ $184 Available Tickets: 1284 Tickets\r\n* Oct 03 2024 7:30 PM Thu Charli XCX \u0026 Troye Sivan VIP Packages State Farm Arena - GA | Capacity: 21000 30303, One Philips Drive, Atlanta, GA, US Prices from $97 Avg. price ~ $277 Available Tickets: 753 Tickets\r\n* Oct 03 2024 7:30 PM Thu The Lion King Fabulous Fox Theatre - Atlanta | Capacity: 4678 30305, 660 Peachtree St NE, Atlanta, GA, US Prices from $61 Avg. price ~ $159 Available Tickets: 577 Tickets\r\n\r\nShow More### All Events This Month\r\n\r\n[NL Wild Card Upcoming Events: 9 Citi Field Oct 1, 2024 3:30 AM](https://atlanta-ga.events/tickets/nl-wild-card/)[MLB Postseason Upcoming Events: 33 Target Field Oct 1, 2024 3:30 AM](https://atlanta-ga.events/tickets/mlb-postseason/)[Atlanta Braves Upcoming Events: 33 Truist Park Oct 1, 2024 3:30 AM](https://atlanta-ga.events/tickets/atlanta-braves/)[India Shawn Upcoming Events: 2 Mable House Barnes Amphitheatre Oct 1, 2024 7:00 PM](https://atlanta-ga.events/tickets/india-shawn/)[Brooks Nielsen Upcoming Events: 2 Hell at The Masquerade - GA Oct 1, 2024 7:00 PM](https://atlanta-ga.events/tickets/brooks-nielsen/)[The Used Upcoming Events: 2 The Eastern - GA Oct 1, 2024 7:00 PM](https://atlanta-ga.events/tickets/the-used/)[Todd Rundgren Upcoming Events: 2 Buckhead Theatre Oct 1, 2024 8:00 PM](https://atlanta-ga.events/tickets/todd-rundgren/)[M. Ward Upcoming Events: 2 Terminal West At King Plow Arts Center Oct 1, 2024 8:00 PM](https://atlanta-ga.events/tickets/m-ward/)[Kandace Springs Upcoming Events: 2 City Winery - Atlanta Oct 1, 2024 8:00 PM](https://atlanta-ga.events/tickets/kandace-springs/)[Julius Rodriguez Upcoming Events: 2 Vinyl - GA Oct 1, 2024 8:00 PM](https://atlanta-ga.events/tickets/julius-rodriguez/)[The Beaches Upcoming Events: 2 Heaven Stage at Masquerade - GA Oct 2, 2024 7:00 PM](https://atlanta-ga.events/tickets/the-beaches/)[Dayseeker - Band Upcoming Events: 2 The Eastern - GA Oct 2, 2024 7:00 PM](https://atlanta-ga.events/tickets/dayseeker-band/)[Blu DeTiger Upcoming Events: 2 Altar at Masquerade - GA Oct 2, 2024 7:00 PM](https://atlanta-ga.events/tickets/blu-detiger/)[Atlanta United FC Upcoming Events: 6 Mercedes-Benz Stadium Oct 2, 2024 7:30 PM](https://atlanta-ga.events/tickets/atlanta-united-fc/)[CF Montreal Upcoming Events: 3 Mercedes-Benz Stadium Oct 2, 2024 7:30 PM](https://atlanta-ga.events/tickets/cf-montreal/)[Jinjer Upcoming Events: 2 Buckhead Theatre Oct 2, 2024 7:30 PM](https://atlanta-ga.events/tickets/jinjer/)[The Lion King Upcoming Events: 46 Fabulous Fox Theatre - Atlanta Oct 2, 2024 7:30 PM](https://atlanta-ga.events/tickets/the-lion-king/)[Fred Wesley and The New JBs Upcoming Events: 2 City Winery - Atlanta Oct 2, 2024 8:00 PM](https://atlanta-ga.events/tickets/fred-wesley-and-the-new-jbs/)[Lola Young Upcoming Events: 2 Center Stage Theatre Oct 2, 2024 8:00 PM](https://atlanta-ga.events/tickets/lola-young/)[Zheani Upcoming Events: 2 Terminal West At King Plow Arts Center Oct 2, 2024 8:00 PM](https://atlanta-ga.events/tickets/zheani/)[Cactus Tate Upcoming Events: 2 Helium Comedy Club - Atlanta Oct 2, 2024 8:00 PM](https://atlanta-ga.events/tickets/cactus-tate/)[The Hives Upcoming Events: 2 The Tabernacle - GA Oct 2, 2024 8:00 PM](https://atlanta-ga.events/tickets/the-hives/)[Micro Wrestling Upcoming Events: 3 Fowling Warehouse Atlanta Oct 2, 2024 8:00 PM](https://atlanta-ga.events/tickets/micro-wrestling/)[The Brothers Comatose Upcoming Events: 2 Aisle 5 Oct 2, 2024 8:00 PM](https://atlanta-ga.events/tickets/the-brothers-comatose/)[La Luz Upcoming Events: 2 Purgatory Stage at Masquerade - GA Oct 3, 2024 7:00 PM](https://atlanta-ga.events/tickets/la-luz/)[The Slaps Upcoming Events: 2 Altar at Masquerade - GA Oct 3, 2024 7:00 PM](https://atlanta-ga.events/tickets/the-slaps/)[Sepultura Upcoming Events: 2 Buckhead Theatre Oct 3, 2024 7:00 PM](https://atlanta-ga.events/tickets/sepultura/)[Charli XCX Upcoming Events: 2 State Farm Arena - GA Oct 3, 2024 7:30 PM](https://atlanta-ga.events/tickets/charli-xcx/)[Ziggy Marley Upcoming Events: 2 Lakewood Amphitheatre Oct 3, 2024 7:30 PM](https://atlanta-ga.events/tickets/ziggy-marley/)[Stephen Marley Upcoming Events: 2 Lakewood Amphitheatre Oct 3, 2024 7:30 PM](https://atlanta-ga.events/tickets/stephen-marley/)[Troye Sivan Upcoming Events: 2 State Farm Arena - GA Oct 3, 2024 7:30 PM](https://atlanta-ga.events/tickets/troye-sivan/)[The Marley Brothers Upcoming Events: 2 Lakewood Amphitheatre Oct 3, 2024 7:30 PM](https://atlanta-ga.events/tickets/the-marley-brothers/)[Elle King Upcoming Events: 2 The Tabernacle - GA Oct 3, 2024 8:00 PM](https://atlanta-ga.events/tickets/elle-king/)[Angry Blackmen Upcoming Events: 2 529 Oct 3, 2024 8:00 PM](https://atlanta-ga.events/tickets/angry-blackmen/)[Horse Jumper of Love Upcoming Events: 2 Aisle 5 Oct 3, 2024 8:00 PM](https://atlanta-ga.events/tickets/horse-jumper-of-love/)[Will Burkart Upcoming Events: 2 City Winery - Atlanta Oct 3, 2024 8:00 PM](https://atlanta-ga.events/tickets/will-burkart/)[MJ Lenderman Upcoming Events: 4 Terminal West At King Plow Arts Center Oct 3, 2024 8:00 PM](https://atlanta-ga.events/tickets/mj-lenderman/)[Johnny Marr Upcoming Events: 2 The Eastern - GA Oct 3, 2024 8:00 PM](https://atlanta-ga.events/tickets/johnny-marr/)[vaultboy Upcoming Events: 2 Vinyl - GA Oct 3, 2024 8:00 PM](https://atlanta-ga.events/tickets/vaultboy/)[Tampa Bay Buccaneers Upcoming Events: 3 Mercedes-Benz Stadium Oct 3, 2024 8:15 PM](https://atlanta-ga.events/tickets/tampa-bay-buccaneers/)Show MoreAtlanta Events Calendar[](https://atlanta-ga.events)[Contact Us](#)Stay up to date! Join our newsletter now to stay up to date on all events in Atlanta plus the occasional special offer and discount code! Epic music events and incredible deals straight to your inbox. Either way, you\u2019ll be the first to know about the best shows in your area.\r\n\r\n[](https://www.facebook.com/)[](https://www.youtube.com/)[](https://www.instagram.com/)[](https://twitter.com/)[](https://www.pinterest.com/)Subscription DealsNameEmail7 \u002B 15 =SubscribeAtlanta-GA.Events|[Sitemap](https://atlanta-ga.events/sitemaps/)|[Privacy Policy](#)|All Rights Reserved 2024"
```

I called the tool named GetContentFromUrl with these arguments:
```
{"url": "https://gosouthatlanta.com/things-to-do/events/atlanta-in-october.html"}
```
and got this result:
```
"Atlanta Events - October 2024This site uses cookies. Opt out at any time. More details\r\n\r\nHome \u0026rsaquo; Things To Do \u0026rsaquo; October Events\r\n\r\n# Atlanta Events - October 2024\r\n\r\nInterested in a specific type of event? See Atlanta\u0026rsquo;s calendars of upcoming: \u0026ndash; Comedy shows \u0026ndash; Beer and wine festivals \u0026ndash; Music festivals \u0026ndash; Musicals \u0026ndash; Food festivals\r\n\r\nSee also: \u0026ndash; Concerts in Atlanta in October \u0026ndash; Atlanta\u0026rsquo;s biggest annual festivals \u0026ndash; Atlanta weather in October \u0026ndash; Things to do in Atlanta\r\n\r\n!Event dates, venue and other details can change at short notice. Please check all information at the official website before making plans to attend an event.\r\n\r\n## Events From October 1 To The Weekend Of October 5-6 2024\r\n\r\n\u0026rarr; See September events\r\n\r\n\u0026rarr; For concerts this week, see Atlanta concerts in October 2024\r\n\r\n\u0026ndash; Daddy Long Legs, September 13 - October 6 2024Musical at The Legacy Theatre, Tyrone. More details\r\n\r\n\u0026ndash; Boh\u0026egrave;me Project: La boh\u0026egrave;me, September 18 - October 5 2024Opera at Pullman Yards. More details\r\n\r\n\u0026ndash; The Chinese Lady, September 18 - October 13 2024Play at Alliance Theatre. More details\r\n\r\n\u0026ndash; Boh\u0026egrave;me Project: Rent, September 19 - October 6 2024Opera at Pullman Yards. More details\r\n\r\n\u0026ndash; The Curious Cardinal, September 21 - October 27 2024Children\u0026rsquo;s performance at Alliance Theatre. More details\r\n\r\n\u0026ndash; Out on Film, September 26 - October 6 2024Annual festival of LGBTQ\u002B filmmaking. More details\r\n\r\n\u0026ndash; The Moth Atlanta GrandSLAM, October 1 2024Storytelling competition at Center Stage. More details\r\n\r\n\u0026ndash; Disney\u0026rsquo;s The Lion King, October 2-20 2024Musical at the Fox Theatre. More details\r\n\r\n\u0026ndash; Kumail Nanjiani: Doing This Again, October 4 2024Comedy show at The Tabernacle. More details\r\n\r\n\u0026ndash; Stand Up Comedy at the Clay! Hosted by Yoshee, October 4 2024Comedy show at Red Clay Music Foundry, Duluth. More details\r\n\r\n\u0026ndash; ExpoContratista, October 4-5 2024Hispanic construction trade show at the Cobb Galleria Centre. More details\r\n\r\n\u0026ndash; How to Train Your Dragon in Concert with the ASO, October 4-5 2024Film screening with live musical accompaniment at Atlanta Symphony Hall. More details\r\n\r\n\u0026ndash; Atlanta Horror Film Festival, October 4-6 2024Annual festival of horror film at Limeliight Theater. More details\r\n\r\n\u0026ndash; DreamHack Atlanta, October 4-6 2024Annual esports convention at the Georgia World Congress Center with tournament prizes across 20 gaming contests. More details\r\n\r\n\u0026ndash; Atlanta Sci-Fi Film Festival, October 4-6 2024Annual science fiction film festival in Peachtree Corners. More details\r\n\r\n\u0026ndash; Lakewood Antiques Market, October 4-6 2024Big monthly antiques and collectibles market in Cumming. More details\r\n\r\n\u0026ndash; Am\u0026eacute;lie: The Musical, October 4 - November 3 2024Musical at Horizon Theatre. More details\r\n\r\n\u0026ndash; Wings \u0026amp; Whitewalls, October 5 2024Fall car show at the Aviation History \u0026amp; Technology Center, Marietta, with vintage aircraft, cars and more. More details\r\n\r\n\u0026ndash; Taste of Tucker, October 5 2024Annual food festival along Main Street, Tucker, featuring food and drink from local restaurants and breweries, arts and crafts, music and more. More details\r\n\r\n\u0026ndash; EGGtoberfest, October 5 2024Barbecue festival at Coolray Field, Lawrenceville, with cook-off and cooking demonstrations, children\u0026rsquo;s activities and more. More details\r\n\r\n\u0026ndash; WWE Bad Blood, October 5 2024Wrestling event at State Farm Arena. More details\r\n\r\n\u0026ndash; Festival on Ponce, October 5-6 2024Twice-annual arts festival in Olmsted Linear Park, with 125\u002B vendors of fine art, folk art and outsider art, plus food and drink, and children\u0026rsquo;s activities. More details\r\n\r\n\u0026ndash; Candler Park Fall Fest, October 5-6 2024Popular annual festival in Candler Park and neighborhood, with live music, art, food, rides, games and more. More details\r\n\r\n\u0026ndash; Atlanta Arab Festival, October 5-6 2024Annual festival of Arabic food and culture at the Alif Institute, with traditional dishes from Arabic countries, live performances, vendors, exhibits, workshops, games and more. More details\r\n\r\n\u0026ndash; Aladdin, October 5-6 2024Ballet performance at the Gas South Theater. More details\r\n\r\n\u0026ndash; ASC Presents Havoc Movement\u0026rsquo;s Dracula: The Failings of Men, October 5 - November 3 2024Performance at the Shakespeare Tavern Playhouse. More details\r\n\r\n\u0026ndash; OYE Fest, October 6 2024Annual festival in Historic Fourth Ward Park celebrating Latinx art and music. More details\r\n\r\n\u0026ndash; La Cotorrisa Tour, October 6 2024Spanish-language comedy podcast show at the Buckhead Theatre. More details\r\n\r\n## Events From October 7 To Columbus Day Weekend, October 12-14 2024\r\n\r\n\u0026rarr; For concerts this week, see Atlanta concerts in October 2024\r\n\r\n\u0026ndash; The Chinese Lady, September 18 - October 13 2024Play at Alliance Theatre. More details\r\n\r\n\u0026ndash; The Curious Cardinal, September 21 - October 27 2024Children\u0026rsquo;s performance at Alliance Theatre. More details\r\n\r\n\u0026ndash; Disney\u0026rsquo;s The Lion King, October 2-20 2024Musical at the Fox Theatre. More details\r\n\r\n\u0026ndash; Am\u0026eacute;lie: The Musical, October 4 - November 3 2024Musical at Horizon Theatre. More details\r\n\r\n\u0026ndash; ASC Presents Havoc Movement\u0026rsquo;s Dracula: The Failings of Men, October 5 - November 3 2024Performance at the Shakespeare Tavern Playhouse. More details\r\n\r\n\u0026ndash; Atlanta miniFEST, October 9 2024Community music and arts festival at Believe Music Hall. More details\r\n\r\n\u0026ndash; Taste Brookhaven, October 10 2024Food-tasting event in the gardens of the Hyatt Regency Atlanta Perimeter, featuring Brookhaven chefs and restaurants plus drinks and live entertainment, to benefit Children\u0026rsquo;s Healthcare of Atlanta. More details\r\n\r\n\u0026ndash; Jake Shane, October 10 2024Comedy show at Center Stage. More details\r\n\r\n\u0026ndash; Scott Antique Market, October 10-13 2024Monthly antiques market at the Atlanta Expo Center. More details\r\n\r\n\u0026ndash; Atsuko Okatsuka, October 11 2024Comedy show at Center Stage. More details\r\n\r\n\u0026ndash; Pride Kickoff, October 11 2024After-hours LGBTQ\u002B party at the Georgia Aquarium. More details\r\n\r\n\u0026ndash; Wire \u0026amp; Wood Alpharetta, October 11-12 2024Songwriter music festival with performances across downtown Alpharetta. More details\r\n\r\n\u0026ndash; Joyce Meyer Live, October 11-12 2024Faith event at the Gas South Arena. More details\r\n\r\n\u0026ndash; Take Your Royal Position Conference, October 11-12 2024Faith and empowerment event in downtown Atlanta. More details\r\n\r\n\u0026ndash; Duluth Fall Home Expo, October 11-13 2024Home renovations trade show at the Gas South Convention Center. More details\r\n\r\n\u0026ndash; Oakhurst Porchfest, October 12 2024Annual community music festival in Decatur over 100 live performances on Oakhurst neighborhood porches. More details\r\n\r\n\u0026ndash; Madison Avenue Derby, October 12 2024Annual races in Decatur in homemade, human-powered soapbox cars, to benefit local children\u0026rsquo;s charities. More details\r\n\r\n\u0026ndash; Politics War Room with James Carville \u0026amp; Al Hunt, October 12 2024Talk event at Variety Playhouse. More details\r\n\r\n\u0026ndash; Atlanta Pride Festival, October 12-13 2024One of the biggest Pride festivals in the country, held in Piedmont Park, with live performances, parade, marches, marketplace and more. More details\r\n\r\n\u0026ndash; Taste \u0026amp; Brews Fall Festival, October 12-13 2024Annual food and beer festival at Etowah River Park, Canton, featuring southern food and beers, musicians from across Georgia, arts and crafts, farmers market, children\u0026rsquo;s activities and more. More details\r\n\r\n\u0026ndash; Old Fourth Ward Fall Arts Festival, October 12-13 2024Twice-annual arts festival in Historic Fourth Ward Park featuring local and Georgian artists, demonstrations, live entertainment, art activities, food and more. More details\r\n\r\n\u0026ndash; Chalktoberfest, October 12-13 2024Popular annual art and beer festival in Marietta Square, featuring professional chalk artists, 120\u002B craft beers, live music, arts and crafts, food trucks, amateur chalk art competition, and more. More details\r\n\r\n\u0026ndash; FashionFest, October 12-13 2024Festival of urban-popular fashion and streetwear at Ambient Plus Studio, with fashion and footwear vendors, food, music and more. More details\r\n\r\n\u0026ndash; Gold Over America Tour, October 13 2024Gymnastics performance at the Gas South Arena. More details\r\n\r\n\u0026ndash; Roswell Town Square Dinner Series, October 13 2024Outdoor community dining event at Roswell Town Square with food from The Mill Kitchen \u0026amp; Bar. More details\r\n\r\n\u0026ndash; Pride Parade Viewing Party, October 13 2024Rooftop LGBTQ\u002B party at the Fox Theatre, to coincide with the annual Pride parade. More details\r\n\r\n\u0026ndash; Joe Wong, October 14 2024Comedy show at Red Clay Music Foundry, Duluth. More details\r\n\r\n## Events From October 15 To The Weekend Of October 19-20 2024\r\n\r\n\u0026rarr; For concerts this week, see Atlanta concerts in October 2024\r\n\r\n\u0026ndash; The Curious Cardinal, September 21 - October 27 2024Children\u0026rsquo;s performance at Alliance Theatre. More details\r\n\r\n\u0026ndash; Disney\u0026rsquo;s The Lion King, October 2-20 2024Musical at the Fox Theatre. More details\r\n\r\n\u0026ndash; Am\u0026eacute;lie: The Musical, October 4 - November 3 2024Musical at Horizon Theatre. More details\r\n\r\n\u0026ndash; ASC Presents Havoc Movement\u0026rsquo;s Dracula: The Failings of Men, October 5 - November 3 2024Performance at the Shakespeare Tavern Playhouse. More details\r\n\r\n\u0026ndash; David Cross \u0026ndash; End of the Beginning of the End, October 16 2024Comedy show at The Tabernacle. More details\r\n\r\n\u0026ndash; Jessica Kirson, October 18 2024Comedy show at Atlanta Symphony Hall. More details\r\n\r\n\u0026ndash; Sharon McMahon, October 18 2024Author talk at Center Stage. More details\r\n\r\n\u0026ndash; Stone Mountain Highland Games and Scottish Festival, October 18-20 2024Annual festival of Scottish heritage at Stone Mountain Park, with traditional Scottish athletics, music, dancing, food and drink, and vendors. All are welcome. More details\r\n\r\n\u0026ndash; Dracula: Presented by Gwinnett Ballet Theatre, October 18-20 2024Ballet performance at the Gas South Theater. More details\r\n\r\n\u0026ndash; Georgia Metropolitan Dance Theatre presents Dracula, October 18-20 2024Ballet performance at the KSU Dance Theatre, Marietta Campus. More details\r\n\r\n\u0026ndash; A Taste of Chamblee, October 19 2024Annual food festival in downtown Chamblee, with food from neighborhood restaurants, craft vendors, children\u0026rsquo;s activities, football screenings, live performances, wine tent, beer garden and more. More details\r\n\r\n\u0026ndash; HarvestFest and Scarecrows In The Square, October 19 2024Annual fall festival in Marietta Square, with arts and crafts show, touch-a-truck, scarecrow displays, costume and pie eating contests, and more. More details\r\n\r\n\u0026ndash; Pop-In For Family Fun: Boo! Halloween, October 19 2024Monthly free family day at the Marietta History Center. More details\r\n\r\n\u0026ndash; Cherokee Heights Arts Festival, October 19 2024Free annual arts festival in the Cherokee Heights neighborhood, with local artists and makers, local musicians, art activities, food, beer and more. More details\r\n\r\n\u0026ndash; NBAF Celebration Under The Stars, October 19 2024Fundraising event for the National Black Arts Festival at Wolf Creek Amphitheater, with live performances and food and drink. More details\r\n\r\n\u0026ndash; Rickey Smiley, October 19 2024Comedy show at the Cobb Energy Performing Arts Centre. More details\r\n\r\n\u0026ndash; Carlos Ballarta: Tlatoani, October 19 2024Comedy show at the Gas South Theater. More details\r\n\r\n\u0026ndash; Witch Perfect, October 19 2024Drag performance at Center Stage. More details\r\n\r\n\u0026ndash; L5P Halloween Festival and Parade, October 19-20 2024Popular annual Halloween celebration in Little Five Points, with live music, parade, haunted house, ghost tours and more. More details\r\n\r\n\u0026ndash; PNC Atlanta 10 Miler \u0026amp; 5K, October 19-20 2024Annual 10-mile, 5K and 1-mile youth runs through Buckhead, from Atlantic Station. More details\r\n\r\n\u0026ndash; Johns Creek Arts Festival, October 19-20 2024Annual arts and crafts festival in Johns Creek featuring 140 artisans from across the country. More details\r\n\r\n\u0026ndash; Abnormal Market and Uncommon Bazaar, October 19-20 2024Art market in association with the Old Fourth Ward Park Arts Festival. More details\r\n\r\n\u0026ndash; Boo at the Zoo, October 19-20, 26-27 2024Annual Halloween event at Zoo Atlanta, with trick or treating, Halloween-themed activities, and more. Costumes welcome. More details\r\n\r\n\u0026ndash; Landlocked 8, October 20 2024Oyster and BBQ festival at Legacy Park, Decatur. More details\r\n\r\n\u0026ndash; MrBallen: The Strange, Dark, \u0026amp; Mysterious Live Tour, October 20 2024Podcast live show at the Coca-Cola Roxy. More details\r\n\r\n\u0026ndash; National Ballet of Ukraine, October 20 2024Ballet performance at the Cobb Energy Performing Arts Centre. More details\r\n\r\n\u0026ndash; Shakespeare Out of a Hat: Macbeth, October 20 2024Part-improvised play at the Shakespeare Tavern Playhouse. More details\r\n\r\n## Events From October 21 To The Weekend Of October 26-27 2024\r\n\r\n\u0026rarr; For concerts this week, see Atlanta concerts in October 2024\r\n\r\n\u0026ndash; The Curious Cardinal, September 21 - October 27 2024Children\u0026rsquo;s performance at Alliance Theatre. More details\r\n\r\n\u0026ndash; Am\u0026eacute;lie: The Musical, October 4 - November 3 2024Musical at Horizon Theatre. More details\r\n\r\n\u0026ndash; ASC Presents Havoc Movement\u0026rsquo;s Dracula: The Failings of Men, October 5 - November 3 2024Performance at the Shakespeare Tavern Playhouse. More details\r\n\r\n\u0026ndash; Malcolm Gladwell in conversation with Virginia Prescott \u0026ndash; Revenge of the Tipping Point, October 22 2024Author talk at Atlanta Symphony Hall. More details\r\n\r\n\u0026ndash; Jim Henson\u0026rsquo;s Labyrinth: In Concert, October 23 2024Film screening with live musical accompaniment at The Eastern. More details\r\n\r\n\u0026ndash; A Conversation with Larry David, October 24 2024Talk event at the Fox Theatre. More details\r\n\r\n\u0026ndash; Whitney Cummings: Big Baby, October 25 2024Comedy show at Atlanta Symphony Hall. More details\r\n\r\n\u0026ndash; Midtown Halloween Block Party, October 25 2024Halloween-themed block party and costume contest at Politan Row at Colony Square. More details\r\n\r\n\u0026ndash; Sips Under the Sea Halloween Party: Part I, October 25 2024After-hours Halloween-themed cocktail party at the Georgia Aquarium. Costumes welcome. More details\r\n\r\n\u0026ndash; Benteen Park Fall Fest, October 26 2024Annual fall festival in Benteen Park, with live music, artist market and food vendors. More details\r\n\r\n\u0026ndash; Yappy Hour, October 26 2024Pet-friendly event series at The Hamilton Hotel Alpharetta, with pet adoptions and specialty cocktails. More details\r\n\r\n\u0026ndash; Jerry Seinfeld, October 26 2024Comedy show at the Fox Theatre. More details\r\n\r\n\u0026ndash; Felipe Esparza, October 26 2024Comedy show at Center Stage. More details\r\n\r\n\u0026ndash; Halloween in the Highland, October 26 2024Halloween-themed block party and costume contest in the Virginia-Highland neighborhood. More details\r\n\r\n\u0026ndash; Buckhead Creepy Crawl, October 26 2024Halloween-themed bar crawl and costume contest in the Buckhead entertainment district. More details\r\n\r\n\u0026ndash; Boo at the Zoo, October 19-20, 26-27 2024Annual Halloween event at Zoo Atlanta, with trick or treating, Halloween-themed activities, and more. Costumes welcome. More details\r\n\r\n\u0026ndash; Air Dot Show Tour, October 26-27 2024Air show at Atlanta Regional Airport, Peachtree City. More details\r\n\r\n\u0026ndash; ONE Musicfest, October 26-27 2024Big annual festival of urban progressive music, in Piedmont Park. More details\r\n\r\n\u0026ndash; Menopause The Musical 2: Cruising Through \u0026lsquo;The Change\u0026rsquo;, October 26-27 2024Musical at the Gas South Theater. More details\r\n\r\n\u0026ndash; CompassionCon, October 27 2024Annual community wellness festival in Legacy Park, Decatur, with art, music, wellness activities, health services, food trucks and more. More details\r\n\r\n\u0026ndash; Hillary Clinton Live, October 27 2024Author talk at the Fox Theatre. More details\r\n\r\n## Events From October 28 To THe Weekend Of November 2-3 2024\r\n\r\n\u0026rarr; For concerts this week, see Atlanta concerts in October 2024\r\n\r\n\u0026ndash; Am\u0026eacute;lie: The Musical, October 4 - November 3 2024Musical at Horizon Theatre. More details\r\n\r\n\u0026ndash; ASC Presents Havoc Movement\u0026rsquo;s Dracula: The Failings of Men, October 5 - November 3 2024Performance at the Shakespeare Tavern Playhouse. More details\r\n\r\n\u0026ndash; Historical Marker Dedication: Booker T Washington High School, October 29 2024Historical marker dedication by the Georgia Historical Society at Atlanta\u0026rsquo;s first public secondary school for Black students, at Booker T Washington High School, 45 Whitehouse Drive SW. More details\r\n\r\n\u0026ndash; Sips Under the Sea Halloween Party: Part II, October 31 2024After-hours Halloween-themed cocktail party at the Georgia Aquarium. Costumes welcome. More details\r\n\r\n\u0026ndash; Native American Festival and Pow Wow, October 31 - November 3 2024Big annual festival of Native American heritage at Stone Mountain Park, with live traditional dance, music and storytelling performances, craft demonstrations, wildlife presentations, arts and crafts and more. More details\r\n\r\n\u0026ndash; Two Hot Takes Podcast: No Takes Left Behind, November 1 2024Podcast live show at the Buckhead Theatre. More details\r\n\r\n\u0026ndash; Black Girls Rock!, November 1 2024Annual awards ceremony at the Fox Theatre celebrating the accomplishments of Black women. More details\r\n\r\n\u0026ndash; Ghostbusters in Concert, November 1-2 2024Film screening with live orchestral accompaniment at Atlanta Symphony Hall. More details\r\n\r\n\u0026ndash; Party With Impact \u0026ndash; Positive Impact Health Centers, November 1-2 2024Comedy show at the Gas South Theater to benefit LGBTQ\u002B healthcare services in Atlanta. More details\r\n\r\n\u0026ndash; Lakewood Antiques Market, November 1-3 2024Big monthly antiques and collectibles market in Cumming. More details\r\n\r\n\u0026ndash; Rescue Dog Games: Howl-o-Ween Festival, November 2 2024Annual pet-friendly festival in Piedmont Park, with dog activities, dog costume parade and contest, pet-centric vendors, live music, food and beer, adoptable pets, and more. More details\r\n\r\n\u0026ndash; Chomp and Stomp, November 2 2024Annual neighborhood festival in Cabbagetown, with chili, beer, bluegrass music and more, to benefit the Friends of Cabbagetown Conservancy. More details\r\n\r\n\u0026ndash; D\u0026iacute;a de los Muertos Festival, November 2 2024Free family festival at Woodruff Arts Center, with live music, mariachi band, vendors, food trucks, children\u0026rsquo;s activities and more. More details\r\n\r\n\u0026ndash; Suwanee Wine Fest, November 2 2024Annual festival in Suwanee Town Center Park, with wine samples from across the world, Georgia wines, live music, arts and crafts vendors, food and more. More details\r\n\r\n\u0026ndash; \u0026ldquo;Honoring Our Past Through Stories\u0026rdquo; Historic Landscape Preservation Grant Fund Benefit, November 2 2024Outdoor event at the Marietta Educational Garden Center to benefit the Garden Club of Georgia\u0026rsquo;s Historic Landscape Preservation Grant, with speakers, plein air painters, exhibits, music and more. More details\r\n\r\n\u0026ndash; Chastain Park Arts Festival, November 2-3 2024Popular twice-annual arts festival at Chastain Park, with 175\u002B arts and crafts vendors, food trucks, local musicians, children\u0026rsquo;s activities and more. More details\r\n\r\n\u0026ndash; The Elegant Elf Marketplace, November 2-3 2024Annual holiday marketplace with 80\u002B vendors at the Sandy Springs Performing Arts \u0026amp; Conference Center, to benefit The Sandy Springs Society. More details\r\n\r\n\u0026ndash; Jeff Arcuri: The Full Beans Fall Tour, November 3 2024Comedy show at The Tabernacle. More details\r\n\r\n\u0026ndash; Disney Jr Live On Tour: Let\u0026rsquo;s Play, November 3 2024Children\u0026rsquo;s performance at the Fox Theatre. More details\r\n\r\n\u0026rarr; See November events\r\n\r\n\u0026copy; Melanie K Jones 2024\r\n\r\n* Privacy\r\n* Terms\r\n* Contact\r\n\r\n"
```

Here's a comprehensive list of events happening in Atlanta over the next four weekends:

### Weekend of September 28-29, 2024:
1. **Atlanta Food & Wine Festival** - A premier event featuring more than 200 chefs, sommeliers, and mixologists.
2. **Sweet Auburn Music Fest** - A free two-day music festival with diverse performances and food vendors.
3. **Tunes from the Tombs** - A music festival at Oakland Cemetery, combining with the Victorian fall festival.

For more details, visit [EventCartel](https://eventcartel.com/events/all-categories/atlanta/from_2024-09-28_to_2024-09-29).

### Weekend of October 5-6, 2024:
1. **Festival on Ponce** - An arts festival in Olmsted Linear Park, featuring over 125 vendors.
2. **Candler Park Fall Fest** - A community festival with live music, art, and food.
3. **Atlanta Arab Festival** - Celebrating Arabic food and culture at the Alif Institute.

For more details, visit [Go South Atlanta](https://gosouthatlanta.com/things-to-do/events/atlanta-in-october.html).

### Weekend of October 12-13, 2024:
1. **Atlanta Pride Festival** - One of the largest Pride festivals in the country, held at Piedmont Park.
2. **Chalktoberfest** - An art and beer festival in Marietta Square, featuring professional chalk artists and craft beers.
3. **Taste & Brews Fall Festival** - A food and beer festival in Canton, with southern food, musicians, and a farmers market.

For more details, visit [Go South Atlanta](https://gosouthatlanta.com/things-to-do/events/atlanta-in-october.html).

### Weekend of October 19-20, 2024:
1. **Little Five Points Halloween Festival and Parade** - A popular Halloween celebration with live music, a parade, and a haunted house.
2. **Johns Creek Arts Festival** - An annual arts and crafts festival featuring 140 artisans.
3. **Boo at the Zoo** - A Halloween event at Zoo Atlanta with trick-or-treating and themed activities.

For more details, visit [Go South Atlanta](https://gosouthatlanta.com/things-to-do/events/atlanta-in-october.html).

Enjoy your weekends in Atlanta!

![Events in Atlanta](https://www.bing.com/th?id=OIP.Ctt7cutpo0kduVop1QcE-AAAAA&pid=Api)

For more events and updates, you can explore:
- [Atlanta GA Events](https://atlanta-ga.events/september/)
- [Discover Atlanta](https://discoveratlanta.com/events/main/)


In [7]:
using Microsoft.DotNet.Interactive;
using Microsoft.DotNet.Interactive.Formatting;

output2.LastMessage.DisplayAs("text/markdown");


Here's a comprehensive list of events happening in Atlanta over the next four weekends:

### Weekend of September 28-29, 2024:
1. **Atlanta Food & Wine Festival** - A premier event featuring more than 200 chefs, sommeliers, and mixologists.
2. **Sweet Auburn Music Fest** - A free two-day music festival with diverse performances and food vendors.
3. **Tunes from the Tombs** - A music festival at Oakland Cemetery, combining with the Victorian fall festival.

For more details, visit [EventCartel](https://eventcartel.com/events/all-categories/atlanta/from_2024-09-28_to_2024-09-29).

### Weekend of October 5-6, 2024:
1. **Festival on Ponce** - An arts festival in Olmsted Linear Park, featuring over 125 vendors.
2. **Candler Park Fall Fest** - A community festival with live music, art, and food.
3. **Atlanta Arab Festival** - Celebrating Arabic food and culture at the Alif Institute.

For more details, visit [Go South Atlanta](https://gosouthatlanta.com/things-to-do/events/atlanta-in-october.html).

### Weekend of October 12-13, 2024:
1. **Atlanta Pride Festival** - One of the largest Pride festivals in the country, held at Piedmont Park.
2. **Chalktoberfest** - An art and beer festival in Marietta Square, featuring professional chalk artists and craft beers.
3. **Taste & Brews Fall Festival** - A food and beer festival in Canton, with southern food, musicians, and a farmers market.

For more details, visit [Go South Atlanta](https://gosouthatlanta.com/things-to-do/events/atlanta-in-october.html).

### Weekend of October 19-20, 2024:
1. **Little Five Points Halloween Festival and Parade** - A popular Halloween celebration with live music, a parade, and a haunted house.
2. **Johns Creek Arts Festival** - An annual arts and crafts festival featuring 140 artisans.
3. **Boo at the Zoo** - A Halloween event at Zoo Atlanta with trick-or-treating and themed activities.

For more details, visit [Go South Atlanta](https://gosouthatlanta.com/things-to-do/events/atlanta-in-october.html).

Enjoy your weekends in Atlanta!

![Events in Atlanta](https://www.bing.com/th?id=OIP.Ctt7cutpo0kduVop1QcE-AAAAA&pid=Api)

For more events and updates, you can explore:
- [Atlanta GA Events](https://atlanta-ga.events/september/)
- [Discover Atlanta](https://discoveratlanta.com/events/main/)

## Clean Up

In [8]:
var assistant = (await client.AssistantList())?.Data?.FirstOrDefault(o => o.Name == "Web Search and Scrape");
if(assistant != null) {
    await client.AssistantDelete(assistant.Id);
    Console.WriteLine("Deleted assistant");
}
else
{
    Console.WriteLine("Didn't find Web Search and Scrape");
}

Deleted assistant
